# Requirements

In [ ]:
import os, sys

path_project= "D:\\Projects\\SystemTrading\\JnQ\\"
sys.path.append(path_project)

from Bank.main import *  # run main.py (load Bank)

# Alertion

## Indicator

### II

In [ ]:
bank.chat_id = "5320962614"
bank.token = "7156517117:AAF_Qsz3pPTHSHWY-ZKb8LSKP6RyHSMTupo"

bank.get_messenger_bot(bank)

In [ ]:
bank.push_msg(bank, 'test')

In [ ]:
from funcs.binance.futures_concat_candlestick_ftr_v2 import concat_candlestick_v2

In [ ]:
days = 1  # 330
end_date = None  # "2023-01-06" "2021-04-12" "2021-03-23"
# end_date = "2023-12-20"
intervals = ['1m']  # ['1m', '3m', '5m', '15m', '30m', '1h', '4h'] - old
limit = 1500

# bank.symbol = 'THETAUSDT'
symbols = ['THETAUSDT', 'BICOUSDT']
# bank_exchange_info = bank.exchange_info()
# symbols = [data['symbol'] for data in bank.exchange_info()['symbols'] if 'USDT' in data['symbol'] if '_' not in data['symbol']]

while 1:
    for bank.symbol in symbols:        
        bank.df_res, end_date = concat_candlestick_v2(bank.symbol,
                                                      '1m',
                                                      days,
                                                      limit=limit,
                                                      end_date=end_date,
                                                      show_process=True,
                                                      timesleep=0.2)
        # signal
            # get II
        bank.df_res = get_II(bank.df_res)
        display(bank.df_res.tail())
        
            # open
        if bank.df_res.iiSource[-2] > 0 > bank.df_res.iiSource[-3]:
            msg = "signal : {} {}".format(bank.symbol, 'OPEN')
            bank.push_msg(bank, msg)
            # close
        elif bank.df_res.iiSource[-2] < 0 < bank.df_res.iiSource[-3]:
            msg = "signal : {} {}".format(bank.symbol, 'CLOSE')
            bank.push_msg(bank, msg)
    
        time.sleep(0.1)  
        
    time.sleep(30)
    clear_output()    

In [ ]:
get_II(bank.df_res).tail()
# len(get_II(bank.df_res))

#### get_id_cols

In [ ]:
def get_id_cols(df_chunk):
    
    df_chunk['idVolume'] =  df_chunk.volume.cumsum()
    df_chunk['idRangeH'] =  df_chunk.high.cummax()
    df_chunk['idRangeL'] =  df_chunk.low.cummin()

    return df_chunk

#### get_II

In [ ]:
def get_II(df, period=21):
    
    idx_startDay = df.index.map(lambda x : '09:00:00' in str(x))

    idx_startDay_arr = np.argwhere(idx_startDay)
    idx_startDay_arr = np.insert(idx_startDay_arr, 0, 0)
    idx_startDay_arr = np.append(idx_startDay_arr, len(df))
    
    df_chunks = [get_id_cols(df.iloc[idx_startDay_arr[n]:idx_startDay_arr[n + 1]]) for n in range(len(idx_startDay_arr)-1)]
    df_res_id = pd.concat(df_chunks)
    df_res_id_arr = df_res_id.to_numpy()
    
    idx_col_close = df_res_id.columns.get_loc('close')
    idx_col_idRangeH = df_res_id.columns.get_loc('idRangeH')
    idx_col_idRangeL = df_res_id.columns.get_loc('idRangeL')
    idx_col_idVolume = df_res_id.columns.get_loc('idVolume')
    
    #  # 357 µs ± 45.8
    # %timeit -n1 -r100 df_res_id['iiiValue'] = (2 * df_res_id['close'] - df_res_id['idRangeH'] - df_res_id['idRangeL']) / (df_res_id['idRangeH'] - df_res_id['idRangeL']) * df_res_id['idVolume']
    # # 39.7 µs ± 13.8 µs per loop
    # %timeit -n1 -r100 df_res_id['iiiValue'] = (2 * df_res_id_arr[:, idx_col_close] - df_res_id_arr[:, idx_col_idRangeH] - df_res_id_arr[:, idx_col_idRangeL]) / (df_res_id_arr[:, idx_col_idRangeH] - df_res_id_arr[:, idx_col_idRangeL]) * df_res_id_arr[:, idx_col_idVolume]
    
    # # 381 µs ± 31.7 µs per loop
    # %timeit -n1 -r100 df_res_id['iiSource'] = df_res_id.iiiValue.rolling(21).sum().to_numpy() / df_res_id.idVolume.rolling(21).sum().to_numpy() * 100
    # # 471 µs ± 52.8 µs per loop
    # %timeit -n1 -r100 df_res_id['iiSource'] = df_res_id.iiiValue.rolling(21).sum() / df_res_id.idVolume.rolling(21).sum() * 100
    
    df_res_id['iiiValue'] = (2 * df_res_id_arr[:, idx_col_close] - df_res_id_arr[:, idx_col_idRangeH] - df_res_id_arr[:, idx_col_idRangeL]) / (df_res_id_arr[:, idx_col_idRangeH] - df_res_id_arr[:, idx_col_idRangeL]) * df_res_id_arr[:, idx_col_idVolume]
    df_res_id['iiSource'] = df_res_id.iiiValue.rolling(period).sum().to_numpy() / df_res_id.idVolume.rolling(period).sum().to_numpy() * 100

    return df_res_id

# Pipeline

## configure

In [ ]:

param_dict = \
{
  "selection_id": "1",
  "show_detail": 1,
  "trader_set": {
    "backtrade": 0,
    # "back_data_path": "D:\\Projects\\SystemTrading\\JnQ\\database\\binance\\cum\\2023-01-12\\2023-01-12 ETHUSDT_1m.ftr",
    # "start_datetime": "2022-10-05 00:00:59.999",
    # "ticker_list_path": "D:\\Projects\\SystemTrading\\JnQ\\Bank\\tickers\\binance_20240102.pkl",
    "run": 1,
    # "df_log": 0,
    "show_detail": 0,
    "latest_index": -1,
    "complete_index": -2,
    "fee_limit": 0.0002,
    "fee_market": 0.0004,
    "initial_asset": 41,
    "income_acc": 0.0,
    "acc_profit_prod": 1,
    "acc_profit_sum": 0,
    "profit_mode": "PROD",
    "asset_changed": 0,
    "messenger_on": 1,
    "itv_list": "['T', '3T', '5T', '15T', '30T', 'H', '4H']",
    "row_list": "[500, 1, 1, 1, 1, 1, 1]",
    "rec_row_list": "[1, 1, 1, 1, 1, 1, 1]",
    "offset_list": "['1h', '1h', '1h', '1h', '1h', '1h', '1h']",
    "get_df_new_timeout": 1,
    "loop_duration": 3.6,
    "realtime_term": 0.2,
    "api_term": 3,
    "order_term": 0.5,
    "market_check_term": 5,
    "exec_open_check_term": 5,
    "quantity_open_exec_ratio": 0.97,
    "tp_exec_check_term": 5
  },
  "pos_set": {
    "short_inversion": 0,
    "long_inversion": 0,
    "short_ban": 0,
    "long_ban": 0,
    "short_fake": 0,
    "long_fake": 0
  },
  "loc_set": {
    "point1": {
      "exp_itv": "5T",
      "tf_entry": "15T",
      "candle_pattern": "CDLMARUBOZU",
    },
    "point2": {
      "wrr_32_min": "None",
      "wrr_32_max": "None",
      "csdbox_range": 0.3,
      "tr_thresh_short": "None",
      "tr_thresh_long": "None",
      "csd_period": "None"
    },
    "zone1": {
      "use_zone": 0,
      "base_roll_period": 50,
      "degree_list": "[]",
      "dtk_itv": "5T",
      "dt_k": "None",
      "dc_period": 135,
      "use_dtk_line": 0,
      "zone_dt_k": 0.4,
      "zone_dc_period": 135
    },
    "zone2": {
      "use_zone": 0
    }
  },
  "tr_set": {
    "check_hlm": 0,
    "wave_itv1": "T",
    "wave_itv2": "T",
    "wave_period1": 20,
    "wave_period2": 20,
    "wave_length_max_short1": "None",
    "wave_length_max_long1": "None",
    "wave_length_min_short1": "None",
    "wave_length_min_long1": "None",
    "wave_spread1": "None",
    "wave_time_ratio1": "None",
    "tc_period": 20,
    "wave_greater1": 0,
    "wave_greater2": 0,
    "wave_lesser1": 2,
    "wave_lesser2": 2,
    "expire_k1": 0.0,
    "expire_k2": 0.0,
    "expire_tick": "None",
    "p2_box_k1": 0,
    "p2_box_k2": 0,
    "p1p2_low": 0.0,
    "tp_gap": 0.0,
    "ep1_gap": 0.7,
    "ep2_gap": 0.3,
    "out_gap": 0,
    "bias_tick": 100
  },
  "ep_set": {
    "entry_type": "LIMIT",
    "static_ep": 1,
    "point2": {
      "entry_type": "LIMIT"
    }
  },
  "tp_set": {
    "non_tp": 0,
    "static_tp": 1,
    "tp_onexec": 0,
    "decay_term": 60,
    "partial_ranges": "[1]",
    "partial_quantity_ratio": "[1]"
  },
  "out_set": {
    "non_out": 0,
    "hl_out": 1,
    "static_out": 1,
    "out_onexec": 0,
    "tf_exit": "None",
    "fisher_exit": "None",
    "rsi_exit": 0,
    "cci_exit": 0
  },
  "lvrg_set": {
    "static_lvrg_short": 1,
    "static_lvrg_long": 1,
    "limit_leverage": "None",
    "leverage": 1.0,
    "target_pct": 0.05,
    "allow_float": 0,
    "lvrg_rejection": 1,
    "multiplier": 1000,
    "log_base": 30
  }
}

## Bank

In [ ]:
class Bank(UMFutures):

    """
    v4.0 --> v4.1    
        allow server_time.
        remove read_and_write_config
        
    last confirmed at, 20240504 2306
    """

    def __init__(self, **kwargs):
        
        api_key, secret_key = self.load_key(kwargs['path_api'])
        
        UMFutures.__init__(self, key=api_key, secret=secret_key)

        self.websocket_client = UMFuturesWebsocketClient()
        self.websocket_client.start()
        self.price_market = {}
        
        self.save_path_log = kwargs['save_path_log']
        self.set_logger()
        
        self.path_config = kwargs['path_config']
        with open(self.path_config, 'r') as f:
            self.config = EasyDict(json.load(f))
        
        # add messegner
        self.msg_bot = None
        
        
        # inital set.
            # balance
        self.balance_available = self.config.trader_set.initial_asset
        self.balance_over = None
        self.balance_min = 5.0  # USDT, not used in Stock.

            # income
        self.income = 0.0
        self.income_accumulated = self.config.trader_set.income_acc

            # profit
        if self.config.trader_set.profit_mode == "SUM":
            self.profit_real_accumulated = self.config.trader_set.acc_profit_sum
        else:
            self.profit_real_accumulated = self.config.trader_set.acc_profit_prod
        self.profit_ideal_accumulated = self.profit_real_accumulated
        
        
    @staticmethod
    def load_key(key_abspath):
        with open(key_abspath, 'rb') as f:
            return pickle.load(f)            

    
    @staticmethod
    def get_precision_by_price(price):
        try:
            precision = len(str(price).split('.')[1])
        except Exception as e:
            precision = 0
        return precision

    @staticmethod
    def calc_with_precision(data, precision_data, def_type='floor'):

        if not pd.isna(data):
            if precision_data > 0:
                if def_type == 'floor':
                    data = math.floor(data * (10 ** precision_data)) / (10 ** precision_data)
                elif def_type == 'round':
                    data = float(round(data, precision_data))
                else:
                    data = math.ceil(data * (10 ** precision_data)) / (10 ** precision_data)
            else:
                data = int(data)

        return data


    def agg_trade_message_handler(self, message):
        """
        1. websocket streaming method 를 이용한 get_price_realtime method.
            a. try --> received data = None 일 경우를 대비한다.
        """
        try:
            self.price_market[message['s']] = float(message['p'])
        except Exception as e:
            pass

    def set_logger(self,):
        
        simple_formatter = logging.Formatter("[%(name)s] %(message)s")
        complex_formatter = logging.Formatter("%(asctime)s %(levelname)s [%(name)s] [%(filename)s:%(lineno)d] - %(message)s")
        
        console_handler = logging.StreamHandler()
        console_handler.setFormatter(simple_formatter)
        console_handler.setLevel(logging.DEBUG)
    
        file_handler = logging.FileHandler(self.save_path_log)
        file_handler.setFormatter(complex_formatter)
        file_handler.setLevel(logging.DEBUG)
    
        self.sys_log = logging.getLogger('Bank')
    
        self.sys_log.handlers.clear()
        self.sys_log.addHandler(console_handler)
        self.sys_log.addHandler(file_handler)
        self.sys_log.setLevel(logging.DEBUG)
        

### Test

In [ ]:
# bank = Bank(path_config="D:\Projects\SystemTrading\JnQ\\funcs\\config.json", 
#                 path_api=r"D:\Projects\SystemTrading\JnQ\Bank\api_keys\binance_mademerich.pkl",
#                 save_path_log='./20240504.log', 
#                 receive_limit_ms=1000*3600)

bank.path_dir_df_res = "D:\Projects\SystemTrading\JnQ\Bank\data\df_res"
bank.path_table_condition = "D:\Projects\SystemTrading\JnQ\Bank\data\\table\\table_condition.xlsx"
bank.path_table_trade = "D:\Projects\SystemTrading\JnQ\Bank\data\\table\\table_trade.ftr"
bank.path_table_log = "D:\Projects\SystemTrading\JnQ\Bank\data\\table\\table_log.xlsx"

In [ ]:
bank.echo = echo
bank.get_messenger_bot = get_messenger_bot
bank.push_msg = push_msg

bank.get_tables = get_tables
bank.set_tables = set_tables


# add funcs.      
bank.get_streamer = get_streamer 
bank.get_df_new_by_streamer = get_df_new_by_streamer
bank.get_df_new = get_df_new

bank.get_df_mtf = get_df_mtf
bank.get_wave_info = get_wave_info

bank.set_price_and_open_signal = set_price_and_open_signal
bank.set_price_box = set_price_box
bank.set_price = set_price
bank.get_open_res = get_open_res
bank.adj_wave_point = adj_wave_point
bank.adj_wave_update_hl_rejection = adj_wave_update_hl_rejection
bank.validate_price = validate_price
bank.get_reward_risk_ratio = get_reward_risk_ratio

bank.get_side_open = get_side_open
bank.entry_point_location = entry_point_location
bank.entry_point2_location = entry_point2_location



bank.get_order_info = get_order_info
bank.get_balance_available = get_balance_available
bank.get_balance_info = get_balance_info
bank.get_price = get_price
bank.get_precision = get_precision
bank.get_price_realtime = get_price_realtime
bank.get_price_liquidation = get_price_liquidation
bank.get_price_expiration = get_price_expiration

bank.get_leverage_limit = get_leverage_limit
bank.get_leverage = get_leverage
bank.set_leverage = set_leverage
bank.set_position_mode = set_position_mode
bank.set_margin_type = set_margin_type


bank.check_stop_loss = check_stop_loss
bank.check_stop_loss_onbarclose = check_stop_loss_onbarclose
bank.check_stop_loss_by_signal = check_stop_loss_by_signal

bank.order_limit = order_limit
bank.order_market = order_market
bank.order_cancel = order_cancel

bank.get_income_info = get_income_info


#### Main

In [ ]:
bank.get_tables(bank)
# bank.idx_prev = -1

# allow only once.
# bank.get_messenger_bot(bank)

# display(bank.table_condition.head())
# display(bank.table_trade.head())
# display(bank.table_log.head())

# bank.set_tables(bank)

In [ ]:
bank.symbol = 'THETAUSDT'
bank.symbol = 'ETHUSDT'
bank.orderId = 13055122537
bank.orderId = 8389765678433432873 
# bank.orderId = 8389765678433382758 

bank.get_order_info(bank)

In [ ]:
bank.table_log

In [ ]:
bank.symbol = 'THETAUSDT'
bank.get_income_info(bank)

In [ ]:

# bank.websocket_client.agg_trade(symbol=bank.symbol, id=1, callback=bank.agg_trade_message_handler)
bank.price_market


In [ ]:


while 1:
    
    loop_table_condition(bank) # allow static time for this.    
    loop_table_trade(bank) # allow static time for this.
    
    # bank.table_trade.reset_index(drop=True, inplace=True) # for index row by 'at', we do not reset_index.
    bank.table_log.reset_index(drop=True, inplace=True) # for index row by 'at'    
    bank.set_tables(bank)
    
    display(bank.table_trade)
    display(bank.table_log)
    
    time.sleep(5)    
    clear_output(wait=True)
    


In [ ]:
# bank.symbol = 'THETAUSDT'
bank.symbol = 'ETHUSDT'


In [ ]:
os.getcwd()

In [ ]:
bank.table_log
# # bank.symbol = '1INCHUSDT'
# # print(bank.symbol)

table_log_by_symbol = bank.table_log[bank.table_log.symbol == bank.symbol]
table_log_valid = table_log_by_symbol[(table_log_by_symbol.status == 'NEW') & (table_log_by_symbol.order_way == 'OPEN')]
# table_log_by_symbol = bank.table_log[bank.table_log.symbol == bank.symbol]
# table_log_valid = table_log_by_symbol[(table_log_by_symbol.status == 'NEW') & (table_log_by_symbol.order_way == 'OPEN')]
# # len(table_log_valid)
table_log_valid
# # table_log_valid.iloc[-1].statusChangeTime
# int(time.time()) - table_log_valid.iloc[-1].statusChangeTime
# if int(time.time()) - table_log_valid.iloc[-1].statusChangeTime > 60:

In [ ]:
df = pd.DataFrame({'a':np.random.randn(10), 'b':np.random.randn(10)})

In [ ]:
for index, row in enumerate(df.iterrows()): # .iterrows(): # copy().
# for index, row in enumerate(df): # .iterrows(): # copy().
    print(index, row)

In [ ]:
for index, row in df.iterrows(): # copy().
    print(index)
    if row['a'] > 0:
        df.drop([index], inplace=True)
        
        df.reset_index(drop=True, inplace=True) # for index row by 'at'

#### loop_table_condition

In [ ]:
def loop_table_condition(self, ):

    """
    v1.0
        modify to data_table mode.
        update with one trade in a minute.

    last confirmed at, 20240505 1025.
    """

    start_time = time.time()
    for idx, row in self.table_condition.iterrows(): # idx can be used, cause table_condition's index are not reset.

        # init.        
        self.table_condition_row = row
        self.symbol = row.symbol
        
        # get idx_prev for continuation.
            # we have time limit for each loop, so the start point of loop (symbol) should be renewed.
        if not self.config.trader_set.backtrade:            
            if idx <= self.idx_prev: # min idx = 0, so if idx_prev = -1, it means all row rotated.
                continue
            else:
                # if idx reaches the last, reset.
                if idx == len(self.table_condition) - 1:
                    self.idx_prev = -1
                else:
                    self.idx_prev = idx                    

            # b. maximum residential duration for loop_table_condition.
            if time.time() - start_time > self.config.trader_set.loop_duration:
                self.sys_log.warning("LoopTableCondition : loop_duration break.")
                break
        

        # get df_res
        self.sys_log.debug("------------------------------------------------")
        start_time = time.time()
        if self.config.trader_set.backtrade:
            # self.get_streamer(self)
            self.get_df_new_by_streamer(self)
        else:
            self.get_df_new(self)

            # prevent delaying by last_index_watch validation from loop_duration.
            if time.time() - start_time > self.config.trader_set.loop_duration:
                start_time = time.time()

        if self.df_res is None:
            continue          
        self.sys_log.info("LoopTableCondition : check df_res last row timestamp : {}".format(self.df_res.index[-1]))
        self.sys_log.debug("LoopTableCondition : elasped time, get_df_new : {:.2f}s".format(time.time() - start_time))
        self.sys_log.debug("------------------------------------------------")
 
            

        # set_price_and_open_signal
        self.sys_log.debug("------------------------------------------------")
        start_time = time.time()
        try:                  
            
            self.get_df_mtf(self) # function usage format maintenance
            self.sys_log.debug("LoopTableCondition : elasped time, sync_check : %.4fs" % (time.time() - start_time))
            self.get_wave_info(self)
            self.sys_log.debug("LoopTableCondition : elasped time, + public_indi : %.4fs" % (time.time() - start_time))
            self.set_price_and_open_signal(self, mode='OPEN', env='BANK') # env select not to use adj_price_unit().
            self.sys_log.debug("LoopTableCondition : elasped time, + set_price_and_open_signal : %.4fs" % (time.time() - start_time)) 
            
        except Exception as e:
            msg = "error in set_price_and_open_signal phase : {}".format(e)
            self.sys_log.error(msg)
            self.push_msg(self, msg)
            continue
        self.sys_log.debug("------------------------------------------------")

        
        # get side_open by table_conditions.
        self.sys_log.debug("------------------------------------------------")
        start_time = time.time()
        try:
            self.get_side_open(self, open_num=1)
        except Exception as e:
            msg = "error in get_side_open : {}".format(e)
            self.sys_log.error(msg)
            self.push_msg(self, msg)
            continue
        self.sys_log.debug("LoopTableCondition : elasped time, get_side_open : %.4fs" % (time.time() - start_time)) 
        self.sys_log.debug("------------------------------------------------")

        
        if self.side_open is not None:

            set_code = 0 

            # one trade in a minute.
                # order OPEN can be excue
            table_log_by_symbol = self.table_log[self.table_log.symbol == self.symbol]
            table_log_valid = table_log_by_symbol[(table_log_by_symbol.status == 'NEW') & (table_log_by_symbol.order_way == 'OPEN')]
            self.sys_log.debug("table_log_valid : {}".format(table_log_valid))

            if len(table_log_valid) > 0:
                if int(time.time()) - table_log_valid.iloc[-1].statusChangeTime > 60:
                    set_code = 1
            else:
                set_code = 1

            if set_code:
                # set symbol & code.
                self.code = "{}_{}".format(row.symbol, str(int(time.time())))
                init_table_trade(self) 

    
        # API term for consecutive request. (just for realtrade)
        if not self.config.trader_set.backtrade:
            time.sleep(self.config.trader_set.realtime_term)

#### init_table_trade

In [ ]:

def init_table_trade(self, ):

    """
    v1.0
    
    last confirmed at, 20240422 1116
    """
        
    # param init.
    # get prices for order.
    self.sys_log.debug("------------------------------------------------")
    start_time = time.time()    
    self.order_motion = 0  # order 하는 것처럼 시간을 소비하기 위함임.
    self.get_price(self)
    self.sys_log.debug("InitTableTrade : elasped time, get_price : %.4fs" % (time.time() - start_time)) 
    self.sys_log.debug("------------------------------------------------")

    
    # get balance
        # Stock 특성상, balance_min = self.price_entry 로 설정함.
        # open_data_dict.pop 은 watch_dict 의 일회성 / 영구성과 무관하다.    
    self.sys_log.debug("------------------------------------------------")
    start_time = time.time()    
    self.get_balance_info(self, balance_min=10)   
    self.sys_log.debug("InitTableTrade : elasped time, get_balance_info : %.4fs" % (time.time() - start_time)) 
    self.sys_log.debug("------------------------------------------------")

    
    # get
        # self.side_close
        # self.side_position
        # self.price_entry open execution, 
    self.sys_log.debug("------------------------------------------------")
    start_time = time.time()    
    self.price_open = self.df_res['open'].to_numpy()[-1]  # open 은 latest_index 의 open 사용
    if self.side_open == OrderSide.BUY:
        self.side_close = OrderSide.SELL
        self.side_position = PositionSide.LONG
        if self.config.pos_set.long_fake:
            self.order_motion = 1
        self.price_entry = min(self.price_open, self.price_entry)
    else:
        self.side_close = OrderSide.BUY
        self.side_position = PositionSide.SHORT
        if self.config.pos_set.short_fake:
            self.order_motion = 1
        self.price_entry = max(self.price_open, self.price_entry)
    self.sys_log.debug("InitTableTrade : elasped time, side_close, side_position, price_entry open execution : %.4fs" % (time.time() - start_time)) 
    self.sys_log.debug("------------------------------------------------")
        

    # get leverage
    self.sys_log.debug("------------------------------------------------")
    start_time = time.time()    
    bank.get_leverage_limit(bank)
    self.get_leverage(self)
    if self.leverage is None:
        self.sys_log.warning("leverage is None : {}".format(self.leverage))
        return
    self.sys_log.debug("InitTableTrade : elasped time, get_leverage : %.4fs" % (time.time() - start_time)) 
    self.sys_log.debug("------------------------------------------------")

     
    # get price_liquidation
    self.sys_log.debug("------------------------------------------------")
    start_time = time.time()   
    self.get_price_liquidation(self)
    self.sys_log.debug("InitTableTrade : elasped time, get_price_liquidation : %.4fs" % (time.time() - start_time)) 
    self.sys_log.debug("------------------------------------------------")
      
    # get price_expiration
    self.sys_log.debug("------------------------------------------------")
    start_time = time.time()  
    self.get_price_expiration(self)
    self.sys_log.debug("InitTableTrade : elasped time, get_price_expiration : %.4fs" % (time.time() - start_time)) 
    self.sys_log.debug("------------------------------------------------")

    
    # adj. precision_price & quantity
    self.sys_log.debug("------------------------------------------------")
    start_time = time.time()  
    self.get_precision(self)
    self.price_take_profit, self.price_entry, self.price_stop_loss, self.price_liquidation = \
    [self.calc_with_precision(price_, self.precision_price) for price_ in [self.price_take_profit, self.price_entry, self.price_stop_loss, self.price_liquidation]]  # includes half-dynamic self.price_take_profit
    self.sys_log.debug("InitTableTrade : elasped time, precision_price & quantity : %.4fs" % (time.time() - start_time)) 
    self.sys_log.debug("------------------------------------------------")
    
    # set leverage
    self.sys_log.debug("------------------------------------------------")
    start_time = time.time()  
    if not self.config.trader_set.backtrade:
        # [ RealTrade ]
        self.set_leverage(self)      
    self.sys_log.debug("InitTableTrade : elasped time, set_leverage : %.4fs" % (time.time() - start_time)) 
    self.sys_log.debug("------------------------------------------------")
    
    # set leverage
    self.sys_log.debug("------------------------------------------------")
    start_time = time.time()  
    if not self.config.trader_set.backtrade:
        # [ RealTrade ]
        self.set_leverage(self)      
    self.sys_log.debug("InitTableTrade : elasped time, set_leverage : %.4fs" % (time.time() - start_time)) 
    self.sys_log.debug("------------------------------------------------")

    
    # get quantity_open
    self.quantity_open = self.calc_with_precision(self.balance_available / self.price_entry * self.leverage, self.precision_quantity)
    
    self.sys_log.info("price_take_profit : {}".format(self.price_take_profit))
    self.sys_log.info("price_entry : {}".format(self.price_entry))
    self.sys_log.info("price_stop_loss : {}".format(self.price_stop_loss))
    self.sys_log.info("price_liquidation : {}".format(self.price_liquidation))
    self.sys_log.info("price_expiration : {}".format(self.price_expiration))
    self.sys_log.info("leverage : {}".format(self.leverage))
    self.sys_log.info("balance_available : {}".format(self.balance_available))
    self.sys_log.info("quantity_open : {}".format(self.quantity_open))
    
    # self.sys_log.debug("LoopTableCondition : elapsed time, init_table_trade %.5f" % (time.time() - start_time))
    self.sys_log.debug("------------------------------------------------")

    
    self.sys_log.debug("------------------------------------------------")
    start_time = time.time()  
    table_trade_new_row = pd.DataFrame(data = np.array([np.nan] * len(self.table_trade.columns)).reshape(-1, 1).T, columns=self.table_trade.columns)

    # save df_res.
    self.path_df_res = "{}\\{}.ftr".format(self.path_dir_df_res, self.code)
    self.df_res.reset_index().to_feather(self.path_df_res , compression='lz4')

    # init table_trade.
    table_trade_new_row.code = self.code 
    table_trade_new_row.path_df_res = self.path_df_res 
    table_trade_new_row.order_motion = self.order_motion 
    table_trade_new_row.side_open = self.side_open 
    table_trade_new_row.side_close = self.side_close 
    table_trade_new_row.side_position = self.side_position 
    table_trade_new_row.price_take_profit = self.price_take_profit 
    table_trade_new_row.price_entry = self.price_entry 
    table_trade_new_row.price_stop_loss = self.price_stop_loss 
    table_trade_new_row.price_liquidation = self.price_liquidation 
    table_trade_new_row.price_expiration = self.price_expiration 
    table_trade_new_row.leverage = self.leverage 
    table_trade_new_row.balance_available = self.balance_available 
    table_trade_new_row.quantity_open = self.quantity_open 
    table_trade_new_row.precision_price = self.precision_price 
    table_trade_new_row.precision_quantity = self.precision_quantity 
    
    table_trade_new_row.symbol = self.symbol 

    # self.sys_log.info("table_trade_new_row : \n{}".format(table_trade_new_row.iloc[0]))

    # append row.
    self.table_trade = pd.concat([self.table_trade, table_trade_new_row]).reset_index(drop=True)
    self.sys_log.debug("InitTableTrade : elasped time, append row : %.4fs" % (time.time() - start_time)) 
    self.sys_log.debug("------------------------------------------------")



#### loop_table_trade

##### v3.0

In [ ]:
bank.table_trade # .dtypes

In [278]:

lookback_day = 10
lookback_ts = (time.time() - lookback_hour * 24 * 60 * 60) * 1000 # milli-sec.

symbols = np.unique(bank.table_trade.symbol)
for bank.symbol in symbols:
    
    order_info_server = bank.get_all_orders(symbol=bank.symbol)

    df_order_info_server = pd.DataFrame([order_info for order_info in order_info_server[::-1] if order_info['updateTime'] > lookback_ts])
    df_order_info_alive = df_order_info_server[(df_order_info_server.status == 'NEW') | (df_order_info_server.status == 'PARTIALLY_FILLED')]

    # if len(df_order_info_alive) == 0: # no order_open in server state doesn't matter...
    #     msg = "There are no order_open in server."
    #     bank.push_msg(bank, msg)

    bank.sys_log.debug("------------------------------------------------")
    start_time = time.time()
    for idx, row in df_order_info_alive.iterrows():

        # if table_trade ahs server orderId
        idxes_match = np.where(bank.table_trade.orderId == row.orderId)[0]
        if len(idxes_match) > 0:

            # update table_trade
            for idx_match in idxes_match:
                idx_replace = bank.table_trade.index[idx_match]
                for k, v in dict(row).items():
                    bank.table_trade.at[idx_replace, k] = v
        else:
            msg = "orderId_server not in TableTrade."
            bank.push_msg(bank, msg)
              
    bank.sys_log.debug("LoopTableTrade : elasped time, update table_trade : %.4fs" % (time.time() - start_time)) 
    bank.sys_log.debug("------------------------------------------------")

[Bank] ------------------------------------------------
[Bank] LoopTableTrade : elasped time, update table_trade : 0.0000s
[Bank] ------------------------------------------------


In [271]:
# np.sum(bank.table_trade.orderId == row.orderId)
# np.where(bank.table_trade.orderId == row.orderId)[0]
bank.table_trade.index[1]
# bank.table_trade.at[np.where(bank.table_trade.orderId != row.orderId)[0], 'orderId']

1

In [ ]:
df_order_info_server.dtypes
# bank.table_trade

In [ ]:
table_trade = bank.table_trade.copy()

In [ ]:
bank.table_trade.astype({'orderId': str}) # .dtypes

In [ ]:
table_trade.dtypes

In [ ]:
bank.sys_log.debug("------------------------------------------------")
start_time = time.time()
for k, v in bank.order_info.items():
    bank.table_trade.at[idx, k] = v
bank.sys_log.debug("LoopTableTrade : elasped time, update table_trade : %.4fs" % (time.time() - start_time)) 
bank.sys_log.debug("------------------------------------------------")

In [ ]:
bank.symbol = 'ETHUSDT'
order_info_server = bank.get_all_orders(symbol=bank.symbol)

In [ ]:
pd.DataFrame(order_info_server[::-1])

In [ ]:
df_order_info_server_typechange = pd.DataFrame(order_info_server[::-1]) # .astype({'orderId': str})
df_order_info_server_typechange.to_csv("dtype_test_20240508.csv", index=False) # .dtypes
df_order_info_server_typechange.to_excel("dtype_test_20240508.xlsx", index=False) # .dtypes .astype({'orderId': str})
df_order_info_server_typechange.to_feather("dtype_test_20240508.ftr") # .dtypes .astype({'orderId': str})
# pd.DataFrame(order_info_server[::-1]).astype({'orderId': str}).dtypes # .to_excel("dtype_test_20240508.xlsx", index=False) # 

In [ ]:
dict(df_order_info_server_typechange.dtypes)
# df_order_info_server_typechange.set_index('index')['dtypes'].astype(str).to_dict()

In [251]:
df_order_info_server_typechange.head()

,orderId,symbol,status,clientOrderId,price,avgPrice,origQty,executedQty,cumQuote,timeInForce,type,reduceOnly,closePosition,side,positionSide,stopPrice,workingType,priceMatch,selfTradePreventionMode,goodTillDate,priceProtect,origType,time,updateTime
0,8389765678582623918,ETHUSDT,FILLED,e11K9ZTu9miObQM7uRUSgu,0,3098.76000,0.013,0.013,40.28388,GTC,MARKET,True,False,SELL,LONG,0,CONTRACT_PRICE,NONE,NONE,0,False,MARKET,1714870008994,1714870008994
1,8389765678582623861,ETHUSDT,FILLED,ciiTp3UFMigp9u8kQUOADo,0,3098.76000,0.013,0.013,40.28388,GTC,MARKET,True,False,SELL,LONG,0,CONTRACT_PRICE,NONE,NONE,0,False,MARKET,1714870008914,1714870008914
2,8389765678582623808,ETHUSDT,FILLED,qVdL66y1bfgTgaOdUvRphG,0,3098.76000,0.013,0.013,40.28388,GTC,MARKET,True,False,SELL,LONG,0,CONTRACT_PRICE,NONE,NONE,0,False,MARKET,1714870008836,1714870008836
3,8389765678433432873,ETHUSDT,CANCELED,IMoRUT3mHJndskL1xUNTBX,3167.45,0.00000,0.013,0,0,GTC,LIMIT,True,False,SELL,LONG,0,CONTRACT_PRICE,NONE,NONE,0,False,LIMIT,1714823603756,1714869634911
4,8389765678433382758,ETHUSDT,FILLED,gS9NqTDyytssoJZW1fdVlv,3129.59,3124.18000,0.013,0.013,40.61434,GTC,LIMIT,False,False,BUY,LONG,0,CONTRACT_PRICE,NONE,NONE,0,False,LIMIT,1714823598328,1714823598328


In [253]:
# df_order_info_server_typechange_read = pd.read_excel("dtype_test_20240508.xlsx")#.dtypes
# df_order_info_server_typechange_read = pd.read_csv("dtype_test_20240508.csv", dtype=dict(df_order_info_server_typechange.dtypes))
# df_order_info_server_typechange_read = pd.read_csv("dtype_test_20240508.csv") # , dtype=dict(df_order_info_server_typechange.dtypes))
df_order_info_server_typechange_read = pd.read_feather("dtype_test_20240508.ftr") # , dtype=dict(df_order_info_server_typechange.dtypes))

df_order_info_server_typechange_read.dtypes
df_order_info_server_typechange_read

# pd.read_excel("D:\Projects\SystemTrading\JnQ\Bank\data\\table\\table_trade - 복사본.xlsx").dtypes 

,orderId,symbol,status,clientOrderId,price,avgPrice,origQty,executedQty,cumQuote,timeInForce,type,reduceOnly,closePosition,side,positionSide,stopPrice,workingType,priceMatch,selfTradePreventionMode,goodTillDate,priceProtect,origType,time,updateTime
0,8389765678582623918,ETHUSDT,FILLED,e11K9ZTu9miObQM7uRUSgu,0,3098.76000,0.013,0.013,40.28388,GTC,MARKET,True,False,SELL,LONG,0,CONTRACT_PRICE,NONE,NONE,0,False,MARKET,1714870008994,1714870008994
1,8389765678582623861,ETHUSDT,FILLED,ciiTp3UFMigp9u8kQUOADo,0,3098.76000,0.013,0.013,40.28388,GTC,MARKET,True,False,SELL,LONG,0,CONTRACT_PRICE,NONE,NONE,0,False,MARKET,1714870008914,1714870008914
2,8389765678582623808,ETHUSDT,FILLED,qVdL66y1bfgTgaOdUvRphG,0,3098.76000,0.013,0.013,40.28388,GTC,MARKET,True,False,SELL,LONG,0,CONTRACT_PRICE,NONE,NONE,0,False,MARKET,1714870008836,1714870008836
3,8389765678433432873,ETHUSDT,CANCELED,IMoRUT3mHJndskL1xUNTBX,3167.45,0.00000,0.013,0,0,GTC,LIMIT,True,False,SELL,LONG,0,CONTRACT_PRICE,NONE,NONE,0,False,LIMIT,1714823603756,1714869634911
4,8389765678433382758,ETHUSDT,FILLED,gS9NqTDyytssoJZW1fdVlv,3129.59,3124.18000,0.013,0.013,40.61434,GTC,LIMIT,False,False,BUY,LONG,0,CONTRACT_PRICE,NONE,NONE,0,False,LIMIT,1714823598328,1714823598328
5,8389765678428095192,ETHUSDT,EXPIRED,b31EK4sjLGKpZFyjBy81AD,3167.45,0.00000,0.013,0,0,GTC,LIMIT,True,False,SELL,LONG,0,CONTRACT_PRICE,NONE,NONE,0,False,LIMIT,1714822350883,1714870008994
6,8389765678428077178,ETHUSDT,FILLED,iPH0ojrK8yrHzWtQIuFpow,3148.94,3148.70000,0.013,0.013,40.93310,GTC,LIMIT,False,False,BUY,LONG,0,CONTRACT_PRICE,NONE,NONE,0,False,LIMIT,1714822344143,1714822344143
7,8389765678427703028,ETHUSDT,FILLED,wbSJsS20WGGFofDJNAxF2s,3148.94,3148.94000,0.013,0.013,40.93622,GTC,LIMIT,False,False,BUY,LONG,0,CONTRACT_PRICE,NONE,NONE,0,False,LIMIT,1714822251883,1714822335178
8,8389765678412012894,ETHUSDT,CANCELED,ZBm1PJa9Z5L7Ahn56G1BtE,3104.92,0.00000,0.013,0,0,GTC,LIMIT,False,False,BUY,LONG,0,CONTRACT_PRICE,NONE,NONE,0,False,LIMIT,1714819234520,1714820766930
9,8389765678401359471,ETHUSDT,FILLED,QzRGZ42i2VDTNaGKbf0itB,3110,3110.00000,0.013,0.013,40.43000,GTC,LIMIT,True,False,SELL,LONG,0,CONTRACT_PRICE,NONE,NONE,0,False,LIMIT,1714815547605,1714818630294


In [ ]:
def loop_table_trade(self, ):

    """
    v1.0
        values come from rows should be static.
        prevent order_market dup.
        rm self.table_trade.at[idx, 'status'] = self.order_res['status'] in order phase.
            considering status_prev. cannot be here, causing no diffenece before / after order_info update.
            self.table_trade.at[idx, 'statusChangeTime'] can be replaced with order_res's updateTime
        use loc istead of iloc for concat table_log (stay comprehension with .at)
        secure orderId dtype
    v1.1
        add, if self.table_trade.at[idx, 'remove_row'] != 1: # if not trade is done,
            if self.table_trade.at[idx, 'order_way'] == 'CLOSE':
                if self.order_market_on:
    v3.0
        load order_info_server.
        
    
    last confirmed at, 20240507 2053.
    """    

    symbols = np.unique(bank.table_trade.symbol)
    for self.symbol in symbols:
        
    
    for idx, order_info_old in self.table_trade.iterrows(): # for save iteration, use copy().
        
        # init.
        self.order_info = None
        self.expired = 0
        self.order_market_on = 0
        self.table_trade.at[idx, 'order'] = 0 # default.        
        
        # update & clean table.
            # 'order_info_old' means un-updated information. (by get_order_info)
        self.code = order_info_old.code
        self.symbol = order_info_old.symbol # we have symbol column in table already.
        self.orderId = order_info_old.orderId
        self.status_prev = order_info_old.status # save before updateing status.        
        self.order_motion = order_info_old.order_motion

        self.sys_log.debug("order_info_old : {}".format(order_info_old))
        self.sys_log.debug("order_info_old.dtypes : {}".format(order_info_old.dtypes))
    

        # set order, order_way (OPEN) (order not exists)
        if pd.isnull(self.orderId):
            
            self.sys_log.debug("------------------------------------------------")
            start_time = time.time()
            
            self.table_trade.at[idx, 'order'] = 1
            self.table_trade.at[idx, 'order_way'] = 'OPEN'
            
            self.websocket_client.agg_trade(symbol=self.symbol, id=1, callback=self.agg_trade_message_handler)
            self.set_position_mode(self)
            self.set_margin_type(self)
            
            self.sys_log.debug("LoopTableTrade : elasped time, set order, order_way (OPEN) : %.4fs" % (time.time() - start_time)) 
            self.sys_log.debug("------------------------------------------------")

        #  order exists.
        else:
        # if not pd.isnull(self.orderId): # this means, status is not None. (order in settled)

            self.get_order_info(self,) # input : symbol & orderId
            
            # update table_trade
            if self.order_info is not None: # order should be exist.
                # get_order_info. (update order_info)
                    # get_order_info should have valid symbol & orderId.   
                
                self.sys_log.debug("------------------------------------------------")
                start_time = time.time()
                for k, v in self.order_info.items():
                    self.table_trade.at[idx, k] = v
                self.sys_log.debug("LoopTableTrade : elasped time, update table_trade : %.4fs" % (time.time() - start_time)) 
                self.sys_log.debug("------------------------------------------------")
                # display(self.table_trade)
                
                # set order, order_way (CLOSE)
                if order_info_old.order_way == 'OPEN':         
                    if (abs(float(self.order_info['executedQty']) / float(self.order_info['origQty'])) >= self.config.trader_set.quantity_open_exec_ratio) or self.order_info['status'] == 'FILLED':           
                        self.sys_log.debug("------------------------------------------------")
                        start_time = time.time()
                        self.table_trade.at[idx, 'order'] = 1
                        self.table_trade.at[idx, 'order_way'] = 'CLOSE'
                        self.sys_log.debug("LoopTableTrade : elasped time, set order, order_way (CLOSE) : %.4fs" % (time.time() - start_time)) 
                        self.sys_log.debug("------------------------------------------------")
                        # display(self.table_trade)
    
                
                # set statusChangeTime 
                    # check if status has been changed.
                if self.status_prev != self.order_info['status']:
                    self.sys_log.debug("------------------------------------------------")
                    start_time = time.time()
                    self.table_trade.at[idx, 'statusChangeTime'] = int(time.time())    
                    self.sys_log.debug("LoopTableTrade : elasped time, set statusChangeTime  : %.4fs" % (time.time() - start_time)) 
                    self.sys_log.debug("------------------------------------------------")      
                    # display(self.table_trade)
                
                    # set remove_row. 
                        # use updated status. 
                    if (self.order_info['status'] in ['CANCELED', 'EXPIRED', 'REJECTED']) or (order_info_old.order_way == 'CLOSE' and self.order_info['status'] == 'FILLED'):
                        self.sys_log.debug("------------------------------------------------")
                        start_time = time.time()
                        self.table_trade.at[idx, 'remove_row'] = 1
                        self.sys_log.debug("LoopTableTrade : elasped time, set remove_row  : %.4fs" % (time.time() - start_time)) 
                        self.sys_log.debug("------------------------------------------------")
                        # display(self.table_trade)
                    
                    # logging : transfer rows to Table - log.        
                    self.table_log = pd.concat([self.table_log, self.table_trade.loc[[idx]]]) # list input persist dataframe output.
                    self.sys_log.debug("LoopTableTrade : elasped time, + logging  : %.4fs" % (time.time() - start_time)) 
                    self.sys_log.debug("------------------------------------------------")  
                    # display(self.table_trade)
    
                
                # check expiration for order_open.
                    # set price_reatlime
                        # for, open expiry & check_stop_loss
                self.get_price_realtime(self,)
                self.table_trade.at[idx, 'price_realtime'] = self.price_realtime            
        
                    # order_info_old.side_position has 2 state (LONG / SHORT) in status below 2 case.
                if order_info_old.order_way == 'OPEN' and self.order_info['status'] in ['NEW', 'PARTIALLY_FILLED']:
                    self.sys_log.debug("------------------------------------------------")
                    start_time = time.time()
                    
                    self.price_expiration = order_info_old.price_expiration
                    if order_info_old.side_position == 'SHORT':
                        if self.price_realtime <= self.price_expiration:
                            self.expired = 1
                    else: # elif order_info_old.side_position == 'LONG':
                        if self.price_realtime >= self.price_expiration:
                            self.expired = 1                
                        
                    if self.expired:      
                        self.order_cancel(self,)
                        
                        # Todo, messeage alertion needed ?
                        self.push_msg(self, "{} has been expired.".format(self.code))
                        self.user_text = 'watch'  # allowing message.
                    self.sys_log.debug("LoopTableTrade : elasped time, check expiration for order_open  : %.4fs" % (time.time() - start_time)) 
                    self.sys_log.debug("------------------------------------------------")
    
                
                # check stop_loss
                    # get order_market_on
                    # order_way == 'CLOSE' : stop_loss / liquidation condition.
                elif order_info_old.order_way == 'CLOSE':
                    self.sys_log.debug("------------------------------------------------")
                    start_time = time.time()   
                    self.price_stop_loss = order_info_old.price_stop_loss
                    self.price_liquidation = order_info_old.price_liquidation
                    self.check_stop_loss(self,)
                    # self.check_stop_loss_by_signal(self,)
                    self.sys_log.debug("LoopTableTrade : elasped time, check_stop_loss  : %.4fs" % (time.time() - start_time)) 
                    self.sys_log.debug("------------------------------------------------")
            

        
        # order_limit
        if not self.config.trader_set.backtrade and not self.order_motion:
            
           if self.table_trade.at[idx, 'order'] == 1:

                self.sys_log.debug("------------------------------------------------")
                start_time = time.time()
                # public set.
                order_type = 'LIMIT' # fixed for order_limit.
                side_position = order_info_old.side_position
            
                # open condition
                if self.table_trade.at[idx, 'order_way'] == 'OPEN':
                
                    # open                    
                    side_order = order_info_old.side_open
                    price = order_info_old.price_entry
                    quantity = order_info_old.quantity_open
    
                # close condition
                else: #  self.table_trade.at[idx, 'order_way'] == 'CLOSE': # upper phase's condition use 'order' = 1
                            
                    # we don't do partial like this anymore.
                        # each price info will be adjusted order by order.
                    # self.order_limit_multiple(self, order_res_list)
                    
                    # close                    
                    side_order = order_info_old.side_close
                    price = order_info_old.price_take_profit
                    quantity = self.order_info['executedQty']
                    
                # [ API - Trading ]                
                self.order_limit(self, order_type, side_order, side_position, price, quantity)
            
                # KIWOOM version.
                    # 주문 단가가 상한선인 경우, 주문 가능할 때까지 keep.
                        # self.price_stop_loss check 는 지속적으로 수행함.
                        # 한번만 append 해야할 것, close_data 에서는 pop 하지 않음.
                            # 이것도 누적인가 ?
               
                # normal state : self.error_code = 0
                if not self.error_code:
                    self.table_trade.at[idx, 'orderId'] = str(self.order_res['orderId']) # this is key for updating order_info (change to OPEN / CLOSE orderId)
                else:
                    time.sleep(self.config.trader_set.order_term)
                self.sys_log.debug("LoopTableTrade : elasped time, order_limit  : %.4fs" % (time.time() - start_time)) 
                self.sys_log.debug("------------------------------------------------")

        
        # order_market
            # prevent order_market duplication.
                # order_way == 'CLOSE' : stop_loss / liquidation condition.
        if self.table_trade.at[idx, 'remove_row'] != 1: # if not trade is done,
            if self.table_trade.at[idx, 'order_way'] == 'CLOSE':
                if self.order_market_on:
                    self.sys_log.debug("------------------------------------------------")
                    start_time = time.time()                
                    self.side_close = order_info_old.side_close
                    self.side_position = order_info_old.side_position
                    self.order_market(self,)
                    self.table_trade.at[idx, 'orderId'] = str(self.order_res['orderId']) # this is key for updating order_info (change to CLOSE orderId)
                    self.sys_log.debug("LoopTableTrade : elasped time, order_market  : %.4fs" % (time.time() - start_time)) 
                    self.sys_log.debug("------------------------------------------------")

        
        # drop rows
            # this phase should be placed in the most below, else order_ will set value as np.nan in invalid rows.
        else:
        # if self.table_trade.at[idx, 'remove_row'] == 1:      
            self.sys_log.debug("------------------------------------------------")
            start_time = time.time()      
            # drow remove_row = 1,
            self.table_trade.drop([idx], inplace=True)
            # display(self.table_trade)

            # remove from websocket_client.
            if self.config.trader_set.messenger_on:
                self.websocket_client.stop_socket("{}@aggTrade".format(self.symbol.lower()))

            self.sys_log.debug("LoopTableTrade : elasped time, drop rows  : %.4fs" % (time.time() - start_time)) 
            self.sys_log.debug("------------------------------------------------")  
            
            self.sys_log.debug("------------------------------------------------")
            start_time = time.time()      
            self.get_income_info(self)
            self.sys_log.debug("LoopTableTrade : elasped time, get_income_info  : %.4fs" % (time.time() - start_time)) 
            self.sys_log.debug("------------------------------------------------")  
            # display(self.table_trade)

##### v2.0

In [ ]:
def loop_table_trade(self, ):

    """
    v1.0
        values come from rows should be static.
        prevent order_market dup.
        rm self.table_trade.iat[idx, 'status'] = self.order_res['status'] in order phase.
            considering status_prev. cannot be here, causing no diffenece before / after order_info update.
            self.table_trade.iat[idx, 'statusChangeTime'] can be replaced with order_res's updateTime
    v2.0
        use iat instead of at
        deprecated.
    
    last confirmed at, 20240503 0950
    """    
    
    for idx, (idx_table, order_info_old) in enumerate(self.table_trade.iterrows()): # for save iteration, use copy().

        self.sys_log.debug("idx : {}".format(idx))
        self.sys_log.debug("type(idx) : {}".format(type(idx)))
        self.sys_log.debug("idx_table : {}".format(idx_table))

        
        # init.
        self.order_info = None
        self.expired = 0
        self.order_market_on = 0
        self.table_trade.iat[idx, 'order'] = 0 # default.        
        
        # update & clean table.
            # 'order_info_old' means un-updated information. (by get_order_info)
        self.code = order_info_old.code
        self.symbol = order_info_old.symbol # we have symbol column in table already.
        self.orderId = order_info_old.orderId
        self.status_prev = order_info_old.status # save before updateing status.       
        

        # set order, order_way (OPEN) (order not exists)
        if pd.isnull(self.orderId):
            
            self.sys_log.debug("------------------------------------------------")
            start_time = time.time()
            
            self.table_trade.iat[idx, 'order'] = 1
            self.table_trade.iat[idx, 'order_way'] = 'OPEN'
            
            self.websocket_client.agg_trade(symbol=self.symbol, id=1, callback=self.agg_trade_message_handler)
            self.set_position_mode(self)
            self.set_margin_type(self)
            
            self.sys_log.debug("LoopTableTrade : elasped time, set order, order_way (OPEN) : %.4fs" % (time.time() - start_time)) 
            self.sys_log.debug("------------------------------------------------")

        #  order exists.
        else:
        # if not pd.isnull(self.orderId): # this means, status is not None. (order in settled)

            self.get_order_info(self,)   
            
            # update table_trade
            if self.order_info is not None: # order should be exist.
                # get_order_info. (update order_info)
                    # get_order_info should have valid symbol & orderId.   
                
                self.sys_log.debug("------------------------------------------------")
                start_time = time.time()
                for k, v in self.order_info.items():
                    self.table_trade.iat[idx, k] = v
                self.sys_log.debug("LoopTableTrade : elasped time, update table_trade : %.4fs" % (time.time() - start_time)) 
                self.sys_log.debug("------------------------------------------------")
                # display(self.table_trade)
                
                # set order, order_way (CLOSE)
                if order_info_old.order_way == 'OPEN':         
                    if (abs(float(self.order_info['executedQty']) / float(self.order_info['origQty'])) >= self.config.trader_set.quantity_open_exec_ratio) or self.order_info['status'] == 'FILLED':           
                        self.sys_log.debug("------------------------------------------------")
                        start_time = time.time()
                        self.table_trade.iat[idx, 'order'] = 1
                        self.table_trade.iat[idx, 'order_way'] = 'CLOSE'
                        self.sys_log.debug("LoopTableTrade : elasped time, set order, order_way (CLOSE) : %.4fs" % (time.time() - start_time)) 
                        self.sys_log.debug("------------------------------------------------")
                        # display(self.table_trade)
    
                
                # set statusChangeTime 
                    # check if status has been changed.
                if self.status_prev != self.order_info['status']:
                    self.sys_log.debug("------------------------------------------------")
                    start_time = time.time()
                    self.table_trade.iat[idx, 'statusChangeTime'] = int(time.time())    
                    self.sys_log.debug("LoopTableTrade : elasped time, set statusChangeTime  : %.4fs" % (time.time() - start_time)) 
                    self.sys_log.debug("------------------------------------------------")      
                    # display(self.table_trade)
                
                    # set remove_row. 
                        # use updated status. 
                    if (self.order_info['status'] in ['CANCELED', 'EXPIRED', 'REJECTED']) or (order_info_old.order_way == 'CLOSE' and self.order_info['status'] == 'FILLED'):
                        self.sys_log.debug("------------------------------------------------")
                        start_time = time.time()
                        self.table_trade.iat[idx, 'remove_row'] = 1
                        self.sys_log.debug("LoopTableTrade : elasped time, set remove_row  : %.4fs" % (time.time() - start_time)) 
                        self.sys_log.debug("------------------------------------------------")
                        # display(self.table_trade)
                    
                    # logging : transfer rows to Table - log.        
                    self.table_log = pd.concat([self.table_log, self.table_trade.iloc[[idx]]])
                    self.sys_log.debug("LoopTableTrade : elasped time, + logging  : %.4fs" % (time.time() - start_time)) 
                    self.sys_log.debug("------------------------------------------------")  
                    # display(self.table_trade)
    
                
                # check expiration for order_open.
                    # set price_reatlime
                        # for, open expiry & check_stop_loss
                self.get_price_realtime(self,)
                self.table_trade.iat[idx, 'price_realtime'] = self.price_realtime            
        
                    # order_info_old.side_position has 2 state (LONG / SHORT) in status below 2 case.
                if order_info_old.order_way == 'OPEN' and self.order_info['status'] in ['NEW', 'PARTIALLY_FILLED']:
                    self.sys_log.debug("------------------------------------------------")
                    start_time = time.time()
                    
                    self.price_expiration = order_info_old.price_expiration
                    if order_info_old.side_position == 'SHORT':
                        if self.price_realtime <= self.price_expiration:
                            self.expired = 1
                    else: # elif order_info_old.side_position == 'LONG':
                        if self.price_realtime >= self.price_expiration:
                            self.expired = 1                
                        
                    if self.expired:      
                        self.order_cancel(self,)
                        
                        # Todo, messeage alertion needed ?
                        self.push_msg(self, "{} has been expired.".format(self.code))
                        self.user_text = 'watch'  # allowing message.
                    self.sys_log.debug("LoopTableTrade : elasped time, check expiration for order_open  : %.4fs" % (time.time() - start_time)) 
                    self.sys_log.debug("------------------------------------------------")
    
                
                # check stop_loss
                    # get order_market_on
                elif order_info_old.order_way == 'CLOSE':
                    self.sys_log.debug("------------------------------------------------")
                    start_time = time.time()   
                    self.price_stop_loss = order_info_old.price_stop_loss
                    self.price_liquidation = order_info_old.price_liquidation
                    self.check_stop_loss(self,)
                    # self.check_stop_loss_by_signal(self,)
                    self.sys_log.debug("LoopTableTrade : elasped time, check_stop_loss  : %.4fs" % (time.time() - start_time)) 
                    self.sys_log.debug("------------------------------------------------")
            

        
        # order_limit
        if not self.config.trader_set.backtrade and not self.order_motion:
            
           if self.table_trade.iat[idx, 'order'] == 1:

                self.sys_log.debug("------------------------------------------------")
                start_time = time.time()
                # public set.
                order_type = 'LIMIT' # fixed for order_limit.
                side_position = order_info_old.side_position
            
                # open condition
                if self.table_trade.iat[idx, 'order_way'] == 'OPEN':
                
                    # open                    
                    side_order = order_info_old.side_open
                    price = order_info_old.price_entry
                    quantity = order_info_old.quantity_open
    
                # close condition
                else: #  self.table_trade.iat[idx, 'order_way'] == 'CLOSE': # upper phase's condition use 'order' = 1
                            
                    # we don't do partial like this anymore.
                        # each price info will be adjusted order by order.
                    # self.order_limit_multiple(self, order_res_list)
                    
                    # close                    
                    side_order = order_info_old.side_close
                    price = order_info_old.price_take_profit
                    quantity = self.order_info['executedQty']
                    
                # [ API - Trading ]                
                self.order_limit(self, order_type, side_order, side_position, price, quantity)
            
                # KIWOOM version.
                    # 주문 단가가 상한선인 경우, 주문 가능할 때까지 keep.
                        # self.price_stop_loss check 는 지속적으로 수행함.
                        # 한번만 append 해야할 것, close_data 에서는 pop 하지 않음.
                            # 이것도 누적인가 ?
               
                # normal state : self.error_code = 0
                if not self.error_code:
                    self.table_trade.iat[idx, 'orderId'] = self.order_res['orderId'] # this is key for updating order_info (change to OPEN / CLOSE orderId)
                else:
                    time.sleep(self.config.trader_set.order_term)
                self.sys_log.debug("LoopTableTrade : elasped time, order_limit  : %.4fs" % (time.time() - start_time)) 
                self.sys_log.debug("------------------------------------------------")

        
        # order_market
            # prevent order_market duplication.
        if self.table_trade.iat[idx, 'type'] != 'MARKET':
            if self.order_market_on:
                self.sys_log.debug("------------------------------------------------")
                start_time = time.time()
                self.order_market(self,)
                self.table_trade.iat[idx, 'orderId'] = self.order_res['orderId'] # this is key for updating order_info (change to CLOSE orderId)
                self.sys_log.debug("LoopTableTrade : elasped time, order_market  : %.4fs" % (time.time() - start_time)) 
                self.sys_log.debug("------------------------------------------------")

        
        # drop rows
            # this phase should be placed in the most below, else order_ will set value as np.nan in invalid rows.
        if self.table_trade.iat[idx, 'remove_row'] == 1:      
            self.sys_log.debug("------------------------------------------------")
            start_time = time.time()      
            # drow remove_row = 1,
            self.table_trade.drop([idx], inplace=True)
            # display(self.table_trade)

            # remove from websocket_client.
            if self.config.trader_set.messenger_on:
                self.websocket_client.stop_socket("{}@aggTrade".format(self.symbol.lower()))
            self.sys_log.debug("LoopTableTrade : elasped time, drop rows  : %.4fs" % (time.time() - start_time)) 
            self.sys_log.debug("------------------------------------------------")  
            # display(self.table_trade)

##### v1.1

In [ ]:
def loop_table_trade(self, ):

    """
    v1.0
        values come from rows should be static.
        prevent order_market dup.
        rm self.table_trade.at[idx, 'status'] = self.order_res['status'] in order phase.
            considering status_prev. cannot be here, causing no diffenece before / after order_info update.
            self.table_trade.at[idx, 'statusChangeTime'] can be replaced with order_res's updateTime
        use loc istead of iloc for concat table_log (stay comprehension with .at)
        secure orderId dtype
    v1.1
        add, if self.table_trade.at[idx, 'remove_row'] != 1: # if not trade is done,
            if self.table_trade.at[idx, 'order_way'] == 'CLOSE':
                if self.order_market_on:
        
    
    last confirmed at, 20240506 2148
    """    
    
    for idx, order_info_old in self.table_trade.iterrows(): # for save iteration, use copy().
        
        # init.
        self.order_info = None
        self.expired = 0
        self.order_market_on = 0
        self.table_trade.at[idx, 'order'] = 0 # default.        
        
        # update & clean table.
            # 'order_info_old' means un-updated information. (by get_order_info)
        self.code = order_info_old.code
        self.symbol = order_info_old.symbol # we have symbol column in table already.
        self.orderId = order_info_old.orderId
        self.status_prev = order_info_old.status # save before updateing status.        
        self.order_motion = order_info_old.order_motion

        self.sys_log.debug("order_info_old : {}".format(order_info_old))
        self.sys_log.debug("order_info_old.dtypes : {}".format(order_info_old.dtypes))
    

        # set order, order_way (OPEN) (order not exists)
        if pd.isnull(self.orderId):
            
            self.sys_log.debug("------------------------------------------------")
            start_time = time.time()
            
            self.table_trade.at[idx, 'order'] = 1
            self.table_trade.at[idx, 'order_way'] = 'OPEN'
            
            self.websocket_client.agg_trade(symbol=self.symbol, id=1, callback=self.agg_trade_message_handler)
            self.set_position_mode(self)
            self.set_margin_type(self)
            
            self.sys_log.debug("LoopTableTrade : elasped time, set order, order_way (OPEN) : %.4fs" % (time.time() - start_time)) 
            self.sys_log.debug("------------------------------------------------")

        #  order exists.
        else:
        # if not pd.isnull(self.orderId): # this means, status is not None. (order in settled)

            self.get_order_info(self,)   
            
            # update table_trade
            if self.order_info is not None: # order should be exist.
                # get_order_info. (update order_info)
                    # get_order_info should have valid symbol & orderId.   
                
                self.sys_log.debug("------------------------------------------------")
                start_time = time.time()
                for k, v in self.order_info.items():
                    self.table_trade.at[idx, k] = v
                self.sys_log.debug("LoopTableTrade : elasped time, update table_trade : %.4fs" % (time.time() - start_time)) 
                self.sys_log.debug("------------------------------------------------")
                # display(self.table_trade)
                
                # set order, order_way (CLOSE)
                if order_info_old.order_way == 'OPEN':         
                    if (abs(float(self.order_info['executedQty']) / float(self.order_info['origQty'])) >= self.config.trader_set.quantity_open_exec_ratio) or self.order_info['status'] == 'FILLED':           
                        self.sys_log.debug("------------------------------------------------")
                        start_time = time.time()
                        self.table_trade.at[idx, 'order'] = 1
                        self.table_trade.at[idx, 'order_way'] = 'CLOSE'
                        self.sys_log.debug("LoopTableTrade : elasped time, set order, order_way (CLOSE) : %.4fs" % (time.time() - start_time)) 
                        self.sys_log.debug("------------------------------------------------")
                        # display(self.table_trade)
    
                
                # set statusChangeTime 
                    # check if status has been changed.
                if self.status_prev != self.order_info['status']:
                    self.sys_log.debug("------------------------------------------------")
                    start_time = time.time()
                    self.table_trade.at[idx, 'statusChangeTime'] = int(time.time())    
                    self.sys_log.debug("LoopTableTrade : elasped time, set statusChangeTime  : %.4fs" % (time.time() - start_time)) 
                    self.sys_log.debug("------------------------------------------------")      
                    # display(self.table_trade)
                
                    # set remove_row. 
                        # use updated status. 
                    if (self.order_info['status'] in ['CANCELED', 'EXPIRED', 'REJECTED']) or (order_info_old.order_way == 'CLOSE' and self.order_info['status'] == 'FILLED'):
                        self.sys_log.debug("------------------------------------------------")
                        start_time = time.time()
                        self.table_trade.at[idx, 'remove_row'] = 1
                        self.sys_log.debug("LoopTableTrade : elasped time, set remove_row  : %.4fs" % (time.time() - start_time)) 
                        self.sys_log.debug("------------------------------------------------")
                        # display(self.table_trade)
                    
                    # logging : transfer rows to Table - log.        
                    self.table_log = pd.concat([self.table_log, self.table_trade.loc[[idx]]]) # list input persist dataframe output.
                    self.sys_log.debug("LoopTableTrade : elasped time, + logging  : %.4fs" % (time.time() - start_time)) 
                    self.sys_log.debug("------------------------------------------------")  
                    # display(self.table_trade)
    
                
                # check expiration for order_open.
                    # set price_reatlime
                        # for, open expiry & check_stop_loss
                self.get_price_realtime(self,)
                self.table_trade.at[idx, 'price_realtime'] = self.price_realtime            
        
                    # order_info_old.side_position has 2 state (LONG / SHORT) in status below 2 case.
                if order_info_old.order_way == 'OPEN' and self.order_info['status'] in ['NEW', 'PARTIALLY_FILLED']:
                    self.sys_log.debug("------------------------------------------------")
                    start_time = time.time()
                    
                    self.price_expiration = order_info_old.price_expiration
                    if order_info_old.side_position == 'SHORT':
                        if self.price_realtime <= self.price_expiration:
                            self.expired = 1
                    else: # elif order_info_old.side_position == 'LONG':
                        if self.price_realtime >= self.price_expiration:
                            self.expired = 1                
                        
                    if self.expired:      
                        self.order_cancel(self,)
                        
                        # Todo, messeage alertion needed ?
                        self.push_msg(self, "{} has been expired.".format(self.code))
                        self.user_text = 'watch'  # allowing message.
                    self.sys_log.debug("LoopTableTrade : elasped time, check expiration for order_open  : %.4fs" % (time.time() - start_time)) 
                    self.sys_log.debug("------------------------------------------------")
    
                
                # check stop_loss
                    # get order_market_on
                    # order_way == 'CLOSE' : stop_loss / liquidation condition.
                elif order_info_old.order_way == 'CLOSE':
                    self.sys_log.debug("------------------------------------------------")
                    start_time = time.time()   
                    self.price_stop_loss = order_info_old.price_stop_loss
                    self.price_liquidation = order_info_old.price_liquidation
                    self.check_stop_loss(self,)
                    # self.check_stop_loss_by_signal(self,)
                    self.sys_log.debug("LoopTableTrade : elasped time, check_stop_loss  : %.4fs" % (time.time() - start_time)) 
                    self.sys_log.debug("------------------------------------------------")
            

        
        # order_limit
        if not self.config.trader_set.backtrade and not self.order_motion:
            
           if self.table_trade.at[idx, 'order'] == 1:

                self.sys_log.debug("------------------------------------------------")
                start_time = time.time()
                # public set.
                order_type = 'LIMIT' # fixed for order_limit.
                side_position = order_info_old.side_position
            
                # open condition
                if self.table_trade.at[idx, 'order_way'] == 'OPEN':
                
                    # open                    
                    side_order = order_info_old.side_open
                    price = order_info_old.price_entry
                    quantity = order_info_old.quantity_open
    
                # close condition
                else: #  self.table_trade.at[idx, 'order_way'] == 'CLOSE': # upper phase's condition use 'order' = 1
                            
                    # we don't do partial like this anymore.
                        # each price info will be adjusted order by order.
                    # self.order_limit_multiple(self, order_res_list)
                    
                    # close                    
                    side_order = order_info_old.side_close
                    price = order_info_old.price_take_profit
                    quantity = self.order_info['executedQty']
                    
                # [ API - Trading ]                
                self.order_limit(self, order_type, side_order, side_position, price, quantity)
            
                # KIWOOM version.
                    # 주문 단가가 상한선인 경우, 주문 가능할 때까지 keep.
                        # self.price_stop_loss check 는 지속적으로 수행함.
                        # 한번만 append 해야할 것, close_data 에서는 pop 하지 않음.
                            # 이것도 누적인가 ?
               
                # normal state : self.error_code = 0
                if not self.error_code:
                    self.table_trade.at[idx, 'orderId'] = str(self.order_res['orderId']) # this is key for updating order_info (change to OPEN / CLOSE orderId)
                else:
                    time.sleep(self.config.trader_set.order_term)
                self.sys_log.debug("LoopTableTrade : elasped time, order_limit  : %.4fs" % (time.time() - start_time)) 
                self.sys_log.debug("------------------------------------------------")

        
        # order_market
            # prevent order_market duplication.
                # order_way == 'CLOSE' : stop_loss / liquidation condition.
        if self.table_trade.at[idx, 'remove_row'] != 1: # if not trade is done,
            if self.table_trade.at[idx, 'order_way'] == 'CLOSE':
                if self.order_market_on:
                    self.sys_log.debug("------------------------------------------------")
                    start_time = time.time()                
                    self.side_close = order_info_old.side_close
                    self.side_position = order_info_old.side_position
                    self.order_market(self,)
                    self.table_trade.at[idx, 'orderId'] = str(self.order_res['orderId']) # this is key for updating order_info (change to CLOSE orderId)
                    self.sys_log.debug("LoopTableTrade : elasped time, order_market  : %.4fs" % (time.time() - start_time)) 
                    self.sys_log.debug("------------------------------------------------")

        
        # drop rows
            # this phase should be placed in the most below, else order_ will set value as np.nan in invalid rows.
        else:
        # if self.table_trade.at[idx, 'remove_row'] == 1:      
            self.sys_log.debug("------------------------------------------------")
            start_time = time.time()      
            # drow remove_row = 1,
            self.table_trade.drop([idx], inplace=True)
            # display(self.table_trade)

            # remove from websocket_client.
            if self.config.trader_set.messenger_on:
                self.websocket_client.stop_socket("{}@aggTrade".format(self.symbol.lower()))

            self.sys_log.debug("LoopTableTrade : elasped time, drop rows  : %.4fs" % (time.time() - start_time)) 
            self.sys_log.debug("------------------------------------------------")  
            
            self.sys_log.debug("------------------------------------------------")
            start_time = time.time()      
            self.get_income_info(self)
            self.sys_log.debug("LoopTableTrade : elasped time, get_income_info  : %.4fs" % (time.time() - start_time)) 
            self.sys_log.debug("------------------------------------------------")  
            # display(self.table_trade)

##### v1.0

In [ ]:
def loop_table_trade(self, ):

    """
    v1.0
        values come from rows should be static.
        prevent order_market dup.
        rm self.table_trade.at[idx, 'status'] = self.order_res['status'] in order phase.
            considering status_prev. cannot be here, causing no diffenece before / after order_info update.
            self.table_trade.at[idx, 'statusChangeTime'] can be replaced with order_res's updateTime
        use loc istead of iloc for concat table_log (stay comprehension with .at)
        secure orderId dtype
    
    last confirmed at, 20240503 0857
    """    
    
    for idx, order_info_old in self.table_trade.iterrows(): # for save iteration, use copy().
        
        # init.
        self.order_info = None
        self.expired = 0
        self.order_market_on = 0
        self.table_trade.at[idx, 'order'] = 0 # default.        
        
        # update & clean table.
            # 'order_info_old' means un-updated information. (by get_order_info)
        self.code = order_info_old.code
        self.symbol = order_info_old.symbol # we have symbol column in table already.
        self.orderId = order_info_old.orderId
        self.status_prev = order_info_old.status # save before updateing status.        
        self.order_motion = order_info_old.order_motion
        

        # set order, order_way (OPEN) (order not exists)
        if pd.isnull(self.orderId):
            
            self.sys_log.debug("------------------------------------------------")
            start_time = time.time()
            
            self.table_trade.at[idx, 'order'] = 1
            self.table_trade.at[idx, 'order_way'] = 'OPEN'
            
            self.websocket_client.agg_trade(symbol=self.symbol, id=1, callback=self.agg_trade_message_handler)
            self.set_position_mode(self)
            self.set_margin_type(self)
            
            self.sys_log.debug("LoopTableTrade : elasped time, set order, order_way (OPEN) : %.4fs" % (time.time() - start_time)) 
            self.sys_log.debug("------------------------------------------------")

        #  order exists.
        else:
        # if not pd.isnull(self.orderId): # this means, status is not None. (order in settled)

            self.get_order_info(self,)   
            
            # update table_trade
            if self.order_info is not None: # order should be exist.
                # get_order_info. (update order_info)
                    # get_order_info should have valid symbol & orderId.   
                
                self.sys_log.debug("------------------------------------------------")
                start_time = time.time()
                for k, v in self.order_info.items():
                    self.table_trade.at[idx, k] = v
                self.sys_log.debug("LoopTableTrade : elasped time, update table_trade : %.4fs" % (time.time() - start_time)) 
                self.sys_log.debug("------------------------------------------------")
                # display(self.table_trade)
                
                # set order, order_way (CLOSE)
                if order_info_old.order_way == 'OPEN':         
                    if (abs(float(self.order_info['executedQty']) / float(self.order_info['origQty'])) >= self.config.trader_set.quantity_open_exec_ratio) or self.order_info['status'] == 'FILLED':           
                        self.sys_log.debug("------------------------------------------------")
                        start_time = time.time()
                        self.table_trade.at[idx, 'order'] = 1
                        self.table_trade.at[idx, 'order_way'] = 'CLOSE'
                        self.sys_log.debug("LoopTableTrade : elasped time, set order, order_way (CLOSE) : %.4fs" % (time.time() - start_time)) 
                        self.sys_log.debug("------------------------------------------------")
                        # display(self.table_trade)
    
                
                # set statusChangeTime 
                    # check if status has been changed.
                if self.status_prev != self.order_info['status']:
                    self.sys_log.debug("------------------------------------------------")
                    start_time = time.time()
                    self.table_trade.at[idx, 'statusChangeTime'] = int(time.time())    
                    self.sys_log.debug("LoopTableTrade : elasped time, set statusChangeTime  : %.4fs" % (time.time() - start_time)) 
                    self.sys_log.debug("------------------------------------------------")      
                    # display(self.table_trade)
                
                    # set remove_row. 
                        # use updated status. 
                    if (self.order_info['status'] in ['CANCELED', 'EXPIRED', 'REJECTED']) or (order_info_old.order_way == 'CLOSE' and self.order_info['status'] == 'FILLED'):
                        self.sys_log.debug("------------------------------------------------")
                        start_time = time.time()
                        self.table_trade.at[idx, 'remove_row'] = 1
                        self.sys_log.debug("LoopTableTrade : elasped time, set remove_row  : %.4fs" % (time.time() - start_time)) 
                        self.sys_log.debug("------------------------------------------------")
                        # display(self.table_trade)
                    
                    # logging : transfer rows to Table - log.        
                    self.table_log = pd.concat([self.table_log, self.table_trade.loc[[idx]]]) # list input persist dataframe output.
                    self.sys_log.debug("LoopTableTrade : elasped time, + logging  : %.4fs" % (time.time() - start_time)) 
                    self.sys_log.debug("------------------------------------------------")  
                    # display(self.table_trade)
    
                
                # check expiration for order_open.
                    # set price_reatlime
                        # for, open expiry & check_stop_loss
                self.get_price_realtime(self,)
                self.table_trade.at[idx, 'price_realtime'] = self.price_realtime            
        
                    # order_info_old.side_position has 2 state (LONG / SHORT) in status below 2 case.
                if order_info_old.order_way == 'OPEN' and self.order_info['status'] in ['NEW', 'PARTIALLY_FILLED']:
                    self.sys_log.debug("------------------------------------------------")
                    start_time = time.time()
                    
                    self.price_expiration = order_info_old.price_expiration
                    if order_info_old.side_position == 'SHORT':
                        if self.price_realtime <= self.price_expiration:
                            self.expired = 1
                    else: # elif order_info_old.side_position == 'LONG':
                        if self.price_realtime >= self.price_expiration:
                            self.expired = 1                
                        
                    if self.expired:      
                        self.order_cancel(self,)
                        
                        # Todo, messeage alertion needed ?
                        self.push_msg(self, "{} has been expired.".format(self.code))
                        self.user_text = 'watch'  # allowing message.
                    self.sys_log.debug("LoopTableTrade : elasped time, check expiration for order_open  : %.4fs" % (time.time() - start_time)) 
                    self.sys_log.debug("------------------------------------------------")
    
                
                # check stop_loss
                    # get order_market_on
                elif order_info_old.order_way == 'CLOSE':
                    self.sys_log.debug("------------------------------------------------")
                    start_time = time.time()   
                    self.price_stop_loss = order_info_old.price_stop_loss
                    self.price_liquidation = order_info_old.price_liquidation
                    self.check_stop_loss(self,)
                    # self.check_stop_loss_by_signal(self,)
                    self.sys_log.debug("LoopTableTrade : elasped time, check_stop_loss  : %.4fs" % (time.time() - start_time)) 
                    self.sys_log.debug("------------------------------------------------")
            

        
        # order_limit
        if not self.config.trader_set.backtrade and not self.order_motion:
            
           if self.table_trade.at[idx, 'order'] == 1:

                self.sys_log.debug("------------------------------------------------")
                start_time = time.time()
                # public set.
                order_type = 'LIMIT' # fixed for order_limit.
                side_position = order_info_old.side_position
            
                # open condition
                if self.table_trade.at[idx, 'order_way'] == 'OPEN':
                
                    # open                    
                    side_order = order_info_old.side_open
                    price = order_info_old.price_entry
                    quantity = order_info_old.quantity_open
    
                # close condition
                else: #  self.table_trade.at[idx, 'order_way'] == 'CLOSE': # upper phase's condition use 'order' = 1
                            
                    # we don't do partial like this anymore.
                        # each price info will be adjusted order by order.
                    # self.order_limit_multiple(self, order_res_list)
                    
                    # close                    
                    side_order = order_info_old.side_close
                    price = order_info_old.price_take_profit
                    quantity = self.order_info['executedQty']
                    
                # [ API - Trading ]                
                self.order_limit(self, order_type, side_order, side_position, price, quantity)
            
                # KIWOOM version.
                    # 주문 단가가 상한선인 경우, 주문 가능할 때까지 keep.
                        # self.price_stop_loss check 는 지속적으로 수행함.
                        # 한번만 append 해야할 것, close_data 에서는 pop 하지 않음.
                            # 이것도 누적인가 ?
               
                # normal state : self.error_code = 0
                if not self.error_code:
                    self.table_trade.at[idx, 'orderId'] = str(self.order_res['orderId']) # this is key for updating order_info (change to OPEN / CLOSE orderId)
                else:
                    time.sleep(self.config.trader_set.order_term)
                self.sys_log.debug("LoopTableTrade : elasped time, order_limit  : %.4fs" % (time.time() - start_time)) 
                self.sys_log.debug("------------------------------------------------")

        
        # order_market
            # prevent order_market duplication.
        if self.table_trade.at[idx, 'type'] != 'MARKET':
            if self.order_market_on:
                self.sys_log.debug("------------------------------------------------")
                start_time = time.time()                
                self.side_close = order_info_old.side_close
                self.side_position = order_info_old.side_position
                self.order_market(self,)
                self.table_trade.at[idx, 'orderId'] = str(self.order_res['orderId']) # this is key for updating order_info (change to CLOSE orderId)
                self.sys_log.debug("LoopTableTrade : elasped time, order_market  : %.4fs" % (time.time() - start_time)) 
                self.sys_log.debug("------------------------------------------------")

        
        # drop rows
            # this phase should be placed in the most below, else order_ will set value as np.nan in invalid rows.
        if self.table_trade.at[idx, 'remove_row'] == 1:      
            self.sys_log.debug("------------------------------------------------")
            start_time = time.time()      
            # drow remove_row = 1,
            self.table_trade.drop([idx], inplace=True)
            # display(self.table_trade)

            # remove from websocket_client.
            if self.config.trader_set.messenger_on:
                self.websocket_client.stop_socket("{}@aggTrade".format(self.symbol.lower()))

            self.sys_log.debug("LoopTableTrade : elasped time, drop rows  : %.4fs" % (time.time() - start_time)) 
            self.sys_log.debug("------------------------------------------------")  
            
            self.sys_log.debug("------------------------------------------------")
            start_time = time.time()      
            self.get_income_info(self)
            self.sys_log.debug("LoopTableTrade : elasped time, get_income_info  : %.4fs" % (time.time() - start_time)) 
            self.sys_log.debug("------------------------------------------------")  
            # display(self.table_trade)

### BankSettings

#### echo

In [ ]:
def echo(self, update, context):
    self.user_text = update.message.text
     

#### get_messenger_bot

##### v2.0

In [ ]:
   
def get_messenger_bot(self, ):

    """
    v2.0
        use token from self directly.

    last confirmed at, 20240505 1720.
    """
    
    self.sys_log.info("self.msg_bot : {}".format(self.msg_bot))
    
    if self.msg_bot is None:
        self.msg_bot = telegram.Bot(token=self.token)
        
        # 2. get updater.
            # a. init.
        self.user_text = None # messenger buffer.
        
        self.updater = Updater(token=self.token, use_context=True)
        dispatcher = self.updater.dispatcher
        
        echo_handler = MessageHandler(Filters.text & (~Filters.command), self.echo)
        dispatcher.add_handler(echo_handler)
        
        # 3. polling.
        self.updater.start_polling()
    else:
        self.sys_log.debug("msg_bot already assigned.")

##### v1.0

In [ ]:
   
def get_messenger_bot(self, ):
    
    # 1. set bot.
        # a. chat_id 는 env 동일.
    self.sys_log.info("self.msg_bot : {}".format(self.msg_bot))
    
    if self.msg_bot is None:
        self.msg_bot = telegram.Bot(token=self.config.trader_set.token)
        self.chat_id = self.config.trader_set.chat_id
        
        # 2. get updater.
            # a. init.
        self.user_text = None # messenger buffer.
        
        self.updater = Updater(token=self.config.trader_set.token, use_context=True)
        dispatcher = self.updater.dispatcher
        
        echo_handler = MessageHandler(Filters.text & (~Filters.command), self.echo)
        dispatcher.add_handler(echo_handler)
        
        # 3. polling.
        self.updater.start_polling()
    else:
        self.sys_log.debug("msg_bot already assigned.")

#### push_msg

In [ ]:
def push_msg(self, msg):    
    try:
        self.msg_bot.sendMessage(chat_id=self.chat_id, text=msg)
    except Exception as e:
        self.sys_log.error(e)

#### read_and_write_config

In [ ]:
def read_and_write_config(self, mode='r', config_edited=None):
    
    try:
        file_config = open(self.path_config, mode)

        if mode == 'r':
            self.config = EasyDict(json.load(file_config))
            
        elif mode == 'w':
            assert config_edited is not None, "assert config_edited is not None"
            json.dump(config_edited, file_config, indent=2)
        else:
            assert mode in ['r', 'w'], "assert mode in ['r', 'w']"

        # 1. opened files should be closed.
        file_config.close()

    except Exception as e:
        msg = "error in read_and_write_config : {}".format(e)
        self.sys_log.error(msg)
        self.push_msg(self, msg)
        

#### get_tables

In [ ]:
def get_tables(self, ):

    """
    v1.0
        we need this func. only once (program start, loading saved data.)
        replace csv to excel : preserving dtypes.
        
    last confirmed at, 20240505 0910.
    """
    
    self.table_condition = pd.read_excel(self.path_table_condition)
    self.table_trade = pd.read_excel(self.path_table_trade)
    self.table_log = pd.read_excel(self.path_table_log)

#### set_tables

In [ ]:
def set_tables(self, ):

    """
    v1.0
        we need this func. only once (program start, loading saved data.)        
        replace csv to excel : preserving dtypes.
        
    last confirmed at, 20240505 0910.
    """
    
    # self.table_condition.to_xlsx(self.path_table_condition)
    # self.table_trade.astype(str).to_excel(self.path_table_trade, index=False)
    # self.table_log.astype(str).to_excel(self.path_table_log, index=False)
    self.table_trade.to_excel(self.path_table_trade, index=False)
    self.table_log.to_excel(self.path_table_log, index=False)

### LoopTableCondition

In [ ]:
bank.streamer = bank.get_streamer(bank)

In [ ]:
bank.get_df_new_by_streamer(bank)
len(bank.df_res)# .tail()

In [ ]:
bank.symbol = 'THETAUSDT'
_ = bank.get_df_new(bank)
bank.df_res.tail()

In [ ]:
bank.get_df_mtf(bank)

In [ ]:
bank.get_wave_info(bank)
bank.df_res.dtypes#tail()

In [ ]:
bank.set_price_and_open_signal(bank)

In [ ]:
bank.get_side_open(bank)
print(bank.side_open)

In [ ]:
# table_condition = pd.read_excel("D:\Projects\SystemTrading\JnQ\Bank\data\condition.csv")
# %timeit -n1 -r100 table_condition = pd.read_excel("D:\Projects\SystemTrading\JnQ\Bank\data\condition.csv") # 1.17 ms ± 93.4 µs per loop (mean ± std. dev. of 100 runs, 1 loop each)
# %timeit -n1 -r100 table_condition = pd.read_excel("D:\Projects\SystemTrading\JnQ\Bank\data\condition.xlsx") # 11.8 ms ± 447 µs per loop (mean ± std. dev. of 100 runs, 1 loop each)

In [ ]:
# bank.table_trade.info()
# bank.table_log.info()
# bank.symbol = 'THETAUSDT'
bank.code = "{}_{}".format(bank.symbol, int(time.time()))
print(bank.symbol)
print(bank.code)

In [ ]:
bank.config.lvrg_set.leverage = 1
bank.config.trader_set.fee_limit = 0.0002
bank.config.trader_set.fee_market  = 0.0004

#### get_streamer

In [ ]:

def get_streamer(self):
    
    row_list = literal_eval(self.config.trader_set.row_list)
    itv_list = literal_eval(self.config.trader_set.itv_list)
    rec_row_list = literal_eval(self.config.trader_set.rec_row_list)
    use_rows, days = calc_rows_and_days(itv_list, row_list, rec_row_list)

    back_df = pd.read_feather(self.config.trader_set.back_data_path, columns=None, use_threads=True).set_index("index")

    if self.config.trader_set.start_datetime != "None":
        target_datetime = pd.to_datetime(self.config.trader_set.start_datetime)
        if target_datetime in back_df.index:
            start_idx = np.argwhere(back_df.index == target_datetime).item()
        else:
            start_idx = use_rows
    else:
        start_idx = use_rows

    # 시작하는 idx 는 필요로하는 rows 보다 커야함.
    assert start_idx >= use_rows, "more dataframe rows required"

    for i in range(start_idx + 1, len(back_df)):  # +1 for i inclusion
        yield back_df.iloc[i - use_rows:i]


#### get_df_new_by_streamer

In [ ]:

def get_df_new_by_streamer(self, ):
    
    try:
        self.df_res = next(self.streamer)  # Todo, 무결성 검증 미진행
    except Exception as e:
        msg = "error in get_df_new_by_streamer : {}".format(e)
        self.sys_log.error(msg)
        self.push_msg(self, msg)
        # self.kill_proc()  # error 처리할 필요없이, backtrader 프로그램 종료
        # return None, None   # output len 유지
    # else:
    #     return df_res

#### get_df_new

In [ ]:

def get_df_new(self, calc_rows=True, mode="OPEN"):

    """
    v2.0
        add self.config.trader_set.get_df_new_timeout

    last confirmed at, 20240414 2347
    """

    start_time = time.time()
    while 1:
        try:
            row_list = literal_eval(self.config.trader_set.row_list)
            if calc_rows:
                itv_list = literal_eval(self.config.trader_set.itv_list)
                rec_row_list = literal_eval(self.config.trader_set.rec_row_list)
                use_rows, days = calc_rows_and_days(itv_list, row_list, rec_row_list)
            else:
                use_rows, days = row_list[0], 1  # load_df_new3 사용 의도 = close, time logging
            # print(use_rows, days)
            use_rows = min(1500, use_rows)

            df_new, _ = concat_candlestick(self.symbol, '1m',
                                            days=days,
                                            limit=use_rows,
                                            timesleep=None,
                                            show_process=0)

            # 1. df_new validity check (Stock 의 경우 59.999 초를 더해주어야함, timeindex 의 second = 00)
            #    -a. 2020-12-25 19:02:59.999000 --> 19:03:00 이전까지의 데이터를 의미함.
            #    -b. 2020-12-25 19:02:00 --> 19:03:00 이전까지의 데이터를 의미함.
            #    a. datetime timestamp() 의 기준은 second 임.
            if str(df_new.index[-1]).split(':')[-1] == '00':
                last_row_timestamp = datetime.timestamp(df_new.index[-1]) + 59.999
            else:
                last_row_timestamp = datetime.timestamp(df_new.index[-1])

            if last_row_timestamp < datetime.now().timestamp():
                self.sys_log.warning("dataframe last row validation warning, last index : {}".format(str(df_new.index[-1])))
                if time.time() - start_time > self.config.trader_set.get_df_new_timeout:
                    return
                time.sleep(self.config.trader_set.api_term)
                continue

            if mode == "OPEN":
                self.sys_log.info("complete df_new's time : {}".format(datetime.now()))

        except Exception as e:
            msg = "error in get_df_new : {}".format(e)
            self.sys_log.error(msg)
            if 'sum_df' not in msg:
                self.push_msg(self, msg)
            #        1. adequate term for retries
            time.sleep(self.config.trader_set.api_term)
            continue
        else:
            self.df_res = df_new
            break


#### get_df_mtf

##### v2.0

In [ ]:
def get_df_mtf(self, mode="OPEN", row_slice=True):

    """
    v2.0   
        remove not used phase.
        Class mode.

    last confirmed at, 20240412 2005.
    """

    try:
        make_itv_list = [m_itv.replace('m', 'T') for m_itv in literal_eval(self.config.trader_set.itv_list)]
        row_list = literal_eval(self.config.trader_set.row_list)
        rec_row_list = literal_eval(self.config.trader_set.rec_row_list)
        offset_list = literal_eval(self.config.trader_set.offset_list)

        assert len(make_itv_list) == len(offset_list), "length of itv & offset_list should be equal"
        htf_df_list = [to_htf(self.df_res, itv=itv_, offset=offset_) for itv_idx, (itv_, offset_) in enumerate(zip(make_itv_list, offset_list)) if itv_idx != 0]
        htf_df_list.insert(0, self.df_res)
        

        # 1. row slicing considered latecy for indicator gerenation.
        if row_slice:
            df, df_3T, df_5T, df_15T, df_30T, df_H, df_4H = [df_s.iloc[-row_list[row_idx]:].copy() for row_idx, df_s in enumerate(htf_df_list)]
            rec_df, rec_df_3T, rec_df_5T, rec_df_15T, rec_df_30T, rec_df_H, rec_df_4H = [df_s.iloc[-rec_row_list[row_idx]:].copy() for row_idx, df_s in enumerate(htf_df_list)]
        else:
            df, df_3T, df_5T, df_15T, df_30T, df_H, df_4H = htf_df_list
            rec_df, rec_df_3T, rec_df_5T, rec_df_15T, rec_df_30T, rec_df_H, rec_df_4H = htf_df_list

    except Exception as e:
        self.sys_log.error("error in sync_check :", e)
    # else:
    #     return df

#### get_wave_info

##### v2.0

In [ ]:

def get_wave_info(self, mode="OPEN"):

    """
    v2.0
        Class mode.
    
    last confirmed at, 20240412 2025
    """
    
    roll_hl_cnt = 3
    
    try:
        # 1. self.config.tr_set.wave_period1
        if itv_to_number(self.config.tr_set.wave_itv1) > 1:
            offset = '1h' if self.config.tr_set.wave_itv1 != 'D' else '9h'
            htf_df_ = to_htf(self.df_res, self.config.tr_set.wave_itv1, offset=offset)  # to_htf 는 ohlc, 4개의 col 만 존재 (현재까지)
            htf_df = htf_df_[~pd.isnull(htf_df_.close)]

            htf_df = wave_range_cci_v4_1(htf_df, self.config.tr_set.wave_period1, itv=self.config.tr_set.wave_itv1)

            cols = list(htf_df.columns[4:])  # 15T_ohlc 를 제외한 wave_range_cci_v4 로 추가된 cols, 다 넣어버리기 (추후 혼란 방지)

            valid_co_prime_idx, valid_cu_prime_idx, roll_co_idx_arr, roll_cu_idx_arr = roll_wave_hl_idx_v5(htf_df,
                                                                                                           self.config.tr_set.wave_itv1,
                                                                                                           self.config.tr_set.wave_period1,
                                                                                                           roll_hl_cnt=roll_hl_cnt)

            """ 
            1. wave_bb 의 경우 roll_hl 의 기준이 co <-> cu 변경됨 (cci 와 비교)
            2. wave_bb : high_fill_ -> cu_prime_idx 사용
            """
            htf_df = get_roll_wave_data_v2(htf_df, valid_co_prime_idx, roll_co_idx_arr, 'wave_high_fill_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1), roll_hl_cnt)
            cols += list(htf_df.columns[-roll_hl_cnt:])

            htf_df = get_roll_wave_data_v2(htf_df, valid_cu_prime_idx, roll_cu_idx_arr, 'wave_low_fill_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1), roll_hl_cnt)
            cols += list(htf_df.columns[-roll_hl_cnt:])

            htf_df = wave_range_ratio_v4_2(htf_df, self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1, roll_hl_cnt=roll_hl_cnt)
            cols += list(htf_df.columns[-4:])

            self.df_res = get_wave_length(self.df_res, valid_co_prime_idx, valid_cu_prime_idx, roll_co_idx_arr, roll_cu_idx_arr, self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1, roll_hl_cnt=roll_hl_cnt)
            cols += list(htf_df.columns[-4:])

            # ------ 필요한 cols 만 join (htf's idx 정보는 ltf 와 sync. 가 맞지 않음 - join 불가함) ------ #
            self.df_res.drop(cols, inplace=True, axis=1, errors='ignore')
            self.df_res = self.df_res.join(to_lower_tf_v4(self.df_res, htf_df, cols, backing_i=0, ltf_itv='T'), how='inner')

        else:
            self.df_res = wave_range_cci_v4_1(self.df_res, self.config.tr_set.wave_period1, itv=self.config.tr_set.wave_itv1)

            valid_co_prime_idx, valid_cu_prime_idx, roll_co_idx_arr, roll_cu_idx_arr = roll_wave_hl_idx_v5(self.df_res,
                                                                                                           self.config.tr_set.wave_itv1,
                                                                                                           self.config.tr_set.wave_period1,
                                                                                                           roll_hl_cnt=roll_hl_cnt)

            self.df_res = get_roll_wave_data_v2(self.df_res, valid_co_prime_idx, roll_co_idx_arr, 'wave_high_fill_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1), roll_hl_cnt)
            self.df_res = get_roll_wave_data_v2(self.df_res, valid_cu_prime_idx, roll_cu_idx_arr, 'wave_low_fill_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1), roll_hl_cnt)

            self.df_res = wave_range_ratio_v4_2(self.df_res, self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1, roll_hl_cnt=roll_hl_cnt)

            self.df_res = get_wave_length(self.df_res, valid_co_prime_idx, valid_cu_prime_idx, roll_co_idx_arr, roll_cu_idx_arr, self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1, roll_hl_cnt=roll_hl_cnt)

        # if self.config.tr_set.wave_itv1 != self.config.tr_set.wave_itv2 or self.config.tr_set.wave_period1 != self.config.tr_set.wave_period2:
        #     if itv_to_number(self.config.tr_set.wave_itv2) > 1:
        #         offset = '1h' if self.config.tr_set.wave_itv2 != 'D' else '9h'
        #         htf_df = to_htf(self.df_res, self.config.tr_set.wave_itv2, offset=offset)
        #         htf_df = wave_range_cci_v4_1(htf_df, self.config.tr_set.wave_period2, itv=self.config.tr_set.wave_itv2)
        #         # htf_df = wave_range_bb_v1(htf_df, self.config.tr_set.wave_period2, itv=self.config.tr_set.wave_itv2)

        #         # cols = list(htf_df.columns[-15:-4])  # except idx col
        #         cols = list(htf_df.columns[4:])  # 15T_ohlc 를 제외한 wave_range_cci_v4 로 추가된 cols, 다 넣어버리기 (추후 혼란 방지)

        #         # ------ 필요한 cols 만 join (htf's idx 정보는 ltf 와 sync. 가 맞지 않음 - join 불가함) ------ #
        #         self.df_res.drop(cols, inplace=True, axis=1, errors='ignore')
        #         self.df_res = self.df_res.join(to_lower_tf_v4(self.df_res, htf_df, cols, backing_i=0, ltf_itv='T'), how='inner')  # tf_entry 진입이면, backing_i = 0 가 가능한 것 아닌가.

        #     else:
        #         self.df_res = wave_range_cci_v4_1(self.df_res, self.config.tr_set.wave_period2, itv=self.config.tr_set.wave_itv2)
        #         self.df_res = wave_range_bb_v1(self.df_res, self.config.tr_set.wave_period2, itv=self.config.tr_set.wave_itv2)
                
    except Exception as e:
        self.sys_log.error("error in get_wave_info : {}".format(e))

#### set_price_and_open_signal

##### v2.0

In [ ]:
def set_price_and_open_signal(self, mode='OPEN', env='BANK'):
    
    """
    v2.0
        Class mode.
    		use self, as parameter.
            modulize self.short_open_res1 *=  phase.
            include np.timeidx
    
    last confirmed at, 20240504 2318
    """
    
    self.len_df = len(self.df_res)
    self.np_timeidx = np.array([intmin_np(date_) for date_ in self.df_res.index.to_numpy()])     
    open_, high, low, close = [self.df_res[col_].to_numpy() for col_ in ['open', 'high', 'low', 'close']]
    
    self.set_price_box(self, )
    self.set_price(self, close)    
    
    if env == 'IDEP':
        self.adj_price_unit(self,)
     
    if mode == 'OPEN':        
        self.get_open_res(self, )
        self.adj_wave_point(self, )
        self.adj_wave_update_hl_rejection(self, )
        self.validate_price(self, close)        
        self.get_reward_risk_ratio(self, )

##### set_price_box

In [ ]:

def set_price_box(self, ):

    """
    2. set tr_set's 1, 0 & spread
        cu's roll_high_[:, -1] = prev_high & cu's roll_low_[:, -1] = current_low
        co's roll_low_[:, -1] = prev_low & co's roll_high_[:, -1] = current_high
        2 사용하는 이유 : tp_1 을 p2_box 기준으로 설정하가 위함 --> enex_pairing_v4 function 과 호환되지 않음
    """        
    
    roll_hl_cnt = 3

    roll_highs1 = [self.df_res['wave_high_fill_{}{}_-{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1, cnt_ + 1)].to_numpy() for cnt_ in reversed(range(roll_hl_cnt))]
    roll_lows1 = [self.df_res['wave_low_fill_{}{}_-{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1, cnt_ + 1)].to_numpy() for cnt_ in reversed(range(roll_hl_cnt))]

    wave_high_fill1_ = self.df_res['wave_high_fill_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1)].to_numpy()
    wave_low_fill1_ = self.df_res['wave_low_fill_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1)].to_numpy()
    
    self.df_res['short_tp_1_{}'.format(self.config.selection_id)] = wave_low_fill1_  # wave_low_fill1_ b2_low_5T
    self.df_res['short_tp_0_{}'.format(self.config.selection_id)] = roll_highs1[-1]  # roll_highs1[-1] wave_high_fill1_
    self.df_res['long_tp_1_{}'.format(self.config.selection_id)] = wave_high_fill1_  # wave_high_fill1_ b2_high_5T
    self.df_res['long_tp_0_{}'.format(self.config.selection_id)] = roll_lows1[-1]  # roll_lows1[-1]  wave_low_fill1_

    self.df_res['short_ep1_1_{}'.format(self.config.selection_id)] = wave_low_fill1_  # wave_low_fill1_   # b2_low_5T
    self.df_res['short_ep1_0_{}'.format(self.config.selection_id)] = wave_high_fill1_  # wave_high_fill1_
    self.df_res['long_ep1_1_{}'.format(self.config.selection_id)] = wave_high_fill1_  # wave_high_fill1_   # b2_high_5T
    self.df_res['long_ep1_0_{}'.format(self.config.selection_id)] = wave_low_fill1_  # wave_low_fill1_

    # --> p2's self.price_entry use p1's self.price_entry
    self.df_res['short_ep2_1_{}'.format(self.config.selection_id)] = wave_low_fill1_  # wave_low_fill2_   # b2_low_5T
    self.df_res['short_ep2_0_{}'.format(self.config.selection_id)] = wave_high_fill1_  # wave_high_fill2_
    self.df_res['long_ep2_1_{}'.format(self.config.selection_id)] = wave_high_fill1_  # wave_high_fill2_   # b2_high_5T
    self.df_res['long_ep2_0_{}'.format(self.config.selection_id)] = wave_low_fill1_  # wave_low_fill2_

    # --> self.price_stop_loss use p1's low, (allow prev_low as self.price_stop_loss for p1_hhm only)
    self.df_res['short_out_1_{}'.format(self.config.selection_id)] = wave_low_fill1_  # wave_low_fill1_   # wave_low_fill2_   # b2_low_5T
    self.df_res['short_out_0_{}'.format(self.config.selection_id)] = roll_highs1[-1]  # roll_highs1[-1] if not self.config.tr_set.check_hlm else wave_high_fill1_   # roll_highs2[-1]  # roll_high_[:, -2]
    self.df_res['long_out_1_{}'.format(self.config.selection_id)] = wave_high_fill1_  # wave_high_fill1_   # wave_high_fill2_   # b2_high_5T
    self.df_res['long_out_0_{}'.format(self.config.selection_id)] = roll_lows1[-1]  # roll_lows1[-1] if not self.config.tr_set.check_hlm else wave_low_fill1_   # roll_lows2[-1]

    self.df_res['short_tp_gap_{}'.format(self.config.selection_id)] = abs(self.df_res['short_tp_1_{}'.format(self.config.selection_id)] - self.df_res['short_tp_0_{}'.format(self.config.selection_id)])
    self.df_res['long_tp_gap_{}'.format(self.config.selection_id)] = abs(self.df_res['long_tp_1_{}'.format(self.config.selection_id)] - self.df_res['long_tp_0_{}'.format(self.config.selection_id)])
    self.df_res['short_ep1_gap_{}'.format(self.config.selection_id)] = abs(self.df_res['short_ep1_1_{}'.format(self.config.selection_id)] - self.df_res['short_ep1_0_{}'.format(self.config.selection_id)])
    self.df_res['long_ep1_gap_{}'.format(self.config.selection_id)] = abs(self.df_res['long_ep1_1_{}'.format(self.config.selection_id)] - self.df_res['long_ep1_0_{}'.format(self.config.selection_id)])

    self.df_res['short_out_gap_{}'.format(self.config.selection_id)] = abs(self.df_res['short_out_1_{}'.format(self.config.selection_id)] - self.df_res['short_out_0_{}'.format(self.config.selection_id)])
    self.df_res['long_out_gap_{}'.format(self.config.selection_id)] = abs(self.df_res['long_out_1_{}'.format(self.config.selection_id)] - self.df_res['long_out_0_{}'.format(self.config.selection_id)])
    self.df_res['short_ep2_gap_{}'.format(self.config.selection_id)] = abs(self.df_res['short_ep2_1_{}'.format(self.config.selection_id)] - self.df_res['short_ep2_0_{}'.format(self.config.selection_id)])
    self.df_res['long_ep2_gap_{}'.format(self.config.selection_id)] = abs(self.df_res['long_ep2_1_{}'.format(self.config.selection_id)] - self.df_res['long_ep2_0_{}'.format(self.config.selection_id)])

    self.df_res['short_spread_{}'.format(self.config.selection_id)] = (self.df_res['short_tp_0_{}'.format(self.config.selection_id)].to_numpy() / self.df_res['short_tp_1_{}'.format(self.config.selection_id)].to_numpy() - 1) / 2
    self.df_res['long_spread_{}'.format(self.config.selection_id)] = (self.df_res['long_tp_1_{}'.format(self.config.selection_id)].to_numpy() / self.df_res['long_tp_0_{}'.format(self.config.selection_id)].to_numpy() - 1) / 2

    self.df_res['short_lvrg_needed_{}'.format(self.config.selection_id)] = (1 / self.df_res['short_spread_{}'.format(self.config.selection_id)]) * self.table_condition_row.lvrg_k
    self.df_res['long_lvrg_needed_{}'.format(self.config.selection_id)] = (1 / self.df_res['long_spread_{}'.format(self.config.selection_id)]) * self.table_condition_row.lvrg_k

    if self.table_condition_row.lvrg_ceiling:
        self.df_res['short_lvrg_needed_{}'.format(self.config.selection_id)][self.df_res['short_lvrg_needed_{}'.format(self.config.selection_id)] > self.table_condition_row.lvrg_max_short] = self.table_condition_row.lvrg_max_short
        self.df_res['long_lvrg_needed_{}'.format(self.config.selection_id)][self.df_res['long_lvrg_needed_{}'.format(self.config.selection_id)] > self.table_condition_row.lvrg_max_long] = self.table_condition_row.lvrg_max_long




##### set_price

In [ ]:

def set_price(self, close):    
    
    # a. self.price_take_profit    
        # i. 기준 : tp_1, gap : tp_box
    self.df_res['short_tp_{}'.format(self.config.selection_id)] = self.df_res['short_tp_1_{}'.format(self.config.selection_id)].to_numpy() - self.df_res['short_tp_gap_{}'.format(self.config.selection_id)].to_numpy() * self.config.tr_set.tp_gap
    self.df_res['long_tp_{}'.format(self.config.selection_id)] = self.df_res['long_tp_1_{}'.format(self.config.selection_id)].to_numpy() + self.df_res['long_tp_gap_{}'.format(self.config.selection_id)].to_numpy() * self.config.tr_set.tp_gap
    
    # b. self.price_entry
        # i. limit_ep1
    if self.config.ep_set.entry_type == "LIMIT": 
            # 1. 기준 : ep1_0, gap : ep1_box
        self.df_res['short_ep1_{}'.format(self.config.selection_id)] = self.df_res['short_ep1_1_{}'.format(self.config.selection_id)].to_numpy() + self.df_res['short_ep1_gap_{}'.format(self.config.selection_id)].to_numpy() * self.config.tr_set.ep1_gap
        self.df_res['long_ep1_{}'.format(self.config.selection_id)] = self.df_res['long_ep1_1_{}'.format(self.config.selection_id)].to_numpy() - self.df_res['long_ep1_gap_{}'.format(self.config.selection_id)].to_numpy() * self.config.tr_set.ep1_gap        
        # ii. market_ep1
    else:
        self.df_res['short_ep1_{}'.format(self.config.selection_id)] = close
        self.df_res['long_ep1_{}'.format(self.config.selection_id)] = close    
    
        # ii. limit_ep2
    if self.config.ep_set.point2.entry_type == "LIMIT":
            # 1. 기준 : ep2_0, gap : ep2_box
        self.df_res['short_ep2_{}'.format(self.config.selection_id)] = self.df_res['short_ep2_1_{}'.format(self.config.selection_id)].to_numpy() + self.df_res['short_ep2_gap_{}'.format(self.config.selection_id)].to_numpy() * self.config.tr_set.ep2_gap
        self.df_res['long_ep2_{}'.format(self.config.selection_id)] = self.df_res['long_ep2_1_{}'.format(self.config.selection_id)].to_numpy() - self.df_res['long_ep2_gap_{}'.format(self.config.selection_id)].to_numpy() * self.config.tr_set.ep2_gap        
        # iv. market_ep2
    else:
        self.df_res['short_ep2_{}'.format(self.config.selection_id)] = close
        self.df_res['long_ep2_{}'.format(self.config.selection_id)] = close

    # c. self.price_stop_loss
    if self.config.tr_set.check_hlm == 0:
        # ii. 기준 : out_0, gap : out_box
        self.df_res['short_out_{}'.format(self.config.selection_id)] = self.df_res['short_out_0_{}'.format(self.config.selection_id)].to_numpy() + self.df_res['short_out_gap_{}'.format(self.config.selection_id)].to_numpy() * self.config.tr_set.out_gap  
        self.df_res['long_out_{}'.format(self.config.selection_id)] = self.df_res['long_out_0_{}'.format(self.config.selection_id)].to_numpy() - self.df_res['long_out_gap_{}'.format(self.config.selection_id)].to_numpy() * self.config.tr_set.out_gap  

    elif self.config.tr_set.check_hlm == 1:
        self.df_res['short_out_{}'.format(self.config.selection_id)] = self.df_res['short_ep1_0_{}'.format(self.config.selection_id)].to_numpy() + self.df_res['short_ep1_gap_{}'.format(self.config.selection_id)].to_numpy() * self.config.tr_set.out_gap  
        self.df_res['long_out_{}'.format(self.config.selection_id)] = self.df_res['long_ep1_0_{}'.format(self.config.selection_id)].to_numpy() - self.df_res['long_ep1_gap_{}'.format(self.config.selection_id)].to_numpy() * self.config.tr_set.out_gap  

    else:  # p2_hlm
        # ii. 기준 : ep2_0, gap : ep2_box
        self.df_res['short_out_{}'.format(self.config.selection_id)] = self.df_res['short_ep2_1_{}'.format(self.config.selection_id)].to_numpy() + self.df_res['short_ep2_gap_{}'.format(self.config.selection_id)].to_numpy() * self.config.tr_set.ep2_gap  # * 5
        self.df_res['long_out_{}'.format(self.config.selection_id)] = self.df_res['long_ep2_1_{}'.format(self.config.selection_id)].to_numpy() - self.df_res['long_ep2_gap_{}'.format(self.config.selection_id)].to_numpy() * self.config.tr_set.ep2_gap  # * 5



##### adj_price_unit

In [ ]:

def adj_price_unit(self):   
    
    calc_with_hoga_unit_vecto = np.vectorize(calc_with_precision)
    self.df_res['short_tp_{}'.format(self.config.selection_id)] = calc_with_hoga_unit_vecto(self.df_res['short_tp_{}'.format(self.config.selection_id)].to_numpy(), 2)
    self.df_res['long_tp_{}'.format(self.config.selection_id)] = calc_with_hoga_unit_vecto(self.df_res['long_tp_{}'.format(self.config.selection_id)].to_numpy(), 2)
    self.df_res['short_ep1_{}'.format(self.config.selection_id)] = calc_with_hoga_unit_vecto(self.df_res['short_ep1_{}'.format(self.config.selection_id)].to_numpy(), 2)
    self.df_res['long_ep1_{}'.format(self.config.selection_id)] = calc_with_hoga_unit_vecto(self.df_res['long_ep1_{}'.format(self.config.selection_id)].to_numpy(), 2)
    self.df_res['short_ep2_{}'.format(self.config.selection_id)] = calc_with_hoga_unit_vecto(self.df_res['short_ep2_{}'.format(self.config.selection_id)].to_numpy(), 2)
    self.df_res['long_ep2_{}'.format(self.config.selection_id)] = calc_with_hoga_unit_vecto(self.df_res['long_ep2_{}'.format(self.config.selection_id)].to_numpy(), 2)
    self.df_res['short_out_{}'.format(self.config.selection_id)] = calc_with_hoga_unit_vecto(self.df_res['short_out_{}'.format(self.config.selection_id)].to_numpy(), 2)
    self.df_res['long_out_{}'.format(self.config.selection_id)] = calc_with_hoga_unit_vecto(self.df_res['long_out_{}'.format(self.config.selection_id)].to_numpy(), 2)




##### get_open_res

In [ ]:

def get_open_res(self, ):
    
    #    a. init open_res
    self.short_open_res1 = np.ones(self.len_df)  # .astype(object)
    self.long_open_res1 = np.ones(self.len_df)  # .astype(object)
    self.short_open_res2 = np.ones(self.len_df)  # .astype(object)
    self.long_open_res2 = np.ones(self.len_df)  # .astype(object)

    if self.config.show_detail:
        self.sys_log.info("init open_res")
        self.sys_log.info("np.sum(self.short_open_res1 == 1) : {}".format(np.sum(self.short_open_res1 == 1)))
        self.sys_log.info("np.sum(self.long_open_res1 == 1) : {}".format(np.sum(self.long_open_res1 == 1)))



##### adj_wave_point

In [ ]:

def adj_wave_point(self, ):
    
    #    a. wave_point
    # notnan_short_tc = ~pd.isnull(self.df_res['short_tc_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1)].to_numpy())  # isnull for object
    # notnan_long_tc = ~pd.isnull(self.df_res['long_tc_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1)].to_numpy())  # isnull for object

    notnan_cu = ~pd.isnull(self.df_res['wave_cu_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1)].to_numpy())  # isnull for object
    notnan_co = ~pd.isnull(self.df_res['wave_co_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1)].to_numpy())
    # notnan_cu2 = ~pd.isnull(self.df_res['wave_cu_{}{}'.format(self.config.tr_set.wave_itv2, self.config.tr_set.wave_period2)].to_numpy())  # isnull for object
    # notnan_co2 = ~pd.isnull(self.df_res['wave_co_{}{}'.format(self.config.tr_set.wave_itv2, self.config.tr_set.wave_period2)].to_numpy())

    self.short_open_res1 *= self.df_res['wave_cu_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1)].to_numpy().astype(bool) * notnan_cu  # object로 변환되는 경우에 대응해, bool 로 재정의
    self.long_open_res1 *= self.df_res['wave_co_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1)].to_numpy().astype(bool) * notnan_co  # np.nan = bool type 으로 True 임..
    # self.short_open_res1 *= self.df_res['short_tc_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1)].to_numpy().astype(bool) * notnan_short_tc
    # self.long_open_res1 *= self.df_res['long_tc_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1)].to_numpy().astype(bool) * notnan_long_tc

    # self.short_open_res2 *= self.df_res['wave_cu_{}{}'.format(self.config.tr_set.wave_itv2, self.config.tr_set.wave_period2)].to_numpy().astype(bool) * notnan_cu2  # object로 변환되는 경우에 대응해, bool 로 재정의
    # self.long_open_res2 *= self.df_res['wave_co_{}{}'.format(self.config.tr_set.wave_itv2, self.config.tr_set.wave_period2)].to_numpy().astype(bool) * notnan_co2  # np.nan = bool type 으로 True 임..
    # self.short_open_res2 *= self.df_res['short_tc_{}{}'.format(self.config.tr_set.wave_itv2, self.config.tr_set.tc_period)].to_numpy()
    # self.long_open_res2 *= self.df_res['long_tc_{}{}'.format(self.config.tr_set.wave_itv2, self.config.tr_set.tc_period)].to_numpy()

    if self.config.show_detail:
        self.sys_log.info("wave_point")
        self.sys_log.info("np.sum(self.short_open_res1 == 1) : {}".format(np.sum(self.short_open_res1 == 1)))
        self.sys_log.info("np.sum(self.long_open_res1 == 1) : {}".format(np.sum(self.long_open_res1 == 1)))
        # self.sys_log.info("np.sum(self.short_open_res2 == 1) : {}".format(np.sum(self.short_open_res2 == 1)))
        # self.sys_log.info("np.sum(self.long_open_res2 == 1) : {}".format(np.sum(self.long_open_res2 == 1)))



##### adj_wave_update_hl_rejection

In [ ]:

def adj_wave_update_hl_rejection(self, ):
    
    notnan_update_low_cu = ~pd.isnull(self.df_res['wave_update_low_cu_bool_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1)].to_numpy())
    notnan_update_high_co = ~pd.isnull(self.df_res['wave_update_high_co_bool_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1)].to_numpy())
    # notnan_update_low_cu2 = ~pd.isnull(self.df_res['wave_update_low_cu_bool_{}{}'.format(self.config.tr_set.wave_itv2, self.config.tr_set.wave_period2)].to_numpy())
    # notnan_update_high_co2 = ~pd.isnull(self.df_res['wave_update_high_co_bool_{}{}'.format(self.config.tr_set.wave_itv2, self.config.tr_set.wave_period2)].to_numpy())

    self.short_open_res1 *= ~(self.df_res['wave_update_low_cu_bool_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1)].to_numpy().astype(bool)) * notnan_update_low_cu
    self.long_open_res1 *= ~(self.df_res['wave_update_high_co_bool_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1)].to_numpy().astype(bool)) * notnan_update_high_co
    # self.short_open_res2 *= ~(self.df_res['wave_update_low_cu_bool_{}{}'.format(self.config.tr_set.wave_itv2, self.config.tr_set.wave_period2)].to_numpy().astype(bool)) * notnan_update_low_cu2
    # self.long_open_res2 *= ~(self.df_res['wave_update_high_co_bool_{}{}'.format(self.config.tr_set.wave_itv2, self.config.tr_set.wave_period2)].to_numpy().astype(bool)) * notnan_update_high_co2

    if self.config.show_detail:
        self.sys_log.info("reject update_hl")
        self.sys_log.info("np.sum(self.short_open_res1 == 1) : {}".format(np.sum(self.short_open_res1 == 1)))
        self.sys_log.info("np.sum(self.long_open_res1 == 1) : {}".format(np.sum(self.long_open_res1 == 1)))
        # self.sys_log.info("np.sum(self.short_open_res2 == 1) : {}".format(np.sum(self.short_open_res2 == 1)))
        # self.sys_log.info("np.sum(self.long_open_res2 == 1) : {}".format(np.sum(self.long_open_res2 == 1)))


##### adj_wave_itv

In [ ]:

def adj_wave_itv(self, ):
    
    if self.config.tr_set.wave_itv1 != 'T':
        wave_itv1_num = itv_to_number(self.config.tr_set.wave_itv1)
        self.short_open_res1 *= self.np_timeidx % wave_itv1_num == (wave_itv1_num - 1)
        self.long_open_res1 *= self.np_timeidx % wave_itv1_num == (wave_itv1_num - 1)

        if self.config.show_detail:
            self.sys_log.info("self.config.tr_set.wave_itv1")
            self.sys_log.info("np.sum(self.short_open_res1 == 1) : {}".format(np.sum(self.short_open_res1 == 1)))
            self.sys_log.info("np.sum(self.long_open_res1 == 1) : {}".format(np.sum(self.long_open_res1 == 1)))

    if self.config.tr_set.wave_itv2 != 'T':
        wave_itv2_num = itv_to_number(self.config.tr_set.wave_itv2)
        self.short_open_res2 *= self.np_timeidx % wave_itv2_num == (wave_itv2_num - 1)
        self.long_open_res2 *= self.np_timeidx % wave_itv2_num == (wave_itv2_num - 1)

        if self.config.show_detail:
            self.sys_log.info("self.config.tr_set.wave_itv2")
            self.sys_log.info("np.sum(self.short_open_res2 == 1) : {}".format(np.sum(self.short_open_res2 == 1)))
            self.sys_log.info("np.sum(self.long_open_res2 == 1) : {}".format(np.sum(self.long_open_res2 == 1)))




##### adj_wave_term

In [ ]:

def adj_wave_term(self, ):
    
    wave_high_terms_cnt_fill1_ = self.df_res['wave_high_terms_cnt_fill_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1)].to_numpy()
    wave_low_terms_cnt_fill1_ = self.df_res['wave_low_terms_cnt_fill_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1)].to_numpy()

    self.short_open_res1 *= (wave_high_terms_cnt_fill1_ > self.config.tr_set.wave_greater2) & (wave_low_terms_cnt_fill1_ > self.config.tr_set.wave_greater1)
    self.long_open_res1 *= (wave_low_terms_cnt_fill1_ > self.config.tr_set.wave_greater2) & (wave_high_terms_cnt_fill1_ > self.config.tr_set.wave_greater1)

    # wave_high_terms_cnt_fill2_ = self.df_res['wave_high_terms_cnt_fill_{}{}'.format(self.config.tr_set.wave_itv2, self.config.tr_set.wave_period2)].to_numpy()
    # wave_low_terms_cnt_fill2_ = self.df_res['wave_low_terms_cnt_fill_{}{}'.format(self.config.tr_set.wave_itv2, self.config.tr_set.wave_period2)].to_numpy()

    # self.short_open_res2 *= (wave_high_terms_cnt_fill2_ > self.config.tr_set.wave_greater2) & (wave_low_terms_cnt_fill2_ > self.config.tr_set.wave_greater1)
    # self.long_open_res2 *= (wave_low_terms_cnt_fill2_ > self.config.tr_set.wave_greater2) & (wave_high_terms_cnt_fill2_ > self.config.tr_set.wave_greater1)

    if self.config.show_detail:
        self.sys_log.info("wave_term")
        self.sys_log.info("np.sum(self.short_open_res1 == 1) : {}".format(np.sum(self.short_open_res1 == 1)))
        self.sys_log.info("np.sum(self.long_open_res1 == 1) : {}".format(np.sum(self.long_open_res1 == 1)))
        # self.sys_log.info("np.sum(self.short_open_res2 == 1) : {}".format(np.sum(self.short_open_res2 == 1)))
        # self.sys_log.info("np.sum(self.long_open_res2 == 1) : {}".format(np.sum(self.long_open_res2 == 1)))


##### adj_wave_length

In [ ]:

def adj_wave_length(self, ):
    
    if self.config.tr_set.wave_length_min_short1 != "None":
        short_wave_length_fill_ = self.df_res['short_wave_length_fill_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1)].to_numpy()
        self.short_open_res1 *= short_wave_length_fill_ >= self.config.tr_set.wave_length_min_short1
        if self.config.show_detail:
            self.sys_log.info("wave_length_min_short1")
            self.sys_log.info("np.sum(self.short_open_res1 == 1) : {}".format(np.sum(self.short_open_res1 == 1)))

    if self.config.tr_set.wave_length_max_short1 != "None":
        short_wave_length_fill_ = self.df_res['short_wave_length_fill_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1)].to_numpy()
        self.short_open_res1 *= short_wave_length_fill_ <= self.config.tr_set.wave_length_max_short1
        if self.config.show_detail:
            self.sys_log.info("wave_length_max_short1")
            self.sys_log.info("np.sum(self.short_open_res1 == 1) : {}".format(np.sum(self.short_open_res1 == 1)))

    if self.config.tr_set.wave_length_min_long1 != "None":
        long_wave_length_fill_ = self.df_res['long_wave_length_fill_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1)].to_numpy()
        self.long_open_res1 *= long_wave_length_fill_ >= self.config.tr_set.wave_length_min_long1
        if self.config.show_detail:
            self.sys_log.info("wave_length_min_long1")
            self.sys_log.info("np.sum(self.long_open_res1 == 1) : {}".format(np.sum(self.long_open_res1 == 1)))

    if self.config.tr_set.wave_length_max_long1 != "None":
        long_wave_length_fill_ = self.df_res['long_wave_length_fill_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1)].to_numpy()
        self.long_open_res1 *= long_wave_length_fill_ <= self.config.tr_set.wave_length_max_long1
        if self.config.show_detail:
            self.sys_log.info("wave_length_max_long1")
            self.sys_log.info("np.sum(self.long_open_res1 == 1) : {}".format(np.sum(self.long_open_res1 == 1)))


##### validate_price

In [ ]:
def validate_price(self, close):

    short_tp_ = self.df_res['short_tp_{}'.format(self.config.selection_id)].to_numpy()
    short_ep1_ = self.df_res['short_ep1_{}'.format(self.config.selection_id)].to_numpy()
    short_ep2_ = self.df_res['short_ep2_{}'.format(self.config.selection_id)].to_numpy()
    short_out_ = self.df_res['short_out_{}'.format(self.config.selection_id)].to_numpy()

    long_tp_ = self.df_res['long_tp_{}'.format(self.config.selection_id)].to_numpy()
    long_ep1_ = self.df_res['long_ep1_{}'.format(self.config.selection_id)].to_numpy()
    long_ep2_ = self.df_res['long_ep2_{}'.format(self.config.selection_id)].to_numpy()
    long_out_ = self.df_res['long_out_{}'.format(self.config.selection_id)].to_numpy()

    
    #     a. p1 validate_price
    #             i. tr_set validation reject nan data
    #             ii. 정상 거래 위한 self.price_take_profit > self.price_entry           
    
    self.short_open_res1 *= (short_tp_ < short_ep1_) & (short_ep1_ < short_out_)
    #             iii. reject hl_out exec_openution -> close always < ep1_0 at wave_p1
    self.short_open_res1 *= close < short_out_  # self.df_res['short_ep1_0_{}'.format(self.config.selection_id)].to_numpy()
    # self.short_open_res1 *= close < short_ep1_   # reject entry exec_openution
    # short_out_  self.df_res['short_tp_0_{}'.format(self.config.selection_id)].to_numpy() self.df_res['short_ep1_0_{}'.format(self.config.selection_id)].to_numpy()

    self.long_open_res1 *= (long_tp_ > long_ep1_) & (long_ep1_ > long_out_)  # (long_tp_ > long_ep_)
    self.long_open_res1 *= close > long_out_  # self.df_res['long_ep1_0_{}'.format(self.config.selection_id)].to_numpy()
    # self.long_open_res1 *= close > long_ep1_  # reject entry exec_openution
    # long_out_ self.df_res['long_tp_0_{}'.format(self.config.selection_id)].to_numpy() self.df_res['long_ep1_0_{}'.format(self.config.selection_id)].to_numpy()

    
    #     b. p2 validate_price => deprecated => now, executed in en_ex_pairing() function.
    # self.short_open_res2 *= (short_ep2_ < short_out_) # tr_set validation (short_tp_ < short_ep_) # --> p2_box location (cannot be vectorized)
    # self.short_open_res2 *= close < short_out_    # reject hl_out exec_openution
    # self.long_open_res2 *= (long_ep2_ > long_out_)  # tr_set validation (long_tp_ > long_ep_) &   # p2's self.price_entry & self.price_stop_loss cannot be vectorized
    # self.long_open_res2 *= close > long_out_    # reject hl_out exec_openution


    self.df_res['short_open1_{}'.format(self.config.selection_id)] = self.short_open_res1 * (not self.config.pos_set.short_ban)
    self.df_res['long_open1_{}'.format(self.config.selection_id)] = self.long_open_res1 * (not self.config.pos_set.long_ban)

    self.df_res['short_open2_{}'.format(self.config.selection_id)] = self.short_open_res2
    self.df_res['long_open2_{}'.format(self.config.selection_id)] = self.long_open_res2

    if self.config.show_detail:
        self.sys_log.info("point validation")
        self.sys_log.info("np.sum(self.short_open_res1 == 1) : {}".format(np.sum(self.short_open_res1 == 1)))
        self.sys_log.info("np.sum(self.long_open_res1 == 1) : {}".format(np.sum(self.long_open_res1 == 1)))
        self.sys_log.info("np.sum(self.short_open_res2 == 1) : {}".format(np.sum(self.short_open_res2 == 1)))
        self.sys_log.info("np.sum(self.long_open_res2 == 1) : {}".format(np.sum(self.long_open_res2 == 1)))



##### get_reward_risk_ratio

In [ ]:

def get_reward_risk_ratio(self, ):
    
    start_time = time.time()
    short_tp_ = self.df_res['short_tp_{}'.format(self.config.selection_id)].to_numpy()
    short_ep1_ = self.df_res['short_ep1_{}'.format(self.config.selection_id)].to_numpy()
    short_out_ = self.df_res['short_out_{}'.format(self.config.selection_id)].to_numpy()

    long_tp_ = self.df_res['long_tp_{}'.format(self.config.selection_id)].to_numpy()
    long_ep1_ = self.df_res['long_ep1_{}'.format(self.config.selection_id)].to_numpy()
    long_out_ = self.df_res['long_out_{}'.format(self.config.selection_id)].to_numpy()

    if self.config.tr_set.check_hlm == 2:
        self.df_res['short_risk_reward_ratio_{}'.format(self.config.selection_id)] = np.nan
        self.df_res['long_risk_reward_ratio_{}'.format(self.config.selection_id)] = np.nan
    else:
        self.df_res['short_risk_reward_ratio_{}'.format(self.config.selection_id)] = abs((short_ep1_ / short_tp_ - self.config.trader_set.fee_limit - 1) / (short_ep1_ / short_out_ - self.config.trader_set.fee_market - 1))
        self.df_res['long_risk_reward_ratio_{}'.format(self.config.selection_id)] = abs((long_tp_ / long_ep1_ - self.config.trader_set.fee_limit - 1) / (long_out_ / long_ep1_ - self.config.trader_set.fee_market - 1))


##### legacy : conditions.

In [ ]:


        #    b. bars in a hour.
        #         timestamp_index = np.array(list(map(lambda x: int(datetime.timestamp(x)), pd.Series(df_res.index))))
        #         timestamp_index_shifted = np.array(list(map(lambda x: int(datetime.timestamp(x)) if not pd.isnull(x) else 0, pd.Series(df_res.index).shift(60))))
        #         bars_in_hour = timestamp_index - timestamp_index_shifted < 4800

        #         short_open_res1 *= bars_in_hour
        #         long_open_res1 *= bars_in_hour

        #         if self.config.show_detail:
        #             sys_log.warning("bars in a hour")
        #             sys_log.warning("np.sum(short_open_res1 == 1) : {}".format(np.sum(short_open_res1 == 1)))
        #             sys_log.warning("np.sum(long_open_res1 == 1) : {}".format(np.sum(long_open_res1 == 1)))



    #         i. magnetic level
    #     cu_wrr_32_ = df_res['cu_wrr_32_{}{}'.format(wave_itv1, wave_period1)].to_numpy()
    #     co_wrr_32_ = df_res['co_wrr_32_{}{}'.format(wave_itv1, wave_period1)].to_numpy()

    #     short_magnetic_tpg = np.vectorize(get_next_wave_level)(cu_wrr_32_, 'self.price_take_profit')
    #     long_magnetic_tpg = np.vectorize(get_next_wave_level)(co_wrr_32_, 'self.price_take_profit')

    #     df_res['short_tp_{}'.format(selection_id)] = df_res['short_tp_1_{}'.format(selection_id)].to_numpy() - df_res['short_tp_gap_{}'.format(selection_id)].to_numpy() * short_magnetic_tpg  # self.price_entry 와 마찬가지로, tpg 기준 가능
    #     df_res['long_tp_{}'.format(selection_id)] = df_res['long_tp_1_{}'.format(selection_id)].to_numpy() + df_res['long_tp_gap_{}'.format(selection_id)].to_numpy() * long_magnetic_tpg
    
    # df_res['short_tp_{}'.format(selection_id)] = df_res['short_out_1_{}'.format(selection_id)].to_numpy() - df_res[
    #     'short_out_gap_{}'.format(selection_id)].to_numpy() * tpg
    # df_res['long_tp_{}'.format(selection_id)] = df_res['long_out_1_{}'.format(selection_id)].to_numpy() + df_res[
    #     'long_out_gap_{}'.format(selection_id)].to_numpy() * tpg

    # df_res['short_tp_{}'.format(selection_id)] = short_tp_1 - short_epout_gap * tpg
    # df_res['long_tp_{}'.format(selection_id)] = long_tp_1 + long_epout_gap * tpg




        #              1. magnetic level
        # cu_wrr_32_ = df_res['cu_wrr_32_{}{}'.format(wave_itv1, wave_period1)].to_numpy()
        # co_wrr_32_ = df_res['co_wrr_32_{}{}'.format(wave_itv1, wave_period1)].to_numpy()

        #         short_magnetic_outg = np.vectorize(get_next_wave_level)(cu_wrr_32_)
        #         long_magnetic_outg = np.vectorize(get_next_wave_level)(co_wrr_32_)

        #         df_res['short_ep1_{}'.format(selection_id)] = df_res['short_tp_1_{}'.format(selection_id)].to_numpy() - df_res['short_tp_gap_{}'.format(selection_id)].to_numpy() * short_magnetic_outg  # self.price_entry 와 마찬가지로, tpg 기준 가능
        #         df_res['long_ep1_{}'.format(selection_id)] = df_res['long_tp_1_{}'.format(selection_id)].to_numpy() + df_res['long_tp_gap_{}'.format(selection_id)].to_numpy() * long_magnetic_outg

        #              3. 기준 : ep1_0, gap : tp_box
        # p1_hlm 을 위해선, tp_0 를 기준할 수 없음 --> ep1 & ep2 를 기준으로 진행
        # df_res['short_ep1_{}'.format(selection_id)] = df_res['short_ep1_0_{}'.format(selection_id)].to_numpy() + df_res['short_tp_gap_{}'.format(selection_id)].to_numpy() * epg1  # fibonacci 고려하면, self.price_take_profit / self.price_stop_loss gap 기준이 맞지 않을까
        # df_res['long_ep1_{}'.format(selection_id)] = df_res['long_ep1_0_{}'.format(selection_id)].to_numpy() - df_res['long_tp_gap_{}'.format(selection_id)].to_numpy() * epg1

        #              4. 기준 : tp_1, gap : tp_box [ fibo_ep ]
        # df_res['short_ep1_{}'.format(selection_id)] = df_res['short_tp_1_{}'.format(selection_id)].to_numpy() + df_res['short_tp_gap_{}'.format(selection_id)].to_numpy() * epg1  # fibonacci 고려하면, self.price_take_profit / self.price_stop_loss gap 기준이 맞지 않을까
        # df_res['long_ep1_{}'.format(selection_id)] = df_res['long_tp_1_{}'.format(selection_id)].to_numpy() - df_res['long_tp_gap_{}'.format(selection_id)].to_numpy() * epg1



        #              1. magnetic level
        # cu_wrr_32_ = df_res['cu_wrr_32_{}{}'.format(wave_itv1, wave_period1)].to_numpy()
        # co_wrr_32_ = df_res['co_wrr_32_{}{}'.format(wave_itv1, wave_period1)].to_numpy()

        #         short_magnetic_outg = np.vectorize(get_next_wave_level)(cu_wrr_32_)
        #         long_magnetic_outg = np.vectorize(get_next_wave_level)(co_wrr_32_)

        #         df_res['short_ep2_{}'.format(selection_id)] = df_res['short_tp_1_{}'.format(selection_id)].to_numpy() - df_res['short_tp_gap_{}'.format(selection_id)].to_numpy() * short_magnetic_outg  # self.price_entry 와 마찬가지로, tpg 기준 가능
        #         df_res['long_ep2_{}'.format(selection_id)] = df_res['long_tp_1_{}'.format(selection_id)].to_numpy() + df_res['long_tp_gap_{}'.format(selection_id)].to_numpy() * long_magnetic_outg

        #              3. 기준 : ep2_0, gap : out_box
        # df_res['short_ep2_{}'.format(selection_id)] = df_res['short_ep2_0_{}'.format(selection_id)].to_numpy() + df_res['short_out_gap_{}'.format(selection_id)].to_numpy() * epg2
        # df_res['long_ep2_{}'.format(selection_id)] = df_res['long_ep2_0_{}'.format(selection_id)].to_numpy() - df_res['long_out_gap_{}'.format(selection_id)].to_numpy() * epg2

        #              4. 기준 : tp_1, gap : tp_box [ fibo_ep ]
        # df_res['short_ep2_{}'.format(selection_id)] = df_res['short_tp_1_{}'.format(selection_id)].to_numpy() - df_res['short_tp_gap_{}'.format(selection_id)].to_numpy() * epg2  # fibonacci 고려하면, self.price_take_profit / self.price_stop_loss gap 기준이 맞지 않을까
        # df_res['long_ep2_{}'.format(selection_id)] = df_res['long_tp_1_{}'.format(selection_id)].to_numpy() + df_res['long_tp_gap_{}'.format(selection_id)].to_numpy() * epg2

        #              5. 기준 : out_0, gap : out_box [ fibo_ep ]
        # df_res['short_ep2_{}'.format(selection_id)] = df_res['short_out_0_{}'.format(selection_id)].to_numpy() + df_res[
        #     'short_out_gap_{}'.format(selection_id)].to_numpy() * epg2  # fibonacci 고려하면, self.price_take_profit / self.price_stop_loss gap 기준이 맞지 않을까
        # df_res['long_ep2_{}'.format(selection_id)] = df_res['long_out_0_{}'.format(selection_id)].to_numpy() - df_res[
        #     'long_out_gap_{}'.format(selection_id)].to_numpy() * epg2


        # cu_wrr_32_ = df_res['cu_wrr_32_{}{}'.format(wave_itv1, wave_period1)].to_numpy()
        # co_wrr_32_ = df_res['co_wrr_32_{}{}'.format(wave_itv1, wave_period1)].to_numpy()

        #     i. magnetic level
        #         short_magnetic_outg = np.vectorize(get_next_wave_level)(cu_wrr_32_, 'self.price_stop_loss')
        #         long_magnetic_outg = np.vectorize(get_next_wave_level)(co_wrr_32_, 'self.price_stop_loss')

        #         df_res['short_out_{}'.format(selection_id)] = df_res['short_tp_1_{}'.format(selection_id)].to_numpy() - df_res['short_tp_gap_{}'.format(selection_id)].to_numpy() * short_magnetic_outg  # self.price_entry 와 마찬가지로, tpg 기준 가능
        #         df_res['long_out_{}'.format(selection_id)] = df_res['long_tp_1_{}'.format(selection_id)].to_numpy() + df_res['long_tp_gap_{}'.format(selection_id)].to_numpy() * long_magnetic_outg

        #     iii. 기준 : tp_0, gap : tp_box
        # df_res['short_out_{}'.format(selection_id)] = df_res['short_tp_0_{}'.format(selection_id)].to_numpy() + df_res['short_tp_gap_{}'.format(selection_id)].to_numpy() * outg
        # df_res['long_out_{}'.format(selection_id)] = df_res['long_tp_0_{}'.format(selection_id)].to_numpy() - df_res['long_tp_gap_{}'.format(selection_id)].to_numpy() * outg

        #     iv. 기준 : ep1_0, gap : ep1_box
        # df_res['short_out_{}'.format(selection_id)] = df_res['short_ep1_0_{}'.format(selection_id)].to_numpy() + df_res['short_ep1_gap_{}'.format(selection_id)].to_numpy() * outg
        # df_res['long_out_{}'.format(selection_id)] = df_res['long_ep1_0_{}'.format(selection_id)].to_numpy() - df_res['long_ep1_gap_{}'.format(selection_id)].to_numpy() * outg

        # df_res['short_out_{}'.format(selection_id)] = df_res['short_ep1_1_{}'.format(selection_id)].to_numpy() + df_res['short_ep1_gap_{}'.format(selection_id)].to_numpy() * outg
        # df_res['long_out_{}'.format(selection_id)] = df_res['long_ep1_1_{}'.format(selection_id)].to_numpy() - df_res['long_ep1_gap_{}'.format(selection_id)].to_numpy() * outg

        #     v. 기준 : temporary refix for flexible p1_hhm
        # df_res['short_tp_0_{}'.format(selection_id)] = df_res['short_out_{}'.format(selection_id)]
        # df_res['long_tp_0_{}'.format(selection_id)] = df_res['long_out_{}'.format(selection_id)]


        

        # ------ 1_tr - ep1box as outg ------ #
        # df_res['short_out_{}'.format(selection_id)] = df_res['short_ep1_0_{}'.format(selection_id)].to_numpy() + df_res['short_ep1_gap_{}'.format(selection_id)].to_numpy() * outg
        # df_res['long_out_{}'.format(selection_id)] = df_res['long_ep1_0_{}'.format(selection_id)].to_numpy() - df_res['long_ep1_gap_{}'.format(selection_id)].to_numpy() * outg

        # ------ 1_tr - auto_calculation by ep1 ------ #
        # df_res['short_out_{}'.format(selection_id)] = df_res['short_ep1_{}'.format(selection_id)] + (df_res['short_ep1_{}'.format(selection_id)].to_numpy() - df_res['short_tp_{}'.format(selection_id)].to_numpy())
        # df_res['long_out_{}'.format(selection_id)] = df_res['long_ep1_{}'.format(selection_id)].to_numpy() - (df_res['long_tp_{}'.format(selection_id)].to_numpy() - df_res['long_ep1_{}'.format(selection_id)].to_numpy())

        # ------ tpbox as outg ------ #
        # df_res['short_out_{}'.format(selection_id)] = df_res['short_ep1_0_{}'.format(selection_id)].to_numpy() + df_res['short_tp_gap_{}'.format(selection_id)].to_numpy() * outg  # self.price_entry 와 마찬가지로, tpg 기준 가능
        # df_res['long_out_{}'.format(selection_id)] = df_res['long_ep1_0_{}'.format(selection_id)].to_numpy() - df_res['long_tp_gap_{}'.format(selection_id)].to_numpy() * outg

        # ------ [fibo_out] = 기준 : tp_0, gap = tp_box ------ #
        # df_res['short_out_{}'.format(selection_id)] = df_res['short_tp_0_{}'.format(selection_id)].to_numpy() + df_res['short_tp_gap_{}'.format(selection_id)].to_numpy() * outg  # self.price_entry 와 마찬가지로, tpg 기준 가능
        # df_res['long_out_{}'.format(selection_id)] = df_res['long_tp_0_{}'.format(selection_id)].to_numpy() - df_res['long_tp_gap_{}'.format(selection_id)].to_numpy() * outg
        #     i. 기준 : out_0, gap : out_box
        # df_res['short_out_{}'.format(selection_id)] = df_res['short_out_0_{}'.format(selection_id)].to_numpy() + df_res['short_out_gap_{}'.format(selection_id)].to_numpy() * outg
        # df_res['long_out_{}'.format(selection_id)] = df_res['long_out_0_{}'.format(selection_id)].to_numpy() - df_res['long_out_gap_{}'.format(selection_id)].to_numpy() * outg

        # df_res['short_out_{}'.format(selection_id)] = df_res['short_tp_0_{}'.format(selection_id)].to_numpy() + df_res['short_tp_gap_{}'.format(selection_id)].to_numpy() * outg
        # df_res['long_out_{}'.format(selection_id)] = df_res['long_tp_0_{}'.format(selection_id)].to_numpy() - df_res['long_tp_gap_{}'.format(selection_id)].to_numpy() * outg

        #     iii. ep2box as self.price_stop_loss
        # df_res['short_out_{}'.format(selection_id)] = df_res['short_ep2_0_{}'.format(selection_id)].to_numpy() + df_res['short_ep2_gap_{}'.format(selection_id)].to_numpy() * outg   # p2's ep_box 를 self.price_stop_loss 으로 사용한다?
        # df_res['long_out_{}'.format(selection_id)] = df_res['long_ep2_0_{}'.format(selection_id)].to_numpy() - df_res['long_ep2_gap_{}'.format(selection_id)].to_numpy() * outg


    # df_res['short_out_{}'.format(selection_id)] = short_tp_0 + short_tp_gap * outg            # 1. for hhm check -> 규칙성과 wave_range 기반 거래 기준의 hhm 확인
    # df_res['long_out_{}'.format(selection_id)] = long_tp_0 - long_tp_gap * outg




    #     roll_highs2 = [df_res['wave_high_fill_{}{}_-{}'.format(wave_itv2, wave_period2, cnt_ + 1)].to_numpy() for cnt_ in reversed(range(roll_hl_cnt))]
    #     roll_lows2 = [df_res['wave_low_fill_{}{}_-{}'.format(wave_itv2, wave_period2, cnt_ + 1)].to_numpy() for cnt_ in reversed(range(roll_hl_cnt))]

    #     wave_high_fill2_ = df_res['wave_high_fill_{}{}'.format(wave_itv2, wave_period2)].to_numpy()
    #     wave_low_fill2_ = df_res['wave_low_fill_{}{}'.format(wave_itv2, wave_period2)].to_numpy()

    # df_res['short_wave_spread_fill_{}{}'.format(wave_itv1, wave_period1)] = roll_highs1[-1] / wave_low_fill1_
    # df_res['long_wave_spread_fill_{}{}'.format(wave_itv1, wave_period1)] = wave_high_fill1_ / roll_lows1[-1]
    # df_res['short_wave_spread_fill_{}{}'.format(wave_itv1, wave_period1)] = wave_high_fill1_ / wave_low_fill1_
    # df_res['long_wave_spread_fill_{}{}'.format(wave_itv1, wave_period1)] = wave_high_fill1_ / wave_low_fill1_

    # itvnum = itv_to_number(wave_itv1)
    # itvnum2 = itvnum * 2

    # high_ = df_res['high_{}'.format(wave_itv1)].to_numpy()
    # low_ = df_res['low_{}'.format(wave_itv1)].to_numpy()

    # b1_close_ = df_res['close_{}'.format(wave_itv1)].shift(itvnum).to_numpy()
    # b1_open_ = df_res['open_{}'.format(wave_itv1)].shift(itvnum).to_numpy()
    # b1_high_ = df_res['high_{}'.format(wave_itv1)].shift(itvnum).to_numpy()
    # b1_low_ = df_res['low_{}'.format(wave_itv1)].shift(itvnum).to_numpy()
    # b2_high_ = df_res['high_{}'.format(wave_itv1)].shift(itvnum2).to_numpy()
    # b2_low_ = df_res['low_{}'.format(wave_itv1)].shift(itvnum2).to_numpy()

    # max_high_ = np.maximum(b1_high_, b2_high_)
    # min_low_ = np.minimum(b1_low_, b2_low_)

    # df_res['b1_close_{}'.format(wave_itv1)] = b1_close_
    # df_res['b1_open_{}'.format(wave_itv1)] = b1_open_
    # df_res['b1_high_{}'.format(wave_itv1)] = b1_high_
    # df_res['b1_low_{}'.format(wave_itv1)] = b1_low_
    # df_res['b2_high_{}'.format(wave_itv1)] = b2_high_
    # df_res['b2_low_{}'.format(wave_itv1)] = b2_low_
    # df_res['max_high_{}'.format(wave_itv1)] = max_high_
    # df_res['min_low_{}'.format(wave_itv1)] = min_low_



        #     #     f. wave_spread
        #     if config.tr_set.wave_spread1 != "None":
        #       short_wave_spread_fill = df_res['short_wave_spread_fill_{}{}'.format(wave_itv1, wave_period1)].to_numpy()
        #       long_wave_spread_fill = df_res['long_wave_spread_fill_{}{}'.format(wave_itv1, wave_period1)].to_numpy()

        #       short_open_res1 *= short_wave_spread_fill >= config.tr_set.wave_spread1
        #       long_open_res1 *= long_wave_spread_fill >= config.tr_set.wave_spread1

        #       if self.config.show_detail:
        #         sys_log.warning("wave_spread")
        #         sys_log.warning("np.sum(short_open_res1 == 1) : {}".format(np.sum(short_open_res1 == 1)))
        #         sys_log.warning("np.sum(long_open_res1 == 1) : {}".format(np.sum(long_open_res1 == 1)))

        #     #     g. wave_time_ratio
        #     if config.tr_set.wave_time_ratio1 != "None":
        #       short_wave_time_ratio = df_res['short_wave_time_ratio_{}{}'.format(wave_itv1, wave_period1)].to_numpy()
        #       long_wave_time_ratio = df_res['long_wave_time_ratio_{}{}'.format(wave_itv1, wave_period1)].to_numpy()

        #       short_open_res1 *= short_wave_time_ratio >= config.tr_set.wave_time_ratio1
        #       long_open_res1 *= long_wave_time_ratio >= config.tr_set.wave_time_ratio1

        #       if self.config.show_detail:
        #         sys_log.warning("wave_time_ratio")
        #         sys_log.warning("np.sum(short_open_res1 == 1) : {}".format(np.sum(short_open_res1 == 1)))
        #         sys_log.warning("np.sum(long_open_res1 == 1) : {}".format(np.sum(long_open_res1 == 1)))

        """
        5. bb points
        """
        #     a. wave prime_idx : 연속된 wave_point 에 대해서 prime_idx 만 허용함 (wave_bb 에서 파생됨)
        #     short_open_res1 *= df_res['wave_cu_idx_{}{}'.format(wave_itv1, wave_period1)] == df_res['wave_cu_prime_idx_fill_{}{}'.format(wave_itv1, wave_period1)]
        #     long_open_res1 *= df_res['wave_co_idx_{}{}'.format(wave_itv1, wave_period1)] == df_res['wave_co_prime_idx_fill_{}{}'.format(wave_itv1, wave_period1)]

        #     if self.config.show_detail:
        #         sys_log.warning("wave prime_idx")
        #         sys_log.warning("np.sum(short_open_res1 == 1) : {}".format(np.sum(short_open_res1 == 1)))
        #         sys_log.warning("np.sum(long_open_res1 == 1) : {}".format(np.sum(long_open_res1 == 1)))

        #     b. inner_triangle
        #     # short_open_res1 *= (roll_highs1[-2] > roll_highs1[-1]) & (roll_lows1[-2] < roll_lows1[-1])
        #     # long_open_res1 *= (roll_lows1[-2] < roll_lows1[-1]) & (roll_highs1[-2] > roll_highs1[-1])
        #     short_open_res1 *= (roll_highs1[-2] > roll_highs1[-1])
        #     long_open_res1 *= (roll_lows1[-2] < roll_lows1[-1])
        #     # short_open_res1 *= (roll_lows1[-2] < roll_lows1[-1])
        #     # long_open_res1 *= (roll_highs1[-2] > roll_highs1[-1])

        #     if self.config.show_detail:
        #         sys_log.warning("inner_triangle")
        #         sys_log.warning("np.sum(short_open_res1 == 1) : {}".format(np.sum(short_open_res1 == 1)))
        #         sys_log.warning("np.sum(long_open_res1 == 1) : {}".format(np.sum(long_open_res1 == 1)))

#### get_side_open

##### v2.0

In [ ]:

def get_side_open(self, open_num=1):

    """
    v2.0    
        rm utils_list, config_list

    last confirmed at, 20240421 0843
    """

    # init.
    self.side_open = None
    # self.side_open = 'BUY'
    
    # point
        # 여기 왜 latest_index 사용했는지 모르겠음.
            # --> bar_close point 를 고려해 mr_res 와의 index 분리
        # vecto. 하는 이상, point x ep_loc 의 index 는 sync. 되어야함 => complete_index 사용 (solved)
    
    if self.df_res['short_open{}_{}'.format(open_num, self.config.selection_id)].to_numpy()[self.config.trader_set.complete_index]:        
        self.sys_log.info("[ short ] true_point selection_id : {}".format(self.config.selection_id))

        # ban
        if self.config.pos_set.short_ban:
            self.sys_log.info("self.config.pos_set.short_ban : {}".format(self.config.pos_set.short_ban))
        else:
            # mr_res
            if open_num == 1:
                mr_res, zone_arr = self.entry_point_location(self, ep_loc_side=OrderSide.SELL)
            else:
                mr_res, zone_arr = self.entry_point2_location(self, ep_loc_side=OrderSide.SELL)

            # assign
            if mr_res[self.config.trader_set.complete_index]:
                self.side_open = OrderSide.SELL

    # point
    if self.df_res['long_open{}_{}'.format(open_num, self.config.selection_id)].to_numpy()[self.config.trader_set.complete_index]:    
    # if 1:          
        self.sys_log.info("[ long ] true_point selection_id : {}".format(self.config.selection_id))

        # ban
        if self.config.pos_set.long_ban:  # additional const. on trader
            self.sys_log.info("self.config.pos_set.long_ban : {}".format(self.config.pos_set.long_ban))
        else:
            # mr_res
            if open_num == 1:
                mr_res, zone_arr = self.entry_point_location(self, ep_loc_side=OrderSide.BUY)
            else:
                mr_res, zone_arr = self.entry_point2_location(self, ep_loc_side=OrderSide.BUY)

            # assign
            if mr_res[self.config.trader_set.complete_index]:
                self.side_open = OrderSide.BUY



##### entry_point_location

###### v4.0

In [ ]:

def entry_point_location(self, ep_loc_side=OrderSide.SELL):
    
    """
    v3.0
        vectorized calc.
            multi-stem 에 따라 dynamic vars.가 입력되기 때문에 class 내부 vars. 로 종속시키지 않음
            min & max variables 사용
    v4.0
        Class mode.
        
    last confirmed at, 20240421 1210
    """

    # 0. param init
    # self.len_df = len(self.df_res)
    mr_res = np.ones(self.len_df)
    zone_arr = np.full(self.len_df, 0)

    
    # 1. WRR32
    if self.table_condition_row.wrr_32_min_short != "None":
        if ep_loc_side == OrderSide.SELL:
            cu_wrr_32_ = self.df_res['cu_wrr_32_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1)].to_numpy()
            mr_res *= cu_wrr_32_ >= self.table_condition_row.wrr_32_min_short
            if self.config.show_detail:
                self.sys_log.info(
                    "cu_wrr_32_ >= self.table_condition_row.wrr_32_min_short : {:.5f} {:.5f} ({})".format(cu_wrr_32_[self.config.trader_set.complete_index],
                                                                                                       self.table_condition_row.wrr_32_min_short,
                                                                                                       mr_res[self.config.trader_set.complete_index]))
    if self.table_condition_row.wrr_32_min_long != "None":
        if ep_loc_side == OrderSide.BUY:
            co_wrr_32_ = self.df_res['co_wrr_32_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1)].to_numpy()
            mr_res *= co_wrr_32_ >= self.table_condition_row.wrr_32_min_long
            if self.config.show_detail:
                self.sys_log.info(
                    "co_wrr_32_ >= self.table_condition_row.wrr_32_min_long : {:.5f} {:.5f} ({})".format(co_wrr_32_[self.config.trader_set.complete_index],
                                                                                                      self.table_condition_row.wrr_32_min_long,
                                                                                                      mr_res[self.config.trader_set.complete_index]))
    if self.table_condition_row.wrr_32_max_short != "None":
        if ep_loc_side == OrderSide.SELL:
            cu_wrr_32_ = self.df_res['cu_wrr_32_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1)].to_numpy()
            mr_res *= cu_wrr_32_ <= self.table_condition_row.wrr_32_max_short
            if self.config.show_detail:
                self.sys_log.info(
                    "cu_wrr_32_ <= self.table_condition_row.wrr_32_max_short : {:.5f} {:.5f} ({})".format(cu_wrr_32_[self.config.trader_set.complete_index],
                                                                                                       self.table_condition_row.wrr_32_max_short,
                                                                                                       mr_res[self.config.trader_set.complete_index]))
    if self.table_condition_row.wrr_32_max_long != "None":
        if ep_loc_side == OrderSide.BUY:
            co_wrr_32_ = self.df_res['co_wrr_32_{}{}'.format(self.config.tr_set.wave_itv1, self.config.tr_set.wave_period1)].to_numpy()
            mr_res *= co_wrr_32_ <= self.table_condition_row.wrr_32_max_long
            if self.config.show_detail:
                self.sys_log.info(
                    "co_wrr_32_ <= self.table_condition_row.wrr_32_max_long : {:.5f} {:.5f} ({})".format(co_wrr_32_[self.config.trader_set.complete_index],
                                                                                                      self.table_condition_row.wrr_32_max_long,
                                                                                                      mr_res[self.config.trader_set.complete_index]))
                

    # 2. spread
    if self.table_condition_row.spread_min_short != "None":  # and not self.config.tr_set.check_hlm:
        if ep_loc_side == OrderSide.SELL:
            short_spread_ = self.df_res['short_spread_{}'.format(self.config.selection_id)].to_numpy()
            mr_res *= short_spread_ >= self.table_condition_row.spread_min_short
            if self.config.show_detail:
                self.sys_log.info(
                    "short_spread_ >= self.table_condition_row.spread_min_short : {:.5f} {:.5f} ({})".format(short_spread_[self.config.trader_set.complete_index], 
                                                                                                             self.table_condition_row.spread_min_short,
                                                                                                             mr_res[self.config.trader_set.complete_index]))

    if self.table_condition_row.spread_min_long != "None":  # and not self.config.tr_set.check_hlm:
        if ep_loc_side == OrderSide.BUY:
            long_spread_ = self.df_res['long_spread_{}'.format(self.config.selection_id)].to_numpy()
            mr_res *= long_spread_ >= self.table_condition_row.spread_min_long
            if self.config.show_detail:
                self.sys_log.info(
                    "long_spread_ >= self.table_condition_row.spread_min_long : {:.5f} {:.5f} ({})".format(long_spread_[self.config.trader_set.complete_index], 
                                                                                                           self.table_condition_row.spread_min_long, 
                                                                                                           mr_res[self.config.trader_set.complete_index]))

    if self.table_condition_row.spread_max_short != "None":  # and not self.config.tr_set.check_hlm:
        if ep_loc_side == OrderSide.SELL:
            short_spread_ = self.df_res['short_spread_{}'.format(self.config.selection_id)].to_numpy()
            mr_res *= short_spread_ <= self.table_condition_row.spread_max_short
            if self.config.show_detail:
                self.sys_log.info(
                    "short_spread_ <= self.table_condition_row.spread_max_short : {:.5f} {:.5f} ({})".format(short_spread_[self.config.trader_set.complete_index], 
                                                                                                             self.table_condition_row.spread_max_short,
                                                                                                             mr_res[self.config.trader_set.complete_index]))

    if self.table_condition_row.spread_max_long != "None":  # and not self.config.tr_set.check_hlm
        if ep_loc_side == OrderSide.BUY:
            long_spread_ = self.df_res['long_spread_{}'.format(self.config.selection_id)].to_numpy()
            mr_res *= long_spread_ <= self.table_condition_row.spread_max_long
            if self.config.show_detail:
                self.sys_log.info(
                    "long_spread_ <= self.table_condition_row.spread_max_long : {:.5f} {:.5f} ({})".format(long_spread_[self.config.trader_set.complete_index], 
                                                                                                           self.table_condition_row.spread_max_long,
                                                                                                           mr_res[self.config.trader_set.complete_index]))

    
    # 3.  zone
    if self.config.loc_set.zone1.use_zone:
        pass
        
    return mr_res, zone_arr  # mr_res 의 True idx 가 open signal

##### entry_point2_location

###### v4.0

In [ ]:

def entry_point2_location(self, ep_loc_side=OrderSide.SELL):
    
    """
    v3.0
        vectorized calc.
            multi-stem 에 따라 dynamic vars.가 입력되기 때문에 class 내부 vars. 로 종속시키지 않음
            min & max variables 사용
    v4.0
        Class mode.
        
    last confirmed at, 20240412 2125
    """
    
    # 0. param init
    # self.len_df = len(self.df_res)
    mr_res = np.ones(self.len_df)
    zone_arr = np.full(self.len_df, 0)

    
    # 1. WRR32
    if self.config.loc_set.point2.wrr_32_min != "None":
        if ep_loc_side == OrderSide.SELL:
            cu_wrr_32_ = self.df_res['cu_wrr_32_{}{}'.format(self.config.tr_set.wave_itv2, self.config.tr_set.wave_period2)].to_numpy()
            mr_res *= cu_wrr_32_ >= self.config.loc_set.point2.wrr_32_min
            if self.config.show_detail:
                self.sys_log.info(
                    "cu_wrr_32_ >= self.config.loc_set.point2.wrr_32_min : {:.5f} {:.5f} ({})".format(cu_wrr_32_[self.config.trader_set.complete_index],
                                                                                                 self.config.loc_set.point2.wrr_32_min,
                                                                                                 mr_res[self.config.trader_set.complete_index]))
        else:
            co_wrr_32_ = self.df_res['co_wrr_32_{}{}'.format(self.config.tr_set.wave_itv2, self.config.tr_set.wave_period2)].to_numpy()
            mr_res *= co_wrr_32_ >= self.config.loc_set.point2.wrr_32_min
            if self.config.show_detail:
                self.sys_log.info(
                    "co_wrr_32_ >= self.config.loc_set.point2.wrr_32_min : {:.5f} {:.5f} ({})".format(co_wrr_32_[self.config.trader_set.complete_index],
                                                                                                 self.config.loc_set.point2.wrr_32_min,
                                                                                                 mr_res[self.config.trader_set.complete_index]))
    if self.config.loc_set.point2.wrr_32_max != "None":
        if ep_loc_side == OrderSide.SELL:
            cu_wrr_32_ = self.df_res['cu_wrr_32_{}{}'.format(self.config.tr_set.wave_itv2, self.config.tr_set.wave_period2)].to_numpy()
            mr_res *= cu_wrr_32_ <= self.config.loc_set.point2.wrr_32_max
            if self.config.show_detail:
                self.sys_log.info(
                    "cu_wrr_32_ <= self.config.loc_set.point2.wrr_32_max : {:.5f} {:.5f} ({})".format(cu_wrr_32_[self.config.trader_set.complete_index],
                                                                                                 self.config.loc_set.point2.wrr_32_max,
                                                                                                 mr_res[self.config.trader_set.complete_index]))
        else:
            co_wrr_32_ = self.df_res['co_wrr_32_{}{}'.format(self.config.tr_set.wave_itv2, self.config.tr_set.wave_period2)].to_numpy()
            mr_res *= co_wrr_32_ <= self.config.loc_set.point2.wrr_32_max
            if self.config.show_detail:
                self.sys_log.info(
                    "co_wrr_32_ <= self.config.loc_set.point2.wrr_32_max : {:.5f} {:.5f} ({})".format(co_wrr_32_[self.config.trader_set.complete_index],
                                                                                                 self.config.loc_set.point2.wrr_32_max,
                                                                                                 mr_res[self.config.trader_set.complete_index]))

    # 2. zone.
    if self.config.loc_set.zone2.use_zone:
        
        cci_ = self.df_res['cci_30T20'].to_numpy()
        b1_cci_ = self.df_res['cci_30T20'].shift(30).to_numpy()
        # base_value = -100

        if ep_loc_side == OrderSide.SELL:
            # mr_res *= cci_ < 0
            mr_res *= cci_ < b1_cci_
            # mr_res *= (cci_ > -100) & (cci_ < -80)
            if self.config.show_detail:
                self.sys_log.info("cci_ < b1_cci_ : {:.5f} {:.5f} ({})".format(cci_[self.config.trader_set.complete_index], 
                                                                             b1_cci_[self.config.trader_set.complete_index], 
                                                                             mr_res[self.config.trader_set.complete_index]))
        else:
            # mr_res *= cci_ < 0
            mr_res *= cci_ > b1_cci_
            # mr_res *= (cci_ > 80) & (cci_ < 100)
            if self.config.show_detail:
                self.sys_log.info("cci_ > b1_cci_ : {:.5f} {:.5f} ({})".format(cci_[self.config.trader_set.complete_index], 
                                                                             b1_cci_[self.config.trader_set.complete_index], 
                                                                             mr_res[self.config.trader_set.complete_index]))

    return mr_res, zone_arr  # mr_res 의 True idx 가 open signal

### InitTableTrade

#### get_balance_info

In [ ]:

def get_balance_info(self, balance_min, mode="PROD"):

    self.balance_insufficient = 0

    # 1. asset_change - 첫 거래여부와 무관하게 진행
    if self.config.trader_set.asset_changed:
        self.balance_available = self.config.trader_set.initial_asset
        #        1. 이런식으로, cfg 의 값을 변경(dump)하는 경우를 조심해야함 - config_list[0] 에 입력해야한다는 의미
        self.config.trader_set.asset_changed = 0
        with open(self.path_config, 'w') as cfg:
            json.dump(self.config, cfg, indent=2)

        self.sys_log.info("asset_changed 1 --> 0")

    # 2. get availableBalance
    if not self.config.trader_set.backtrade:
        balance_available_max = self.get_balance_available(self)
        self.sys_log.info('balance_available_max : {:.2f}'.format(balance_available_max))

        if self.balance_over is not None:
            if self.balance_over <= balance_available_max * 0.998:  # 정상화 가능한 상태 (over_balanced 된 양을 허용할 수준의 max_balance)
                self.balance_available = self.balance_over  # mode="SUM" 의 경우에도 self.balance_over 에는 initial_asset 만큼만 담길 수 있기 때문에 구분하지 않음.
                self.balance_over = None
            else:  # 상태가 어찌되었든 일단은 지속해서 Bank 를 돌리려는 상황. (후조치 한다는 의미, 중단하는게 아니라)
                self.balance_available = balance_available_max * 0.998  # balance_available_max 를 넘지 않는 선
            self.sys_log.info('balance_available (temp) : {:.2f}'.format(self.balance_available))

        # state : requesting balance is larget than you have.
        else:
            if self.balance_available >= balance_available_max:
                self.balance_over = self.balance_available  # 복원 가능하도록 현재 상태를 balance_over 에 저장
                self.balance_available = balance_available_max * 0.998
            self.sys_log.info('balance_available : {:.2f}'.format(self.balance_available))

        # c. balance_min check
        if self.balance_available < balance_min:
            self.sys_log.warning('balance_available {:.2f} < balance_min {:.2f}\n'.format(self.balance_available, balance_min))
            self.balance_insufficient = 1

        # d. API term
        # time.sleep(self.config.trader_set.api_term)
        

#### get_balance_available

In [ ]:

def get_balance_available(self, asset_type='USDT'):
    
    # while 1:
    try:      
        server_time = self.time()['serverTime']
        response = self.balance(recvWindow=6000, timestamp=server_time)
    except Exception as e:
        msg = "error in get_balance() : {}".format(e)
        self.sys_log.error(msg)
        self.push_msg(self, msg)
    else:
        available_asset = float([res['availableBalance'] for res in response if res['asset'] == asset_type][0])
        return self.calc_with_precision(available_asset, 2)

#### get_precision

In [ ]:
def get_precision(self, ):
    
    try:
        response = self.exchange_info()
    except Exception as e:
        msg = "error in get_precision : {}".format(e)
        self.sys_log.error(msg)
        self.push_msg(self, msg)

        # time.sleep(self.config.trader_set.api_term)
    else:
        self.precision_price, self.precision_quantity = [[data['pricePrecision'], data['quantityPrecision']] 
                                                         for data in response['symbols'] if data['symbol'] == self.symbol][0]
        

#### get_price

##### v3.0

In [ ]:

def get_price(self, ):

    """
    v1 --> v3.0
        1. return price_stop_loss2 for exiting method.
    """

    self.sys_log.info('self.side_open : {}'.format(self.side_open))
    selection_id = self.config.selection_id

    tp_j = self.config.trader_set.complete_index
    ep_j = self.config.trader_set.complete_index
    # Todo, complete_index + 1 이 아니라, 해당 index 롤 self.df_res 이 아닌 -> self.df_res 에 입력하면 되는거아닌가
    #  -> use_point2 여부에 따라 다르게 작성할 필요가 없어짐 (solved)
    out_j = self.config.trader_set.complete_index

    # if selection_id in ['v5_2']:
    # if self.config.ep_set.point2.use_point2:
    #     # out_j = self.config.trader_set.complete_index + 1  # Todo, why + 1 ?
    #     out_j = self.config.trader_set.complete_index
    # else:
    #     out_j = self.config.trader_set.complete_index

    #       Todo        #
    #        b. --> self.df_res 사용시, dynamic (pre, whole) 을 허용하겠다는 의미
    #           i. ep_loc_point2 미사용시, self.df_res = self.df_res
    if self.side_open == OrderSide.BUY:
        self.price_entry = self.df_res['long_ep1_{}'.format(selection_id)].to_numpy()[ep_j]
        self.price_stop_loss = self.df_res['long_out_{}'.format(selection_id)].to_numpy()[out_j]
        price_stop_loss2 = self.df_res['long_out2_{}'.format(selection_id)].to_numpy()[out_j]
        self.price_take_profit = self.df_res['long_tp_{}'.format(selection_id)].to_numpy()[tp_j]
    else:
        self.price_entry = self.df_res['short_ep1_{}'.format(selection_id)].to_numpy()[ep_j]
        self.price_stop_loss = self.df_res['short_out_{}'.format(selection_id)].to_numpy()[out_j]
        price_stop_loss2 = self.df_res['short_out2_{}'.format(selection_id)].to_numpy()[out_j]
        self.price_take_profit = self.df_res['short_tp_{}'.format(selection_id)].to_numpy()[tp_j]

    return self.price_take_profit, self.price_entry, self.price_stop_loss, price_stop_loss2, self.side_open



##### v2.0

In [ ]:

def get_price(self, side_open, df_res, df_res):

    """
    v1 --> v2
        1. return self.leverage for ML mode.
    """

    self.sys_log.info('side_open : {}'.format(side_open))
    selection_id = self.config.selection_id

    tp_j = self.config.trader_set.complete_index
    ep_j = self.config.trader_set.complete_index
    # Todo, complete_index + 1 이 아니라, 해당 index 롤 df_res 이 아닌 -> df_res 에 입력하면 되는거아닌가
    #  -> use_point2 여부에 따라 다르게 작성할 필요가 없어짐 (solved)
    out_j = self.config.trader_set.complete_index

    # if selection_id in ['v5_2']:
    # if self.config.ep_set.point2.use_point2:
    #     # out_j = self.config.trader_set.complete_index + 1  # Todo, why + 1 ?
    #     out_j = self.config.trader_set.complete_index
    # else:
    #     out_j = self.config.trader_set.complete_index

    #       Todo        #
    #        b. --> df_res 사용시, dynamic (pre, whole) 을 허용하겠다는 의미
    #           i. ep_loc_point2 미사용시, df_res = df_res
    if side_open == OrderSide.BUY:
        self.price_entry = df_res['long_ep1_{}'.format(selection_id)].to_numpy()[ep_j]

        if self.config.pos_set.long_inversion:
            side_open = OrderSide.SELL  # side change (inversion)
            self.price_stop_loss = df_res['long_tp_{}'.format(selection_id)].to_numpy()[out_j]
            self.price_take_profit = df_res['long_out_{}'.format(selection_id)].to_numpy()[tp_j]
        else:
            self.price_stop_loss = df_res['long_out_{}'.format(selection_id)].to_numpy()[out_j]
            self.price_take_profit = df_res['long_tp_{}'.format(selection_id)].to_numpy()[tp_j]

        leverage_ = df_res['long_lvrg_needed_{}'.format(selection_id)].to_numpy()[ep_j]

    else:
        self.price_entry = df_res['short_ep1_{}'.format(selection_id)].to_numpy()[ep_j]

        if self.config.pos_set.short_inversion:
            side_open = OrderSide.BUY
            self.price_stop_loss = df_res['short_tp_{}'.format(selection_id)].to_numpy()[out_j]
            self.price_take_profit = df_res['short_out_{}'.format(selection_id)].to_numpy()[tp_j]
        else:
            self.price_stop_loss = df_res['short_out_{}'.format(selection_id)].to_numpy()[out_j]
            self.price_take_profit = df_res['short_tp_{}'.format(selection_id)].to_numpy()[tp_j]

        leverage_ = df_res['short_lvrg_needed_{}'.format(selection_id)].to_numpy()[ep_j]

    return self.price_take_profit, self.price_entry, self.price_stop_loss, leverage_, side_open


##### v1.0

In [ ]:

def get_price(self,):

    """
    v1.0
        df_res_open replaced with df_res.

    last confirmed at, 20240417 1853
    """
    
    if self.side_open == OrderSide.BUY:
        self.price_entry = self.df_res['long_ep1_{}'.format(self.config.selection_id)].to_numpy()[self.config.trader_set.complete_index]
        self.price_stop_loss = self.df_res['long_out_{}'.format(self.config.selection_id)].to_numpy()[self.config.trader_set.complete_index]
        self.price_take_profit = self.df_res['long_tp_{}'.format(self.config.selection_id)].to_numpy()[self.config.trader_set.complete_index]
        
    else:
        self.price_entry = self.df_res['short_ep1_{}'.format(self.config.selection_id)].to_numpy()[self.config.trader_set.complete_index]
        self.price_stop_loss = self.df_res['short_out_{}'.format(self.config.selection_id)].to_numpy()[self.config.trader_set.complete_index]
        self.price_take_profit = self.df_res['short_tp_{}'.format(self.config.selection_id)].to_numpy()[self.config.trader_set.complete_index]        



#### get_leverage_limit

In [ ]:

def get_leverage_limit(self,):
    
    server_time = self.time()['serverTime']
    response = self.leverage_brackets(symbol=self.symbol, recvWindow=6000, timestamp=server_time)
    self.leverage_limit = response[0]['brackets'][0]['initialLeverage']
    


#### get_leverage

##### v3.0

In [ ]:
def get_leverage(self, ):

    """
    v2.0 --> v2_1
        math.ceil self.leverage.
    v2_1 --> v2_2
        target_pct pointed at self.price_take_profit. (origin --> self.price_stop_loss)
        adj. get_pr_v8
    v3.0
        Class mode
            add self,
                temporarily not removing origin parameter.
                
            replace tp_ with self.price_stop_loss.

    last confirmed at, 20240417 1901
    """
    
    # init.
    self.leverage = self.config.lvrg_set.leverage
    loss = None

    if not pd.isnull(self.price_stop_loss):
        if not self.config.lvrg_set.static_lvrg_short:
            if self.side_open == OrderSide.SELL:
                loss = self.price_entry / self.price_stop_loss

        if not self.config.lvrg_set.static_lvrg_long:
            if self.side_open == OrderSide.BUY:
                loss = self.price_stop_loss / self.price_entry

        if loss is not None:
            self.leverage = self.config.lvrg_set.target_pct / abs(loss * (1 - (self.config.trader_set.fee_limit + self.config.trader_set.fee_market)) - 1)

    # 1. self.leverage rejection.
    if self.leverage < 1 and self.config.lvrg_set.lvrg_rejection:
        return None, None

    if not self.config.lvrg_set.allow_float:
        self.leverage = math.ceil(self.leverage)
        # self.leverage = int(self.leverage)

    self.leverage = min(self.leverage_limit, max(self.leverage, 1))
    return self.leverage


#### set_leverage

In [ ]:
def set_leverage(self,):

    try:
        server_time = self.time()['serverTime']
        self.change_leverage(symbol=self.symbol, leverage=self.leverage, recvWindow=6000, timestamp=server_time)
    except Exception as e:
        msg = "error in change_initial_leverage : {}".format(e)
        self.sys_log.error(msg)
        self.push_msg(self, msg)
    else:
        self.sys_log.info('self.leverage changed to {}'.format(self.leverage))


#### set_position_mode

In [ ]:
def set_position_mode(self, ):

    """
    v1.0
        pass error -4059 -4046

    last confirmed at, 20240504 1948.
    """

    try:
        server_time = self.time()['serverTime']
        self.change_position_mode(dualSidePosition="true", recvWindow=2000, timestamp=server_time)
    except Exception as e:
        if '-4059' in str(e): # 'No need to change position side.'
            return
        msg = "error in set_position_mode : {}".format(e)
        self.sys_log.error(msg)
        self.push_msg(self, msg)
    else:
        self.sys_log.info("dualSidePosition is true.")

#### set_margin_type

In [ ]:
def set_margin_type(self, margin_type='CROSSED'):

    """
    v1.0
        pass error -4046

    last confirmed at, 20240504 1948.
    """

    # margin type => "cross or isolated"
    try:
        server_time = self.time()['serverTime']
        if margin_type == 'CROSSED':
            self.change_margin_type(symbol=self.symbol, marginType=FuturesMarginType.CROSSED, recvWindow=6000, timestamp=server_time)
        else:
            self.change_margin_type(symbol=self.symbol, marginType=FuturesMarginType.ISOLATED, recvWindow=6000, timestamp=server_time)
            
    except Exception as e:
        if '-4046' in str(e): # 'No need to change margin type.'
            return
        msg = "error in set_margin_type : {}".format(e)
        self.sys_log.error(msg)
        self.push_msg(self, msg)
    else:
        self.sys_log.info("margin type is {} now.".format(margin_type))

### LoopTableTrade

In [ ]:
bank.balance_over = None
bank.balance_available = 10
bank.get_balance_info(bank, 10)
bank.balance_insufficient

In [ ]:
# bank.side_open = 'BUY'
bank.get_price(bank)
print(bank.price_take_profit)
print(bank.price_entry)
print(bank.price_stop_loss)

In [ ]:
bank.get_leverage(bank)

In [ ]:
bank.fee = 0.0006
bank.get_price_liquidation(bank)
print(bank.price_liquidation)

In [ ]:
bank.get_price_expiration(bank)
print(bank.price_expiration)
print(bank.config.tr_set.expire_k1)
print(bank.price_take_profit)

In [ ]:
# bank.set_leverage(bank)
print(bank.leverage)
print(bank.code)

In [ ]:
# bank.code = 'THETAUSDT'
# order_type = 'LIMIT'
# side_order = bank.side_open # 'BUY'
# side_position = 'LONG'
# price = 1.9
# quantity = 10

# bank.order_limit(bank, order_type, side_order, side_position, price, quantity, order_data=None)

In [ ]:
# bank.table_order = bank.table_order.iloc[2]
# bank.table_order = pd.DataFrame(bank.table_order.)
bank.table_trade #  =  bank.table_trade.iloc[2]

In [ ]:
# bank.get_price_realtime(bank)
bank.price_realtime = 2.2
bank.check_stop_loss(bank,)
bank.check_stop_loss_by_signal(bank,)

print(bank.order_market_on)
print(bank.side_open)

In [ ]:
bank.check_stop_loss_onbarclose(bank,)

In [ ]:
bank.order_cancel(bank,)
# bank.get_order_info(bank)
# bank.order_res

In [ ]:
# bank.order_res
bank.order_info

In [ ]:
bank.side_close = 'SELL'
order_res_list = [bank.order_res]
bank.order_market(bank, )

In [ ]:
bank.code

In [ ]:
bank.table_log

In [ ]:
bank.get_profit_ideal# (bank, [], [], [])

In [ ]:
bank.get_income_info(bank)

#### get_order_info

In [ ]:
def get_order_info(self, ):

    """
    v1.0 
        order_res format.
            {'orderId': 12877344699,
              'symbol': 'THETAUSDT',
              'status': 'NEW',
              'clientOrderId': 't1eOIqWG2m72oxaMKLZHKE',
              'price': '1.9500',
              'avgPrice': '0.00',
              'origQty': '10.0',
              'executedQty': '0.0',
              'cumQty': '0.0',
              'cumQuote': '0.00000',
              'timeInForce': 'GTC',
              'type': 'LIMIT',
              'reduceOnly': False,
              'closePosition': False,
              'side': 'BUY',
              'positionSide': 'LONG',
              'stopPrice': '0.0000',
              'workingType': 'CONTRACT_PRICE',
              'priceProtect': False,
              'origType': 'LIMIT',
              'priceMatch': 'NONE',
              'selfTradePreventionMode': 'NONE',
              'goodTillDate': 0,
              'updateTime': 1713354764631},

    last confirmed at, 20240417 2100.
    """
    
    try:
        server_time = self.time()['serverTime']
        self.order_info = self.query_order(symbol=self.symbol, orderId=self.orderId, recvWindow=2000, timestamp=server_time)
        # return self.order_info
    except Exception as e:
        msg = "error in get_order_info : {}".format(e)
        self.sys_log.error(msg)
        self.push_msg(self, msg)
    else:
        self.sys_log.info("order_info : {}".format(self.order_info))

#### get_price_realtime

##### v3.0

In [ ]:
def get_price_realtime(self, ):
    
    """
    v3.0
        get price_market by self.price_market
        add non-error solution.
    
    last confirmed at, 20240504 2353.
    """
    
    while 1:
        try:
            self.price_realtime =  self.price_market[self.symbol]
        except Exception as e:
            
            if self.symbol not in self.price_market.keys():
                self.websocket_client.agg_trade(symbol=self.symbol, id=1, callback=self.agg_trade_message_handler)
                msg = "error in get_price_realtime : {} added to websocket_client.agg_trade".format(self.symbol)
            else:                
                msg = "error in get_price_realtime : {}".format(e)
            self.sys_log.error(msg)
            self.push_msg(self, msg)
            time.sleep(self.config.trader_set.api_term)
            
        else:
            return


##### v2.0

In [ ]:
def get_price_realtime(self, ):
    
    """
    v2.0
        1. agg_trades is faster than ticker_price
            a. subscribe method 가 24시간 이상 유지되지 않는다고해, restful api method 사용함.
    """
    
    response = self.agg_trades(self.symbol)
    return float(response[-1]['p'])


#### get_price_expiration

In [ ]:

def get_price_expiration(self, ):

    """
    v2.0 --> v3.0
        self.expired variable added.
    v4
        Class mode.
            rename function to get_price_expiration.

    last confirmed at, 20240428 1904.
    """

    # if self.config.tr_set.expire_tick != "None":
    #     if time.time() - datetime.timestamp(self.df_res.index[self.config.trader_set.latest_index]) >= self.config.tr_set.expire_tick * 60:
    #         self.expired = 1    
        
    if self.side_open == OrderSide.SELL:
        short_tp_ = self.df_res['short_tp_{}'.format(self.config.selection_id)].to_numpy()
        short_tp_gap_ = self.df_res['short_tp_gap_{}'.format(self.config.selection_id)].to_numpy()        
        self.price_expiration = short_tp_[self.config.trader_set.complete_index] + short_tp_gap_[self.config.trader_set.complete_index] * self.config.tr_set.expire_k1    
    else:
        long_tp_ = self.df_res['long_tp_{}'.format(self.config.selection_id)].to_numpy()  # iloc 5.34 ms, to_numpy() 3.94 ms
        long_tp_gap_ = self.df_res['long_tp_gap_{}'.format(self.config.selection_id)].to_numpy()        
        self.price_expiration = long_tp_[self.config.trader_set.complete_index] - long_tp_gap_[self.config.trader_set.complete_index] * self.config.tr_set.expire_k1
            

#### get_price_liquidation

In [ ]:
def get_price_liquidation(self, ):
    
    if self.side_open == OrderSide.SELL:
        self.price_liquidation = self.price_entry / (1 + self.config.trader_set.fee_limit + self.config.trader_set.fee_market - 1 / self.leverage)
    else:
        self.price_liquidation = self.price_entry * (1 + self.config.trader_set.fee_limit + self.config.trader_set.fee_market - 1 / self.leverage)

    # return price_liquidation

In [ ]:
bank.get_partial_price_and_quantity = get_partial_price_and_quantity
bank.order_limit_multiple = order_limit_multiple

In [ ]:
bank.code = 'THETAUSDT'
bank.quantity_open_executed = 2
print(bank.get_precision(bank))
bank.get_partial_price_and_quantity(bank)

In [ ]:
bank.order_res_list = [None]

bank.side_position = 'LONG'
bank.order_limit_multiple(bank, order_res_list)

In [ ]:
bank.get_order_info = get_order_info
bank.check_quantity_open_exec = check_quantity_open_exec 
# bank.check_breakout_qty = check_breakout_qty
bank.check_order_expiration = check_order_expiration
bank.check_order_expiration_onbarclose = check_order_expiration_onbarclose

In [ ]:
# bank.order_res['orderId']
bank.order_info['orderId']
bank.orderId = bank.order_info['orderId']# ['orderId']

In [ ]:
bank.get_order_info(bank) # realtime.
# bank.order_res

In [ ]:
bank.check_quantity_open_exec(bank)

In [ ]:
bank.websocket_client.agg_trade(symbol=bank.code, id=1, callback=bank.agg_trade_message_handler)

In [ ]:
bank.get_price_realtime = get_price_realtime
bank.symbol = 'THETAUSDT'
bank.get_price_realtime(bank)

In [ ]:
bank.price_realtime = 1.98
bank.check_order_expiration(bank)
bank.expired

In [ ]:
bank.check_order_expiration_onbarclose(bank)

In [ ]:
bank.order_info

In [ ]:
bank.table_trade

In [ ]:
index_target = bank.table_trade[bank.table_trade.orderId == 12886367313].index
bank.table_trade.iloc[index_target].apply(bank.order_info)
# len(bank.table_trade[bank.table_trade.orderId == 12886367308])

#### check_order_expiration_onbarclose

In [ ]:

def check_order_expiration_onbarclose(self,):  # for point2

    """
    v4.0
        Class mode
    
    last confirmed at, 20240418 2459.
    """

    selection_id = self.config.selection_id

    if self.config.tr_set.expire_tick != "None":
        if datetime.timestamp(self.df_res.index[-1]) - datetime.timestamp(self.df_res.index[self.config.trader_set.latest_index]) \
                >= self.config.tr_set.expire_tick * 60:
            self.expired = 1

    if self.config.tr_set.expire_k1 != "None":  # Todo - onbarclose 에서는, self.df_res 으로 open_index 의 self.price_take_profit 정보를 사용
        if self.side_open == OrderSide.SELL:
            low = self.df_res['low'].to_numpy()
            short_tp_ = self.df_res['short_tp_{}'.format(selection_id)].to_numpy()  # id 에 따라 dynamic 변수라 이곳에서 numpy 화 진행
            short_tp_gap_ = self.df_res['short_tp_gap_{}'.format(selection_id)].to_numpy()
            if low[self.config.trader_set.complete_index] <= short_tp_[self.config.trader_set.complete_index] + short_tp_gap_[self.config.trader_set.complete_index] * self.config.tr_set.expire_k1:
                self.expired = 1
        else:
            high = self.df_res['high'].to_numpy()
            long_tp_ = self.df_res['long_tp_{}'.format(selection_id)].to_numpy()  # iloc 이 빠를까, to_numpy() 가 빠를까  # 3.94 ms --> 5.34 ms (iloc)
            long_tp_gap_ = self.df_res['long_tp_gap_{}'.format(selection_id)].to_numpy()
            if high[self.config.trader_set.complete_index] >= long_tp_[self.config.trader_set.complete_index] - long_tp_gap_[self.config.trader_set.complete_index] * self.config.tr_set.expire_k1:
                self.expired = 1

    # if self.expired:
    #     self.sys_log.warning("cancel order_open by ei_k\n")

    # return self.expired


#### check_stop_loss

##### v4.0

In [ ]:

def check_stop_loss(self, ):

    """
    v2 --> v3.0
        1. get self.price_realtime from outer_scope
        2. inversion 고려 아직임. (inversion 사용하게 되면 고려할 것.) (Todo)
        3. add self.log_out
        4. add non_out.
    v4.0
        init self.order_market_on = None
        
    last confirmed at, 20240428 2135.
    """

    # init.
    # self.order_market_on = None
    self.log_out = None

    # check liquidation
    if self.side_open == OrderSide.SELL:
        const_str = "self.price_realtime >= self.price_liquidation"
        if eval(const_str):
            self.order_market_on = True
            self.log_out = self.price_liquidation
            self.sys_log.info("{} : {} {}".format(const_str, self.price_realtime, self.price_liquidation))
    else:
        const_str = "self.price_realtime <= self.price_liquidation"
        if eval(const_str):
            self.order_market_on = True
            self.log_out = self.price_liquidation
            self.sys_log.info("{} : {} {}".format(const_str, self.price_realtime, self.price_liquidation))

    if not self.config.out_set.non_out:

        # hl_out
        if self.config.out_set.hl_out:
            if self.side_open == OrderSide.SELL:
                const_str = "self.price_realtime >= self.price_stop_loss"
                if eval(const_str):
                    self.order_market_on = True
                    self.log_out = self.price_stop_loss
                    self.sys_log.info("{} : {} {}".format(const_str, self.price_realtime, self.price_stop_loss))
            else:
                const_str = "self.price_realtime <= self.price_stop_loss"
                if eval(const_str):
                    self.order_market_on = True
                    self.log_out = self.price_stop_loss
                    self.sys_log.info("{} : {} {}".format(const_str, self.price_realtime, self.price_stop_loss))

        # close_out
        else:
            close = self.df_res['close'].to_numpy()

            if self.side_open == OrderSide.SELL:
                if close[self.config.trader_set.complete_index] >= self.price_stop_loss:
                    self.order_market_on = True
                    self.log_out = close[self.config.trader_set.complete_index]
                    self.sys_log.info("{} : {} {}".format("close >= self.price_stop_loss", self.log_out, self.price_stop_loss))
            else:
                if close[self.config.trader_set.complete_index] <= self.price_stop_loss:
                    self.order_market_on = True
                    self.log_out = close[self.config.trader_set.complete_index]
                    self.sys_log.info("{} : {} {}".format("close <= self.price_stop_loss", self.log_out, self.price_stop_loss))
                    


##### v3.0

In [ ]:

def check_stop_loss(self, ):

    """
    v2 --> v3.0
        1. get self.price_realtime from outer_scope
        2. inversion 고려 아직임. (inversion 사용하게 되면 고려할 것.) (Todo)
        3. add self.log_out
        4. add non_out.
        
    last confirmed at, 20240418 1022.
    """

    self.log_out = None

    # ------ 1. liquidation default check ------ #
    if self.side_open == OrderSide.SELL:
        const_str = "self.price_realtime >= self.price_liquidation"
        if eval(const_str):
            self.order_market_on = True
            self.log_out = self.price_liquidation
            self.sys_log.info("{} : {} {}".format(const_str, self.price_realtime, self.price_liquidation))
    else:
        const_str = "self.price_realtime <= self.price_liquidation"
        if eval(const_str):
            self.order_market_on = True
            self.log_out = self.price_liquidation
            self.sys_log.info("{} : {} {}".format(const_str, self.price_realtime, self.price_liquidation))

    if not self.config.out_set.non_out:

        # ------ 2. hl_out ------ #
        if self.config.out_set.hl_out:
            if self.side_open == OrderSide.SELL:
                const_str = "self.price_realtime >= self.price_stop_loss"
                if eval(const_str):
                    self.order_market_on = True
                    self.log_out = self.price_stop_loss
                    self.sys_log.info("{} : {} {}".format(const_str, self.price_realtime, self.price_stop_loss))
            else:
                const_str = "self.price_realtime <= self.price_stop_loss"
                if eval(const_str):
                    self.order_market_on = True
                    self.log_out = self.price_stop_loss
                    self.sys_log.info("{} : {} {}".format(const_str, self.price_realtime, self.price_stop_loss))

        # ------ 3. close_out ------ #
        else:
            close = self.df_res['close'].to_numpy()

            if self.side_open == OrderSide.SELL:
                j = self.config.trader_set.complete_index
                if close[j] >= self.price_stop_loss:
                    self.order_market_on = True
                    self.log_out = close[j]
                    self.sys_log.info("{} : {} {}".format("close >= self.price_stop_loss", self.log_out, self.price_stop_loss))
            else:
                j = self.config.trader_set.complete_index
                if close[j] <= self.price_stop_loss:
                    self.order_market_on = True
                    self.log_out = close[j]
                    self.sys_log.info("{} : {} {}".format("close <= self.price_stop_loss", self.log_out, self.price_stop_loss))

    # return self.order_market_on, self.log_out


#### check_stop_loss_onbarclose

In [ ]:

def check_stop_loss_onbarclose(self,):

    """
    v1 --> v2.0
        1. add liquidation platform.
        2. add self.log_out
        3. add non_out
    """

    self.log_out = None

    high = self.df_res['high'].to_numpy()[self.config.trader_set.complete_index]
    low = self.df_res['low'].to_numpy()[self.config.trader_set.complete_index]
    close = self.df_res['close'].to_numpy()[self.config.trader_set.complete_index]

    # ------ 1. liquidation default check ------ #
    if self.side_open == OrderSide.SELL:
        const_str = "high >= self.price_liquidation"
        if eval(const_str):
            self.order_market_on = True
            self.log_out = self.price_liquidation
            self.sys_log.info("{} : {} {}".format(const_str, high, self.price_liquidation))
    else:
        const_str = "low <= self.price_liquidation"
        if eval(const_str):
            self.order_market_on = True
            self.log_out = self.price_liquidation
            self.sys_log.info("{} : {} {}".format(const_str, low, self.price_liquidation))

    if not self.config.out_set.non_out:

        # ------ 2. hl_out ------ #
        if self.config.out_set.hl_out:
            if self.side_open == OrderSide.SELL:
                const_str = "high >= self.price_stop_loss"
                if eval(const_str):
                    self.order_market_on = True
                    self.log_out = self.price_stop_loss
                    self.sys_log.info("{} : {} {}".format(const_str, high, self.price_stop_loss))
            else:
                const_str = "low <= self.price_stop_loss"
                if eval(const_str):
                    self.order_market_on = True
                    self.log_out = self.price_stop_loss
                    self.sys_log.info("{} : {} {}".format(const_str, low, self.price_stop_loss))

        # ------ 3. close_out ------ #
        else:
            if self.side_open == OrderSide.SELL:
                if close >= self.price_stop_loss:
                    self.order_market_on = True
                    self.log_out = close
                    self.sys_log.info("{} : {} {}".format("close >= self.price_stop_loss", self.log_out, self.price_stop_loss))
            else:
                if close <= self.price_stop_loss:
                    self.order_market_on = True
                    self.log_out = close
                    self.sys_log.info("{} : {} {}".format("close <= self.price_stop_loss", self.log_out, self.price_stop_loss))

    # return self.order_market_on, self.log_out



#### check_stop_loss_by_signal

In [ ]:

def check_stop_loss_by_signal(self, ):

    """
    v3 --> v4.0
        1. add fisher_exit.
        2. add self.np_timeidx
    """

    self.log_out = None

    close = self.df_res['close'].to_numpy()[self.config.trader_set.complete_index]

    # 1. timestamp
    # if self.config.out_set.tf_exit != "None":
    #     if self.np_timeidx[i] % self.config.out_set.tf_exit == self.config.out_set.tf_exit - 1 and i != open_i:
    #         self.order_market_on = True

    # 2. fisher
    # if self.config.out_set.fisher_exit:

    #     itv_num = itv_to_number(self.table_condition_row.tf_entry)

    #     if self.np_timeidx[self.config.trader_set.complete_index] % itv_num == itv_num - 1:

    #         fisher_ = self.df_res['fisher_{}30'.format(self.table_condition_row.tf_entry)].to_numpy()
    #         fisher_band = self.config.out_set.fisher_band
    #         fisher_band2 = self.config.out_set.fisher_band2

    #         if self.side_open == OrderSide.SELL:
    #             if (fisher_[self.config.trader_set.complete_index - itv_num] > -fisher_band) & (fisher_[self.config.trader_set.complete_index] <= -fisher_band):
    #                 self.order_market_on = True
    #             elif (fisher_[self.config.trader_set.complete_index - itv_num] < fisher_band2) & (fisher_[self.config.trader_set.complete_index] >= fisher_band2):
    #                 self.order_market_on = True
    #         else:
    #             if (fisher_[self.config.trader_set.complete_index - itv_num] < fisher_band) & (fisher_[self.config.trader_set.complete_index] >= fisher_band):
    #                 self.order_market_on = True
    #             elif (fisher_[self.config.trader_set.complete_index - itv_num] > fisher_band2) & (fisher_[self.config.trader_set.complete_index] <= fisher_band2):
    #                 self.order_market_on = True

    # 3. rsi_exit
    if self.config.out_set.rsi_exit:
        rsi_ = self.df_res['rsi_%s' % self.config.loc_set.point.exp_itv].to_numpy()
        osc_band = self.config.loc_set.point.osc_band

        if self.side_open == OrderSide.SELL:
            if (rsi_[self.config.trader_set.complete_index - 1] >= 50 - osc_band) & (rsi_[self.config.trader_set.complete_index] < 50 - osc_band):
                self.order_market_on = True
        else:
            if (rsi_[self.config.trader_set.complete_index - 1] <= 50 + osc_band) & (rsi_[self.config.trader_set.complete_index] > 50 + osc_band):
                self.order_market_on = True

    # 4. cci_exit
    #           a. deprecated.
    # if self.config.out_set.cci_exit:
    #     wave_itv1 = self.config.tr_set.wave_itv1
    #     wave_period1 = self.config.tr_set.wave_period1
    #
    #     if self.side_open == OrderSide.SELL:
    #         wave_co_ = self.df_res['wave_co_{}{}'.format(wave_itv1, wave_period1)].to_numpy()[self.config.trader_set.complete_index]
    #         if wave_co_:
    #             self.order_market_on = True
    #     else:
    #         wave_cu_ = self.df_res['wave_cu_{}{}'.format(wave_itv1, wave_period1)].to_numpy()[self.config.trader_set.complete_index]
    #         if wave_cu_:
    #             self.order_market_on = True

    # if self.order_market_on:
    #     self.log_out = close
    #     self.sys_log.info("signal self.price_stop_loss : {}".format(self.log_out))

    # return self.order_market_on, self.log_out


#### order_limit

In [ ]:

def order_limit(self, order_type, side_order, side_position, price, quantity):

    """
    v2.0
        1. self.symbol 도입.
        2. rename to retry_count & -1111 error update.
    v3.0
        Class mode
            remove order_data
            
    last confirmed at, 20240428 2042.
    """
    
    # init.
    retry_count = 0    
    self.error_code = 0
    self.order_res = None

    quantity_orig = quantity
    

    while 1:
        try:
            # 0. new_order (limit & market)
            # Todo, quantity 에 float 대신 str 을 입력하는게 정확한 것으로 앎.
            #   1. precision 보호 차원인가.
            server_time = self.time()['serverTime']
            if order_type == OrderType.MARKET:
                # a. market order
                self.order_res = self.new_order(symbol=self.symbol,
                                                side=side_order,
                                                positionSide=side_position,
                                                type=OrderType.MARKET,
                                                quantity=str(quantity),
                                                timestamp=server_time)
            else:
                # b. limit order
                self.order_res = self.new_order(timeInForce=TimeInForce.GTC,
                                                symbol=self.symbol,
                                                side=side_order,
                                                positionSide=side_position,
                                                type=OrderType.LIMIT,
                                                quantity=str(quantity),
                                                price=str(price),
                                                timestamp=server_time)
        except Exception as e:
            msg = "error in order_limit : {}".format(e)
            self.sys_log.error(msg)
            self.push_msg(self, msg)

            # 1. order_limit() 에서 해결할 수 없는 error 일 경우, return
            #       a. -4003 : quantity less than zero
            if "-4003" in str(e):
                self.error_code = -4003

            retry_count += 1
            # 2. order_limit() 에서 해결할 error 일 경우, continue
            #       y. -1111 : Precision is over the maximum defined for this asset = quantity precision error
            if '-1111' in str(e):

                try:
                    #   _, precision_quantity = self.get_precision()
                    precision_quantity = retry_count

                    # close_remain_quantity = get_remaining_quantity(self.symbol)
                    #   error 발생한 경우 정상 체결된 quantity 는 존재하지 않는다고 가정 - close_remain_quantity 에는 변함이 없을거라 봄
                    quantity = self.calc_with_precision(quantity_orig, precision_quantity, def_type='floor')
                    self.sys_log.info("modified quantity & precision : {} {}".format(quantity, precision_quantity))

                except Exception as e:
                    msg = "error in get modified precision_quantity : {}".format(e)
                    self.sys_log.error(msg)
                    self.push_msg(self, msg)

                if retry_count >= 10:
                    retry_count = 0

                # self.error_code = -1111
                # return None, self.balance_over, self.error_code
                time.sleep(self.config.trader_set.order_term)
                continue

            #       a. -4014 : Price not increased by tick size
            if '-4014' in str(e):
                try:
                    realtime_price = self.get_price_realtime(self)
                    self.precision_price = self.get_precision_by_price(realtime_price)
                    realtime_price = self.calc_with_precision(realtime_price, self.precision_price)
                    self.sys_log.info("modified realtime_price & precision : {}, {}".format(realtime_price, self.precision_price))

                except Exception as e:
                    msg = "error in get_price_realtime (order_open phase): {}".format(e)
                    self.sys_log.error(msg)
                    self.push_msg(self, msg)

                if retry_count >= 10:
                    self.error_code = 'unknown'

                time.sleep(self.config.trader_set.order_term)
                continue

            #       b. -2019 : Margin is insufficient
            # Todo - for tp_exec_qty miscalc., self.overbalance 수정되었으면, return 으로 받아야할 것 => 무슨 소리야..?
            if '-2019' in str(e):
                # if order_data is not None:  # None 인 경우, self.leverage 가 정의되지 않음.
                # i. 예기치 못한 오류로 인해 over balance 상태가 되었을때의 조치
                try:
                    # 1. get available quantity
                    balance_available = self.get_balance_available() * 0.9

                    _, precision_quantity = self.get_precision(self)
                    quantity = balance_available / price * self.leverage
                    quantity = self.calc_with_precision(quantity, precision_quantity, def_type='floor')
                    self.sys_log.info('balance_available (temp) : {}'.format(balance_available))
                    self.sys_log.info('quantity : {}'.format(quantity))

                except Exception as e:
                    msg = "error in get_availableBalance (-2019 phase) : {}".format(e)
                    self.sys_log.error(msg)
                    self.push_msg(self, msg)

                # ii. balance 가 근본적으로 부족한 경우, 주문을 허용하지 않는다.
                retry_count += 1
                if retry_count > 5:
                    msg = "retry_count over. : {}".format(retry_count)
                    self.sys_log.error(msg)
                    self.push_msg(self, msg)
                    self.error_code = -2019
                else:
                    time.sleep(self.config.trader_set.order_term)
                    continue
            else:
                msg = "unknown error : {}".format(e)
                self.sys_log.error(msg)
                self.push_msg(self, msg)
                self.error_code = 'unknown'

        else:
            # 3. 정상 order 의 self.error_code = 0.
            # self.order_res = order_res
            self.sys_log.info("order_limit succeed. order_res : {}".format(self.order_res))
            return



#### order_cancel

In [ ]:

def order_cancel(self, ):

    """
    v1.0
        add table_order logic.

    last confirmed at, 20240423 2413.
    """
    
    try:
        server_time = self.time()['serverTime']
        _ = self.cancel_order(symbol=self.symbol, orderId=self.orderId, timestamp=server_time)
    except Exception as e:        
        msg = "error in order_cancel : {}".format(e)
        self.sys_log.error(msg)
        self.push_msg(self, msg)
    else:
        self.sys_log.info("{} {} canceled.".format(self.symbol, self.orderId))

#### order_market

##### v3.0

In [ ]:
def order_market(self, ):

    """
    v2.0
        update retry_count        
    v3.0
        remove order_res_list.
        
    last confirmed at, 20240428 2104.
    """

    while 1:

        # get unexecutedQty (unexecutedQty)
        self.order_cancel(self,)
        
        self.get_order_info(self,)
        unexecutedQty = abs(float(self.order_info['origQty'])) - abs(float(self.order_info['executedQty']))

            # get price, volume updated precision
        self.get_precision(self)
        self.sys_log.info('precision_quantity : {}'.format(self.precision_quantity))
        
        self.sys_log.info('unexecutedQty : {}'.format(unexecutedQty))
        unexecutedQty = self.calc_with_precision(unexecutedQty, self.precision_quantity)
        self.sys_log.info('unexecutedQty (adj. precision) : {}'.format(unexecutedQty))


        # order_market
        retry_count = 0
        error_code = 0
        unexecutedQty_org = unexecutedQty
        while 1:
            try:
                server_time = self.time()['serverTime']
                self.order_res = self.new_order(symbol=self.symbol,
                                                side=self.side_close,
                                                positionSide=self.side_position,
                                                type=OrderType.MARKET,
                                                quantity=str(unexecutedQty),
                                                timestamp=server_time)
            except Exception as e:

                msg = "error in order_market : {}".format(e)
                self.sys_log.error(msg)
                self.push_msg(self, msg)

                # -2022 ReduceOnly Order is rejected
                if '-2022' in str(e):
                    error_code = '-2022'
                    break

                # -4003 quantity less than zero
                if '-4003' in str(e):
                    error_code = '-4003'
                    break
                    
                # -1111 : Precision is over the maximum defined for this asset
                    # = quantity precision error
                    # 상당히 오래전에 만든 대응법인데, 이게 맞는 걸까.. (Todo)
                if '-1111' in str(e):
                    try:
                        # _, precision_quantity = self.get_precision()
                        precision_quantity = retry_count

                        # unexecutedQty = get_remaining_quantity(self.symbol)
                        #   error 발생한 경우 정상 체결된 quantity 는 존재하지 않는다고 가정 - unexecutedQty 에는 변함이 없을거라 봄
                        unexecutedQty = self.calc_with_precision(unexecutedQty_org, precision_quantity, def_type='floor')
                        self.sys_log.info("modified quantity & precision : {} {}".format(unexecutedQty, precision_quantity))

                    except Exception as e:
                        msg = "error in get modified precision_quantity : {}".format(e)
                        self.sys_log.error(msg)
                        self.push_msg(self, msg)

                    retry_count += 1
                    if retry_count >= 10:
                        retry_count = 0

                # self.config.out_set.out_type = OrderType.MARKET  # just 재시도
                time.sleep(self.config.trader_set.order_term)
                continue

            else:
                self.sys_log.info("order_market succeed. : {}".format(self.order_res))
                break

        # 4. term for quantity consumption.
        time.sleep(1)

        # 5.  order_res 가 정의되는 경우 / 그렇지 않은 경우.
        # if error_code in ['-2022', '-4003']:
        #     self.sys_log.info("order_market executed.")
        #     return
        # else:
            # price_close_executed_list, quantity_close_executed_list, _ = self.order_cancel_multiple([self.order_res], order_type=OrderType.MARKET) # this line uses OrderType.MARKET!
            # self.get_order_info(self) # order_cancel_multiple contains this func.

        self.symbol = self.order_res['symbol'] # we have symbol column in table already.
        self.orderId = self.order_res['orderId']
        self.get_order_info(self,)
        
        if self.order_info['status'] == 'FILLED':
            self.sys_log.info("order_market filled.")
            return 
        else:
            self.sys_log.info("order_market failed.")
            continue

##### v2.0

In [ ]:

def order_market(self, order_res_list):

    """
    v2.0
        update retry_count
        
    last confirmed at, 20240418 1308.
    """

    while 1:

        # get quantity_close_remain
        _, quantity_close_executed_list, quantity_close_orig_list = self.order_cancel_multiple(self, order_res_list) # this order_res is about order_close (= remain + executed).

        quantity_close_remain = np.sum(quantity_close_orig_list) - np.sum(quantity_close_executed_list)
        self.sys_log.info('quantity_close_remain : {}'.format(quantity_close_remain))

            # get price, volume precision
                # realtime precision considering price fluctuation.
        self.get_precision(self)
        self.sys_log.info('precision_quantity : {}'.format(self.precision_quantity))
        
        quantity_close_remain = self.calc_with_precision(quantity_close_remain, self.precision_quantity)
        self.sys_log.info('quantity_close_remain (adj. precision) : {}'.format(quantity_close_remain))


        retry_count = 0
        error_code = 0
        close_remain_quantity_org = quantity_close_remain
        while 1:
            try:
                server_time = self.time()['serverTime']
                self.order_res = self.new_order(symbol=self.symbol,
                                                side=self.side_close,
                                                positionSide=self.side_position,
                                                type=OrderType.MARKET,
                                                quantity=str(quantity_close_remain),
                                                timestamp=server_time)
            except Exception as e:

                # errors
                    # a. -2022 ReduceOnly Order is rejected
                if '-2022' in str(e):
                    error_code = '-2022'
                    break

                    # b. -4003 quantity less than zero
                if '-4003' in str(e):
                    error_code = '-4003'
                    break

                # 2022, -4003 은 logging 하지않는다.
                msg = "error in close order : {}".format(e)
                self.sys_log.error(msg)
                self.push_msg(self, msg)

                # c. -1111 : Precision is over the maximum defined for this asset
                    # = quantity precision error
                    # 상당히 오래전에 만든 대응법인데, 이게 맞는 걸까.. (Todo)
                if '-1111' in str(e):
                    try:
                        # _, precision_quantity = self.get_precision()
                        precision_quantity = retry_count

                        # quantity_close_remain = get_remaining_quantity(self.symbol)
                        #   error 발생한 경우 정상 체결된 quantity 는 존재하지 않는다고 가정 - quantity_close_remain 에는 변함이 없을거라 봄
                        quantity_close_remain = self.calc_with_precision(close_remain_quantity_org, precision_quantity, def_type='floor')
                        self.sys_log.info("modified quantity & precision : {} {}".format(quantity_close_remain, precision_quantity))

                    except Exception as e:
                        msg = "error in get modified precision_quantity : {}".format(e)
                        self.sys_log.error(msg)
                        self.push_msg(self, msg)

                    retry_count += 1
                    if retry_count >= 10:
                        retry_count = 0

                # self.config.out_set.out_type = OrderType.MARKET  # just 재시도
                time.sleep(self.config.trader_set.order_term)
                continue

            else:
                self.sys_log.info("order_market succeed. : {}".format(self.order_res))
                # break

        # 4. term for quantity consumption.
        time.sleep(1)

        # 5.  order_res 가 정의되는 경우 / 그렇지 않은 경우.
        if error_code in ['-2022', '-4003']:
            self.sys_log.info("order_market executed.")
            return
        else:
            price_close_executed_list, quantity_close_executed_list, _ = self.order_cancel_multiple([self.order_res], order_type=OrderType.MARKET) # this line uses OrderType.MARKET!
            # self.get_order_info(self) # order_cancel_multiple contains this func.
            
            if self.order_info['status'] == "FILLED":
            # if float(self.order_res['origQty']) - market_exec_qty < 1 / (10 ** precision_quantity):
                self.sys_log.info("order_market executed.")
                self.sys_log.info("price_close_executed_list : {}".format(price_close_executed_list))
                return price_close_executed_list
            else:
                self.sys_log.info("order_market execution failed, do reorder.")
                continue

#### get_income_info

In [ ]:
def get_income_info(self, mode="PROD", currency="USDT"):  # 복수 pos 를 허용하기 위한 api 미사용

    """
    v2.0
        Class mode
            get income from table_log.

    last confimred at, 20240504 2007.    
    """

    table_log = self.table_log.astype({'cumQuote' : 'float'})
    table_valid = table_log[(table_log.code == self.code) & (table_log.cumQuote != 0)]
    
    cumQuote_open = table_valid[table_valid.order == 1].cumQuote.sum()
    cumQuote_close = table_valid[table_valid.order == 0].cumQuote.sum()
    self.sys_log.info("cumQuote_open : {}".format(cumQuote_open))
    self.sys_log.info("cumQuote_close : {}".format(cumQuote_close))
    
    self.income = cumQuote_close - cumQuote_open
    self.income_accumulated += self.income
    self.sys_log.info("income : {:.2f} {}".format(self.income, currency))
    self.sys_log.info("income_accumulated : {:.2f} {}".format(self.income_accumulated, currency))

    if cumQuote_open == 0:
        self.profit = 1.00
    else:
        self.profit = cumQuote_close / cumQuote_open
    self.sys_log.info("profit : {:.5f}".format(self.profit))
    
    if mode == "PROD":
        # add your income.
        self.balance_available += self.income
        
        # update config.
        self.config.trader_set.initial_asset = self.balance_available

        with open(self.path_config, 'w') as cfg:
            json.dump(self.config, cfg, indent=2)
            
    msg = ("cumQuote_open : {:.2f}\n" + 
            "cumQuote_close : {:.2f}\n" + 
            "income : {:.2f}\n" + 
            "income_accumulated : {:.2f}\n" + 
            "profit : {:.2f}\n")\
            .format(cumQuote_open,
                    cumQuote_close,
                    self.income,
                    self.income_accumulated,
                    self.profit)

    self.push_msg(self, msg)

# Others

## expiry

### v1.1

In [ ]:

def expiry_tp(df_res, config, op_idx, e_j, tp_j, np_datas, side_open):
    
    """
    v1.0
        expiration criteria : tp_1 with tp_gap. 
    v1.1
        expiration criteria : self.price_take_profit with tp_gap. 
    
    last confirmed at, 20240412 2031
    """
    
    high, low = np_datas
    selection_id = config.selection_id
    expire = 0

    if config.tr_set.expire_tick != "None":
        if e_j - op_idx >= config.tr_set.expire_tick:
            expire = 1

    if config.tr_set.expire_k1 != "None":
        if side_open == OrderSide.SELL:
            short_tp_ = df_res['short_tp_{}'.format(selection_id)].to_numpy()  # id 에 따라 dynamic 변수라 이곳에서 numpy 화 진행
            short_tp_gap_ = df_res['short_tp_gap_{}'.format(selection_id)].to_numpy()
            if low[e_j] <= short_tp_[tp_j] + short_tp_gap_[tp_j] * config.tr_set.expire_k1:
                expire = 1
        else:
            long_tp_ = df_res['long_tp_{}'.format(
                selection_id)].to_numpy()  # iloc 이 빠를까, to_numpy() 가 빠를까  # 3.94 ms --> 5.34 ms (iloc)
            long_tp_gap_ = df_res['long_tp_gap_{}'.format(selection_id)].to_numpy()
            if high[e_j] >= long_tp_[tp_j] - long_tp_gap_[tp_j] * config.tr_set.expire_k1:
                expire = 1

    return expire

### v1.0

In [ ]:
def expiry(df_res, config, op_idx, e_j, tp_j, np_datas, side_open):

    """
    v1.0
        expiration criteria : tp_1 with tp_gap. 
    
    last confirmed at, 20240412 2030
    """
    
    high, low = np_datas
    selection_id = config.selection_id
    expire = 0

    if config.tr_set.expire_tick != "None":
        if e_j - op_idx >= config.tr_set.expire_tick:
            expire = 1

    if config.tr_set.expire_k1 != "None":
        if side_open == OrderSide.SELL:
            short_tp_1_ = df_res['short_tp_1_{}'.format(selection_id)].to_numpy()  # id 에 따라 dynamic 변수라 이곳에서 numpy 화 진행
            short_tp_gap_ = df_res['short_tp_gap_{}'.format(selection_id)].to_numpy()
            if low[e_j] <= short_tp_1_[tp_j] - short_tp_gap_[tp_j] * config.tr_set.expire_k1:
                expire = 1
        else:
            long_tp_1_ = df_res['long_tp_1_{}'.format(
                selection_id)].to_numpy()  # iloc 이 빠를까, to_numpy() 가 빠를까  # 3.94 ms --> 5.34 ms (iloc)
            long_tp_gap_ = df_res['long_tp_gap_{}'.format(selection_id)].to_numpy()
            if high[e_j] >= long_tp_1_[tp_j] + long_tp_gap_[tp_j] * config.tr_set.expire_k1:
                expire = 1

    return expire

## expiry_between

### v1.0

In [ ]:

def expiry_between(df_res, config, op_idx1, op_idx2, tp1, tp0, tp_gap, np_datas, side_open):
    
    """
    v1.0
        expiration survey between op_idx1 & op_idx2.
    
    last confirmed at, 20240412 2035
    """

    high, low = np_datas
    selection_id = config.selection_id
    expire = 0
    touch_idx = None

    # if config.tr_set.expire_tick != "None":
    #     if e_j - op_idx >= config.tr_set.expire_tick:
    #         expire = 1

    # Todo, p1's tp1, 0 cannot be vectorized
    #   a. expiration 의 조건은 wave1, 0 의 broken
    idx_range = np.arange(op_idx1, op_idx2)
    if config.tr_set.expire_k1 != "None":
        if side_open == OrderSide.SELL:
            touch_idx = np.where((low[op_idx1:op_idx2] <= tp1 + tp_gap * config.tr_set.expire_k1) | \
                                 (high[op_idx1:op_idx2] >= tp0 - tp_gap * config.tr_set.expire_k1),
                                 idx_range, np.nan)
            # if op_idx1 >= 16353:
            #   print("high[16353], tp0 :", high[16353], tp0)
            if np.sum(~np.isnan(touch_idx)) > 0:  # touch 가 존재하면,
                # if low[op_idx1:op_idx2].min() <= tp1 + tp_gap * config.tr_set.expire_k1 or \
                # high[op_idx1:op_idx2].max() >= tp0 - tp_gap * config.tr_set.expire_k1:   # p2_box loc. 이 있어서, op_idx2 + 1 안함
                expire = 1
        else:
            touch_idx = np.where((high[op_idx1:op_idx2] >= tp1 - tp_gap * config.tr_set.expire_k1) | \
                                 (low[op_idx1:op_idx2] <= tp0 + tp_gap * config.tr_set.expire_k1),
                                 idx_range, np.nan)
            if np.sum(~np.isnan(touch_idx)) > 0:
                # if high[op_idx1:op_idx2].max() >= tp1 - tp_gap * config.tr_set.expire_k1 or \
                # low[op_idx1:op_idx2].min() <= tp0 + tp_gap * config.tr_set.expire_k1:
                expire = 1

    return expire, np.nanmin(touch_idx)